# Define Library

In [14]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini_for_threedigitscore

In [15]:
# def calculate_gini_for_threedigitscore(scores, labels):
#     """
#     Calculate Gini coefficient for three-digit scores and binary labels
    
#     Parameters:
#     scores: array-like, three-digit scores (higher is better)
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Combine scores and labels into a DataFrame
#     df = pd.DataFrame({'score': scores, 'label': labels})
    
#     # Sort by score in descending order (assuming higher score is better)
#     df = df.sort_values('score', ascending=False)
    
#     # Calculate cumulative values
#     total_pos = df['label'].sum()
#     total_neg = len(df) - total_pos
    
#     if total_pos == 0 or total_neg == 0:
#         return 0
    
#     # Calculate cumulative proportions
#     cum_pos = df['label'].cumsum()
#     cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
#     # Convert to proportions
#     cum_pos_prop = cum_pos / total_pos
#     cum_neg_prop = cum_neg / total_neg
    
#     # Calculate Gini
#     gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
    
#     return gini


## Modified one

def calculate_gini_for_threedigitscore(scores, labels):
    """
    Calculate Gini coefficient for three-digit scores and binary labels
    
    Parameters:
    scores: array-like, three-digit scores (higher is better)
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Combine scores and labels into a DataFrame
    df = pd.DataFrame({'score': scores, 'label': labels})
    
    # Sort by score in descending order (assuming higher score means lower risk)
    # For default prediction, we want to sort scores in ascending order 
    # since higher default probability should correspond to higher risk
    df = df.sort_values('score', ascending=True)  # Changed to ascending=True
    
    # Calculate cumulative values
    total_pos = df['label'].sum()
    total_neg = len(df) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0
    
    # Calculate cumulative proportions
    cum_pos = df['label'].cumsum()
    cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
    # Convert to proportions
    cum_pos_prop = cum_pos / total_pos
    cum_neg_prop = cum_neg / total_neg
    
    # Calculate area under curve
    auc = np.trapz(cum_pos_prop, cum_neg_prop)
    
    # Calculate Gini
    gini = 2 * auc - 1
    
    return gini

## calculate_gini

In [16]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_hybrid_gini

In [17]:
# def calculate_hybrid_gini(scores, labels):
#     """
#     Calculate Gini coefficient handling both PD values and three-digit scores
    
#     Parameters:
#     scores: array-like, contains either PD values (0-1) or three-digit scores
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Convert inputs to numpy arrays
#     scores = np.array(scores, dtype=float)
#     labels = np.array(labels, dtype=int)
    
#     # Basic validation
#     if len(scores) == 0 or len(labels) == 0:
#         return np.nan
    
#     if len(np.unique(labels)) < 2:
#         return np.nan
        
#     # Determine if scores are PD values or three-digit scores
#     # PD values are between 0 and 1
#     is_pd = np.all((scores >= 0) & (scores <= 1))
    
#     if is_pd:
#         try:
#             auc = roc_auc_score(labels, scores)
#             gini = 2 * auc - 1
#             return gini
#         except ValueError:
#             return np.nan
#     else:
#         # Handle as three-digit score
#         df = pd.DataFrame({'score': scores, 'label': labels})
#         df = df.sort_values('score', ascending=False)
        
#         total_pos = df['label'].sum()
#         total_neg = len(df) - total_pos
        
#         if total_pos == 0 or total_neg == 0:
#             return np.nan
        
#         cum_pos = df['label'].cumsum()
#         cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
#         cum_pos_prop = cum_pos / total_pos
#         cum_neg_prop = cum_neg / total_neg
        
#         gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
#         return gini

## Modified one

def calculate_hybrid_gini(scores, labels):
    """
    Calculate Gini coefficient handling both PD values and three-digit scores
    
    Parameters:
    scores: array-like, contains either PD values (0-1) or three-digit scores
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays
    scores = np.array(scores, dtype=float)
    labels = np.array(labels, dtype=int)
    
    # Basic validation
    if len(scores) == 0 or len(labels) == 0:
        return np.nan
    
    if len(np.unique(labels)) < 2:
        return np.nan
        
    # Determine if scores are PD values or three-digit scores
    # PD values are between 0 and 1
    is_pd = np.all((scores >= 0) & (scores <= 1))
    
    if is_pd:
        try:
            auc = roc_auc_score(labels, scores)
            gini = 2 * auc - 1
            return gini
        except ValueError:
            return np.nan
    else:
        # Handle as three-digit score
        df = pd.DataFrame({'score': scores, 'label': labels})
        # Sort by score in ascending order since higher score means higher risk
        df = df.sort_values('score', ascending=True)
        
        total_pos = df['label'].sum()
        total_neg = len(df) - total_pos
        
        if total_pos == 0 or total_neg == 0:
            return np.nan
        
        cum_pos = df['label'].cumsum()
        cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
        cum_pos_prop = cum_pos / total_pos
        cum_neg_prop = cum_neg / total_neg
        
        # Calculate area under curve
        auc = np.trapz(cum_pos_prop, cum_neg_prop)
        
        # Calculate Gini using the same formula as PD values
        gini = 2 * auc - 1
        return gini

## calculate_periodic_gini_threedigit

In [18]:
# Main processing code
def calculate_periodic_gini_threedigit(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini

In [19]:
def calculate_periodic_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_hybrid_gini

In [20]:
def calculate_periodic_hybrid_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients for mixed score types
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + pd.Timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini_debug

In [21]:
def calculate_periodic_gini_debug(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients with detailed debugging
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    print(f"Original disbursementdate dtype: {df['disbursementdate'].dtype}")
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    print(f"Converted disbursementdate dtype: {df['disbursementdate'].dtype}")
    
    # Ensure score and label columns are numeric
    print(f"\nBefore conversion:")
    print(f"  {score_column} dtype: {df[score_column].dtype}, non-null: {df[score_column].notna().sum()}")
    print(f"  {label_column} dtype: {df[label_column].dtype}, non-null: {df[label_column].notna().sum()}")
    
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    print(f"\nAfter conversion:")
    print(f"  {score_column} - non-null: {df[score_column].notna().sum()}, NaN: {df[score_column].isna().sum()}")
    print(f"  {label_column} - non-null: {df[label_column].notna().sum()}, NaN: {df[label_column].isna().sum()}")
    print(f"  {score_column} sample values: {df[score_column].dropna().head()}")
    print(f"  {label_column} unique values: {df[label_column].dropna().unique()}")
    
    # Drop rows with invalid values
    initial_rows = len(df)
    df = df.dropna(subset=[score_column, label_column])
    print(f"\nRows dropped: {initial_rows - len(df)}")
    print(f"Remaining rows: {len(df)}")
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_groups = df.groupby('week').size()
    print(f"\nWeekly groups: {len(weekly_groups)} weeks")
    print(f"Weekly group sizes:\n{weekly_groups.describe()}\n{weekly_groups.value_counts().head()}")
    print(f"Weeks with < 10 samples: {(weekly_groups < 10).sum()}")
    
    weekly_results = []
    for week, group_df in df.groupby('week'):
        if len(group_df) >= 10:
            gini_val = calculate_gini(group_df[score_column], group_df[label_column])
            print(f"  Week {week}: n={len(group_df)}, gini={gini_val}")
            weekly_results.append({'week': week, 'gini': gini_val, 'count': len(group_df)})
        else:
            print(f"  Week {week}: n={len(group_df)} - SKIPPED (< 10 samples)")
    
    weekly_gini = pd.DataFrame(weekly_results) if weekly_results else pd.DataFrame()
    
    if len(weekly_gini) > 0:
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_groups = df.groupby('month').size()
    print(f"\nMonthly groups: {len(monthly_groups)} months")
    print(f"Monthly group sizes:\n{monthly_groups.describe()}\n{monthly_groups.value_counts().head()}")
    print(f"Months with < 20 samples: {(monthly_groups < 20).sum()}")
    
    monthly_results = []
    for month, group_df in df.groupby('month'):
        if len(group_df) >= 20:
            gini_val = calculate_gini(group_df[score_column], group_df[label_column])
            print(f"  Month {month}: n={len(group_df)}, gini={gini_val}")
            monthly_results.append({'month': month, 'gini': gini_val, 'count': len(group_df)})
        else:
            print(f"  Month {month}: n={len(group_df)} - SKIPPED (< 20 samples)")
    
    monthly_gini = pd.DataFrame(monthly_results) if monthly_results else pd.DataFrame()
    
    if len(monthly_gini) > 0:
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine results
    if len(weekly_gini) > 0 and len(monthly_gini) > 0:
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
    elif len(weekly_gini) > 0:
        gini_results = weekly_gini.copy()
    elif len(monthly_gini) > 0:
        gini_results = monthly_gini.copy()
    else:
        print("\n⚠️  WARNING: No valid Gini calculations produced!")
        return pd.DataFrame()
    
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    print(f"\n✓ Final results: {len(gini_results)} rows")
    return gini_results

## calculate_periodic_gini_producttype

In [22]:
def calculate_periodic_gini_producttype(df, score_column, label_column, namecolumn, product_column=None):
    """
    Calculate periodic Gini coefficients overall and by product type
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    product_column: (optional) name of product type column to segment by
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    if product_column and product_column not in df.columns:
        raise ValueError(f"Product column '{product_column}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process: overall + each product type
    datasets_to_process = [('Overall', df)]
    
    if product_column:
        for product_type in df[product_column].unique():
            if pd.notna(product_type):
                product_df = df[df[product_column] == product_type]
                datasets_to_process.append((str(product_type), product_df))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['version'] = '1.1.0'
        gini_results['bad_rate'] = namecolumn
        gini_results['loan_type'] = dataset_name
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# Calculate overall + by product type
# gini_results = calculate_periodic_gini(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     product_column='new_loan_type'
# )

# Filter results by product type if needed:
# overall_gini = gini_results[gini_results['product_type'] == 'Overall']
# sil_instore_gini = gini_results[gini_results['product_type'] == 'sil_instore']
# sil_zero_gini = gini_results[gini_results['product_type'] == 'sil_zero']

# calculate_periodic_gini_prod_ver_trench

In [23]:
def calculate_periodic_gini_prod_ver_trench(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, product_column=None):
    """
    Calculate periodic Gini coefficients overall and by product type
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column to extract model version from
    trench_column: (optional) name of column to extract trench category from
    product_column: (optional) name of product type column to segment by
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    if product_column and product_column not in df.columns:
        raise ValueError(f"Product column '{product_column}' not found in dataframe")
    
    if model_version_column and model_version_column not in df.columns:
        raise ValueError(f"Model version column '{model_version_column}' not found in dataframe")
    
    if trench_column and trench_column not in df.columns:
        raise ValueError(f"Trench column '{trench_column}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Get version and trench values (take first non-null value as they should be consistent)
    version_value = df[model_version_column].iloc[0] if model_version_column else '1.1.0'
    trench_value = df[trench_column].iloc[0] if trench_column else None
    
    # Define list of datasets to process: overall + each product type
    datasets_to_process = [('Overall', df)]
    
    if product_column:
        for product_type in df[product_column].unique():
            if pd.notna(product_type):
                product_df = df[df[product_column] == product_type]
                datasets_to_process.append((str(product_type), product_df))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['version'] = version_value
        gini_results['bad_rate'] = namecolumn
        gini_results['loan_type'] = dataset_name
        if trench_column:
            gini_results['trench_category'] = trench_value
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_producttype(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     product_column='new_loan_type'
# )

# SIL

# Alpha - CIC-SIL-Model

## FPD0

## Test

In [24]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2409688,52d88049-e14d-416f-a753-cf688c92505e,60824096880026,0.09502879564284657,2025-04-22 10:06:11,2025-04-22,2025-04,Test,0,1,SIL-Instore
1,3394652,7905e5b8-7972-4e7e-8e70-4679c9742e54,60833946520012,0.12556073087979863,2025-04-22 10:13:35,2025-04-22,2025-04,Test,0,1,SIL ZERO
2,3394750,e616ff74-08bc-47c9-be2a-50c88a756c03,60833947500013,0.1261505375815017,2025-04-22 11:20:26,2025-04-22,2025-04,Test,1,1,SIL-Instore
3,3394813,258c2fda-c6f2-42a7-9f0c-66464c40d0b1,60833948130011,0.08865829035945388,2025-04-22 11:51:51,2025-04-22,2025-04,Test,0,1,SIL-Instore
4,3394903,d1444685-888f-4cee-9261-c3f6cf014464,60833949030012,0.09893965440661757,2025-04-22 12:14:46,2025-04-22,2025-04,Test,1,1,SIL-Instore


In [25]:
df1 = dfd.copy()

## Train

In [26]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2147386,8ae91d78-b90e-44cf-8446-aa150e9ba327,60821473860011,0.158327,2023-07-20 17:59:02,2023-07-20,2023-07,Train,0,1,SIL-Instore
1,2026999,d77daee1-feba-4b95-ac76-403e42f2129e,60820269990014,0.062890,2023-05-03 19:02:47,2023-05-03,2023-05,Train,0,1,SIL-Instore
2,1917130,74cbc1fc-bb2d-48f6-afdf-a66443ffa99e,60819171300015,0.151771,2023-02-25 18:03:36,2023-02-25,2023-02,Train,0,1,SIL-Instore
3,1936082,0198fe8d-bad5-469d-bc3c-03615f501a47,60819360820011,0.180175,2023-03-09 18:02:46,2023-03-09,2023-03,Train,0,1,SIL-Instore
4,1969006,adbde58f-ddad-4deb-beee-43dc2e106b31,60819690060015,0.125483,2023-03-30 18:09:41,2023-03-30,2023-03,Train,0,1,SIL-Instore


In [27]:
df2 = dfd.copy()

In [28]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177174 entries, 0 to 177173
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             177174 non-null  object        
 1   digitalLoanAccountId   177174 non-null  object        
 2   loanAccountNumber      177174 non-null  object        
 3   Alpha_cic_sil_score    177174 non-null  object        
 4   appln_submit_datetime  177174 non-null  datetime64[us]
 5   disbursementdate       177174 non-null  dbdate        
 6   Application_month      177174 non-null  object        
 7   Data_selection         177174 non-null  object        
 8   deffpd0                177174 non-null  Int64         
 9   flg_mature_fpd0        177174 non-null  Int64         
 10  new_loan_type          177174 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 15.2+ MB


In [29]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')

In [30]:
# df_concat.to_csv(r"Alpha_cic_sil_score.csv")

In [31]:
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd0', 'FPD0')

gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [32]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,Alpha_cic_sil_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.205187,Month,Alpha_cic_sil_score,1.1.0,FPD0,Overall
1,2023-01-09,2023-01-15,0.226648,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall
2,2023-01-16,2023-01-22,0.375000,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall
3,2023-01-23,2023-01-29,0.110811,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall
4,2023-01-30,2023-02-05,0.076412,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall


In [33]:
f0.groupby(['loan_type', 'period']).size()

loan_type       period
Overall         Month      34
                Week      144
SIL Competitor  Month      10
                Week       38
SIL Repeat      Month       3
                Week       11
SIL ZERO        Month      15
                Week       63
SIL-Instore     Month      34
                Week      144
dtype: int64

## FPD10

## Test

In [34]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd10 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2299863,9670f638-218d-42b4-a1d2-a043c6a9e2e3,60822998630026,0.06427148811414114,2025-07-15 14:10:11,2025-07-15,2025-07,Test,0,1,SIL-Instore
1,3091104,5f4cdfc0-5c41-4d7f-8a21-d72990f50d60,60830911040019,0.10458504085585472,2025-07-15 14:27:35,2025-07-15,2025-07,Test,0,1,SIL-Instore
2,3094488,a5c783c8-8a0d-4925-b5ef-d57a3d59dac7,60830944880028,0.11988427880389088,2025-07-15 13:41:22,2025-07-15,2025-07,Test,0,1,SIL Competitor
3,3291752,99c4c151-6393-4710-a13d-5addf985a258,60832917520017,0.1546931588748243,2025-07-15 13:28:29,2025-07-15,2025-07,Test,0,1,SIL-Instore
4,3558152,3ada23b4-913e-40ba-a555-1bc4c8fe48de,60835581520015,0.07952602505964143,2025-07-15 14:33:16,2025-07-15,2025-07,Test,0,1,SIL Competitor


In [35]:
df1 = dfd.copy()

## Train

In [36]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd10,
  flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2078384,852fe180-cf5c-470f-81b1-4807a6ccbf21,60820783840012,0.103348,2023-06-05 10:30:59,2023-06-05,2023-06,Train,0,1,SIL-Instore
1,2070839,99cfbd15-2925-4157-a02d-f5d418d8c5ae,60820708390011,0.170400,2023-05-31 17:58:05,2023-05-31,2023-05,Train,0,1,SIL-Instore
2,2112503,d37ea61d-58cb-4ce8-840f-fcfa7118ba4e,60821125030013,0.187252,2023-07-09 18:12:13,2023-07-09,2023-07,Train,0,1,SIL-Instore
3,1927439,b62381c2-7d20-483f-84bd-860b296bb908,60819274390018,0.153677,2023-03-04 12:13:36,2023-03-04,2023-03,Train,0,1,SIL-Instore
4,2024358,931337ac-3fde-4e10-8bce-271236bc1376,60820243580019,0.097841,2023-05-02 11:55:27,2023-05-02,2023-05,Train,0,1,SIL-Instore


In [37]:
df2 = dfd.copy()

In [38]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174121 entries, 0 to 174120
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             174121 non-null  object        
 1   digitalLoanAccountId   174121 non-null  object        
 2   loanAccountNumber      174121 non-null  object        
 3   Alpha_cic_sil_score    174121 non-null  object        
 4   appln_submit_datetime  174121 non-null  datetime64[us]
 5   disbursementdate       174121 non-null  dbdate        
 6   Application_month      174121 non-null  object        
 7   Data_selection         174121 non-null  object        
 8   deffpd10               174121 non-null  Int64         
 9   flg_mature_fpd10       174121 non-null  Int64         
 10  new_loan_type          174121 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 14.9+ MB


In [39]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [40]:
f1=gini_results.copy()
f1.groupby('loan_type').size()

loan_type
Overall           177
SIL Competitor     47
SIL Repeat         14
SIL ZERO           77
SIL-Instore       177
dtype: int64

## FPD30

## Test

In [41]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,3427347,64e064c7-6b8a-4667-9f1b-7ccb83e9801a,60834273470019,0.1235626369093496,2025-05-09 10:31:57,2025-05-09,2025-05,Test,1,1,SIL-Instore
1,3427362,46e645cf-e27f-46d3-a1b3-0f6ef7ac026e,60834273620018,0.12675888493226176,2025-05-09 10:41:37,2025-05-09,2025-05,Test,0,1,SIL-Instore
2,3427559,b31d973a-d2c6-4737-a4c1-08a68eed7175,60834275590019,0.0959470730957629,2025-05-09 12:37:21,2025-05-09,2025-05,Test,0,1,SIL ZERO
3,3427785,a6a64657-3368-40c1-800e-439fbb18e743,60834277850017,0.04627600160530132,2025-05-09 14:08:17,2025-05-09,2025-05,Test,0,1,SIL-Instore
4,3427802,dab537ad-66a9-41e4-ad8d-f5409181609f,60834278020011,0.0795958691511927,2025-05-09 14:18:01,2025-05-09,2025-05,Test,0,1,SIL-Instore


In [42]:
df1 = dfd.copy()

## Train

In [43]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd30,
  flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2136992,2d351f42-66bc-4bdc-9d4d-6d097d78f327,60821369920018,0.107030,2023-07-14 16:38:28,2023-07-14,2023-07,Train,0,1,SIL-Instore
1,2000372,eb168e03-b1e5-4f57-b942-1e1792d5958b,60820003720011,0.125541,2023-04-17 18:33:21,2023-04-17,2023-04,Train,0,1,SIL-Instore
2,1896258,fb858339-ed84-491c-b650-e50fd06546d1,60818962580011,0.056549,2023-02-11 15:29:16,2023-02-11,2023-02,Train,0,1,SIL-Instore
3,2082290,887818c0-2eb4-45eb-a44d-3e50533e5f77,60820822900018,0.056862,2023-06-07 16:04:59,2023-06-07,2023-06,Train,1,1,SIL-Instore
4,1927192,fb894c1a-6ce9-4bfa-a2b4-288143f26b20,60819271920014,0.069848,2023-03-04 17:39:55,2023-03-04,2023-03,Train,0,1,SIL-Instore


In [44]:
df2 = dfd.copy()

In [45]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167483 entries, 0 to 167482
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             167483 non-null  object        
 1   digitalLoanAccountId   167483 non-null  object        
 2   loanAccountNumber      167483 non-null  object        
 3   Alpha_cic_sil_score    167483 non-null  object        
 4   appln_submit_datetime  167483 non-null  datetime64[us]
 5   disbursementdate       167483 non-null  dbdate        
 6   Application_month      167483 non-null  object        
 7   Data_selection         167483 non-null  object        
 8   deffpd30               167483 non-null  Int64         
 9   flg_mature_fpd30       167483 non-null  Int64         
 10  new_loan_type          167483 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 14.4+ MB


In [46]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffpd30', 'FPD30')


gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [47]:
f2=gini_results.copy()
f2.groupby('loan_type').size()

loan_type
Overall           173
SIL Competitor     43
SIL Repeat         14
SIL ZERO           73
SIL-Instore       173
dtype: int64

## FSPD30

## Test

In [48]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fspd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2348115,fc4ed5d2-59d3-466c-9145-64b836194fe0,60823481150024,0.13074170816439212,2025-07-18 10:15:02,2025-07-18,2025-07,Test,0,1,SIL-Instore
1,3211730,950b81d6-eba1-447a-a35b-2d46d5dfd5ea,60832117300029,0.12644143154379356,2025-07-18 10:56:02,2025-07-18,2025-07,Test,0,1,SIL-Instore
2,3265761,67099df5-418c-43c4-a3e9-d94d30e9bac9,60832657610027,0.11579772254567665,2025-07-18 10:06:04,2025-07-18,2025-07,Test,0,1,SIL-Instore
3,3563305,a4645fea-8d17-4bea-a414-255f2e0c9063,60835633050019,0.12151312296648767,2025-07-18 08:43:55,2025-07-18,2025-07,Test,0,1,SIL-Instore
4,3563314,a590022b-d03c-48e0-bd74-64ee44ed83f2,60835633140011,0.09684524088748465,2025-07-18 09:37:36,2025-07-18,2025-07,Test,0,1,SIL-Instore


In [49]:
df1 = dfd.copy()

## Train

In [50]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffspd30,
  flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fspd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,1981090,10c329bd-9f22-4fc9-b0ec-6d6c2e049f0a,60819810900015,0.126058,2023-04-06 15:28:26,2023-04-06,2023-04,Train,0,1,SIL-Instore
1,2058617,15aad111-62a6-4148-ad81-7536b7a4ae5a,60820586170011,0.170536,2023-05-23 19:18:51,2023-05-23,2023-05,Train,0,1,SIL-Instore
2,2047500,4f23df1f-3c7f-440a-a566-63cb69197acd,60820475000019,0.123255,2023-05-16 18:52:37,2023-05-16,2023-05,Train,0,1,SIL-Instore
3,1883685,f36e891d-0e0f-4124-bf84-679795d04928,60818836850018,0.077717,2023-02-02 17:16:23,2023-02-02,2023-02,Train,1,1,SIL-Instore
4,1985125,881e8658-28e9-4207-8473-8952ef44af32,60819851250015,0.091860,2023-04-09 11:46:04,2023-04-09,2023-04,Train,0,1,SIL-Instore


In [51]:
df2 = dfd.copy()

In [52]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158529 entries, 0 to 158528
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             158529 non-null  object        
 1   digitalLoanAccountId   158529 non-null  object        
 2   loanAccountNumber      158529 non-null  object        
 3   Alpha_cic_sil_score    158529 non-null  object        
 4   appln_submit_datetime  158529 non-null  datetime64[us]
 5   disbursementdate       158529 non-null  dbdate        
 6   Application_month      158529 non-null  object        
 7   Data_selection         158529 non-null  object        
 8   deffspd30              158529 non-null  Int64         
 9   flg_mature_fspd_30     158529 non-null  Int64         
 10  new_loan_type          158529 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 13.6+ MB


In [53]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [54]:
f3=gini_results.copy()

## FSTPD30

## Test

In [55]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fstpd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,3430606,bf4becd6-fc74-4328-aff6-bcf22b1fc976,60834306060016,0.12554077753482806,2025-05-11 08:55:52,2025-05-11,2025-05,Test,1,1,SIL-Instore
1,3391003,840210ee-0e26-476d-8ad9-d57601dbaaf7,60833910030013,0.11805426217171301,2025-04-20 14:24:05,2025-04-21,2025-04,Test,0,1,SIL-Instore
2,3348923,8529c50c-4e0a-447c-b159-7c552f0b2979,60833489230017,0.16081496226997127,2025-03-30 13:11:29,2025-03-30,2025-03,Test,0,1,SIL-Instore
3,3410174,04e272da-568b-4a10-b1d2-6591ceb45220,60834101740013,0.22705334666018648,2025-04-30 10:12:31,2025-04-30,2025-04,Test,0,1,SIL-Instore
4,3401493,7009f759-85b2-4d92-8f0a-c3eb1c7549fd,60834014930014,0.10475712037266423,2025-04-25 15:39:31,2025-04-27,2025-04,Test,1,1,SIL-Instore


In [56]:
df1 = dfd.copy()

## Train

In [57]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffstpd30,
  flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fstpd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2078384,852fe180-cf5c-470f-81b1-4807a6ccbf21,60820783840012,0.103348,2023-06-05 10:30:59,2023-06-05,2023-06,Train,0,1,SIL-Instore
1,2070839,99cfbd15-2925-4157-a02d-f5d418d8c5ae,60820708390011,0.170400,2023-05-31 17:58:05,2023-05-31,2023-05,Train,0,1,SIL-Instore
2,2112503,d37ea61d-58cb-4ce8-840f-fcfa7118ba4e,60821125030013,0.187252,2023-07-09 18:12:13,2023-07-09,2023-07,Train,0,1,SIL-Instore
3,1927439,b62381c2-7d20-483f-84bd-860b296bb908,60819274390018,0.153677,2023-03-04 12:13:36,2023-03-04,2023-03,Train,0,1,SIL-Instore
4,2024358,931337ac-3fde-4e10-8bce-271236bc1376,60820243580019,0.097841,2023-05-02 11:55:27,2023-05-02,2023-05,Train,0,1,SIL-Instore


In [58]:
df2 = dfd.copy()

In [59]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148228 entries, 0 to 148227
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             148228 non-null  object        
 1   digitalLoanAccountId   148228 non-null  object        
 2   loanAccountNumber      148228 non-null  object        
 3   Alpha_cic_sil_score    148228 non-null  object        
 4   appln_submit_datetime  148228 non-null  datetime64[us]
 5   disbursementdate       148228 non-null  dbdate        
 6   Application_month      148228 non-null  object        
 7   Data_selection         148228 non-null  object        
 8   deffstpd30             148228 non-null  Int64         
 9   flg_mature_fstpd_30    148228 non-null  Int64         
 10  new_loan_type          148228 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 12.7+ MB


In [60]:
df_concat['Alpha_cic_sil_score'] = pd.to_numeric(df_concat['Alpha_cic_sil_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Alpha_cic_sil_score', 'deffstpd30', 'FSTPD30')

gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'  
)

In [61]:
f4 = gini_results.copy()
f4

,start_date,end_date,Alpha_cic_sil_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.091195,Month,Alpha_cic_sil_score,1.1.0,FSTPD30,Overall
1,2023-01-09,2023-01-15,0.175207,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,Overall
2,2023-01-16,2023-01-22,-0.133333,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,Overall
3,2023-01-23,2023-01-29,-0.035088,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,Overall
4,2023-01-30,2023-02-05,0.285088,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,Overall
...,...,...,...,...,...,...,...,...
427,2024-12-23,2024-12-29,0.118359,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,SIL Repeat
428,2024-12-30,2025-01-05,0.234065,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,SIL Repeat
429,2025-01-01,2025-01-31,0.326338,Month,Alpha_cic_sil_score,1.1.0,FSTPD30,SIL Repeat
430,2025-01-06,2025-01-12,0.329517,Week,Alpha_cic_sil_score,1.1.0,FSTPD30,SIL Repeat


## combining the dataframe

In [62]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'Alpha_cic_sil_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'Alpha_cic_sil_score_FPD10_gini', 'Alpha_cic_sil_score_FPD30_gini',
       'Alpha_cic_sil_score_FSPD30_gini',
       'Alpha_cic_sil_score_FSTPD30_gini'], dtype=object)

In [63]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate', 'loan_type','Alpha_cic_sil_score_FPD0_gini','Alpha_cic_sil_score_FPD10_gini', 'Alpha_cic_sil_score_FPD30_gini',  'Alpha_cic_sil_score_FSPD30_gini', 'Alpha_cic_sil_score_FSTPD30_gini']].copy()
final_df['Trench_category'] = 'All'
final_df['Model_display_name'] = 'cic_model_sil'
final_df['Product_type'] = 'SIL'
final_df.dtypes

start_date                          datetime64[ns]
end_date                            datetime64[ns]
period                                      object
Model_Name                                  object
version                                     object
bad_rate                                    object
loan_type                                   object
Alpha_cic_sil_score_FPD0_gini              float64
Alpha_cic_sil_score_FPD10_gini             float64
Alpha_cic_sil_score_FPD30_gini             float64
Alpha_cic_sil_score_FSPD30_gini            float64
Alpha_cic_sil_score_FSTPD30_gini           float64
Trench_category                             object
Model_display_name                          object
Product_type                                object
dtype: object

In [64]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,loan_type,Alpha_cic_sil_score_FPD0_gini,Alpha_cic_sil_score_FPD10_gini,Alpha_cic_sil_score_FPD30_gini,Alpha_cic_sil_score_FSPD30_gini,Alpha_cic_sil_score_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,Alpha_cic_sil_score,1.1.0,FPD0,Overall,0.205187,NaN,NaN,NaN,NaN,All,cic_model_sil,SIL
1,2023-01-09,2023-01-15,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall,0.226648,NaN,NaN,NaN,NaN,All,cic_model_sil,SIL
2,2023-01-16,2023-01-22,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall,0.375000,NaN,NaN,NaN,NaN,All,cic_model_sil,SIL
3,2023-01-23,2023-01-29,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall,0.110811,NaN,NaN,NaN,NaN,All,cic_model_sil,SIL
4,2023-01-30,2023-02-05,Week,Alpha_cic_sil_score,1.1.0,FPD0,Overall,0.076412,NaN,NaN,NaN,NaN,All,cic_model_sil,SIL


In [65]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.Alpha_cic_sil_score_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=caf884c5-d108-49dd-b8c9-a9acf11a6718>

# Alpha Sil Stack Model

## FPD0

## Test

In [66]:
sq = """ 
with modelname as 
(
  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3477634,63e29ed7-e3cf-432e-b7f0-e176c1e705af,60834776340018,0.08919362060089593,2025-06-04 15:39:13,2025-06-04,2025-06,Test,0,1,SIL-Instore
1,3760584,bc453ff5-dfcf-4438-94d9-86e814a12e7b,60837605840011,0.0988301581490622,2025-10-22 13:48:41,2025-10-22,2025-10,Test,0,1,SIL-Instore
2,3760134,afcb8eac-ae1b-439b-80af-4c38e512e81d,60837601340018,0.06564958292282923,2025-10-22 10:40:44,2025-10-22,2025-10,Test,0,1,SIL ZERO
3,3763392,7513b610-74c3-4941-9d6e-7112687042c2,60837633920016,0.025473939710776525,2025-10-23 16:29:23,2025-10-23,2025-10,Test,0,1,SIL-Instore
4,3765595,614243a6-71d1-460b-96ee-666a574ae4f7,60837655950013,0.05891310486434996,2025-10-24 15:58:46,2025-10-24,2025-10,Test,1,1,SIL Competitor


In [67]:
df1 = dfd.copy()

## Train

In [68]:
sq = """ 
  with modelname as 
  (  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and flg_mature_fpd0 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2014810,1fe12d4a-65bb-46b3-857f-44a9d0d4fcc3,60820148100013,0.052768,2023-04-26 15:24:48,2023-04-26,2023-04,Train,0,1,SIL-Instore
1,1879138,57afc593-0ba7-43dd-82a3-4faab3997aa5,60818791380015,0.045775,2023-01-30 12:24:03,2023-01-30,2023-01,Train,0,1,SIL-Instore
2,2126527,4abc7cd4-dd5d-4c6a-8e52-cb0ef4154b4d,60821265270012,0.020925,2023-07-09 10:19:30,2023-07-09,2023-07,Train,0,1,SIL-Instore
3,1985938,537daa92-3a84-404a-9e6d-9221382cb120,60819859380014,0.048511,2023-04-09 19:16:40,2023-04-09,2023-04,Train,0,1,SIL-Instore
4,2090903,c86335d5-8bfb-4bea-aa74-5600ded7d857,60820909030011,0.083930,2023-06-13 13:35:13,2023-06-13,2023-06,Train,0,1,SIL-Instore


In [69]:
df2 = dfd.copy()

In [70]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177174 entries, 0 to 177173
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             177174 non-null  object        
 1   digitalLoanAccountId   177174 non-null  object        
 2   loanAccountNumber      177174 non-null  object        
 3   Sil_Alpha_Stack_score  177174 non-null  object        
 4   appln_submit_datetime  177174 non-null  datetime64[us]
 5   disbursementdate       177174 non-null  dbdate        
 6   Application_month      177174 non-null  object        
 7   Data_selection         177174 non-null  object        
 8   deffpd0                177174 non-null  Int64         
 9   flg_mature_fpd0        177174 non-null  Int64         
 10  new_loan_type          177174 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 15.2+ MB


In [71]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')

In [72]:
df_concat.to_csv(r"Sil_Alpha_Stack_score.csv")

In [73]:
# gini_results = calculate_periodic_gini(df_concat, 'Sil_Alpha_Stack_score', 'deffpd0', 'FPD0')

gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [74]:
f0 = gini_results.copy()

In [75]:
f0.head(10)

,start_date,end_date,Sil_Alpha_Stack_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.252502,Month,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
1,2023-01-09,2023-01-15,0.343407,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
2,2023-01-16,2023-01-22,0.660714,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
3,2023-01-23,2023-01-29,0.113514,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
4,2023-01-30,2023-02-05,0.182724,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
5,2023-02-01,2023-02-28,0.193609,Month,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
6,2023-02-06,2023-02-12,0.313609,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
7,2023-02-13,2023-02-19,-0.062500,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
8,2023-02-20,2023-02-26,0.272727,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall
9,2023-02-27,2023-03-05,0.522222,Week,Sil_Alpha_Stack_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [76]:
sq = """ 
with modelname as 
(
  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd10 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3415687,888afbb4-c712-4737-b0a4-0b1d609a55a9,60834156870013,0.05579244790068675,2025-05-03 09:22:14,2025-05-03,2025-05,Test,0,1,SIL-Instore
1,3416648,559fed5d-fd7c-4f22-b36a-41eaf837859e,60834166480011,0.09116983926020646,2025-05-03 11:34:50,2025-05-03,2025-05,Test,1,1,SIL-Instore
2,3416761,750dc8ed-8314-4bcb-aaa2-02682ecebf34,60834167610011,0.12370234326204846,2025-05-03 12:13:33,2025-05-03,2025-05,Test,0,1,SIL-Instore
3,3416751,6ba06482-7af5-4d52-bb78-c0ba40322911,60834167510018,0.04446030331305798,2025-05-03 12:17:06,2025-05-03,2025-05,Test,0,1,SIL-Instore
4,3416579,13fa435b-6f45-4804-9d75-feba4cd512f1,60834165790014,0.0893011497936029,2025-05-03 11:03:22,2025-05-03,2025-05,Test,0,1,SIL ZERO


In [77]:
df1 = dfd.copy()

## Train

In [78]:
sq = """ 
  with modelname as 
  (  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd10,
  del.flg_mature_fpd10,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd10 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1874984,400b4da9-378e-402d-ac83-6a9067891255,60818749840015,0.226734,2023-01-26 18:40:10,2023-01-26,2023-01,Train,0,1,SIL-Instore
1,2108876,e63b223f-b923-492b-99ad-c5e4862f9751,60821088760018,0.076851,2023-06-26 18:56:21,2023-06-26,2023-06,Train,0,1,SIL-Instore
2,2098171,c99ddbed-2dcd-4f1c-88f9-53d717d3e1d3,60820981710011,0.230376,2023-06-18 10:57:08,2023-06-18,2023-06,Train,0,1,SIL-Instore
3,1875678,461eef90-db74-47cc-ba41-e9fdb3cf190a,60818756780012,0.057596,2023-01-27 11:59:39,2023-01-27,2023-01,Train,0,1,SIL-Instore
4,2103397,755a5091-725d-4ed1-9647-6856b3d69144,60821033970018,0.169558,2023-06-22 10:28:10,2023-06-22,2023-06,Train,0,1,SIL-Instore


In [79]:
df2 = dfd.copy()

In [80]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174121 entries, 0 to 174120
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             174121 non-null  object        
 1   digitalLoanAccountId   174121 non-null  object        
 2   loanAccountNumber      174121 non-null  object        
 3   Sil_Alpha_Stack_score  174121 non-null  object        
 4   appln_submit_datetime  174121 non-null  datetime64[us]
 5   disbursementdate       174121 non-null  dbdate        
 6   Application_month      174121 non-null  object        
 7   Data_selection         174121 non-null  object        
 8   deffpd10               174121 non-null  Int64         
 9   flg_mature_fpd10       174121 non-null  Int64         
 10  new_loan_type          174121 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 14.9+ MB


In [81]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Sil_Alpha_Stack_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [82]:
f1=gini_results.copy()

In [83]:
f1.head(10)

,start_date,end_date,Sil_Alpha_Stack_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.263158,Month,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
1,2023-01-09,2023-01-15,0.275862,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
2,2023-01-16,2023-01-22,0.789474,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
3,2023-01-23,2023-01-29,0.237179,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
4,2023-01-30,2023-02-05,0.181818,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
5,2023-02-01,2023-02-28,0.133333,Month,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
6,2023-02-06,2023-02-12,0.419355,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
7,2023-02-13,2023-02-19,-0.110512,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
8,2023-02-20,2023-02-26,-0.192157,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall
9,2023-02-27,2023-03-05,0.522222,Week,Sil_Alpha_Stack_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [84]:
sq = """ 
with modelname as 
(
  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,3475356,4a55c09c-c1f6-4f5b-bf0b-c4ed9c94420c,60834753560017,0.023969200872189456,2025-06-03 11:34:24,2025-06-03,2025-06,Test,0,1,SIL-Instore
1,3475390,73aa6b2a-4b51-401e-bdf9-0d47ecede389,60834753900019,0.17598053009510617,2025-06-03 11:47:53,2025-06-03,2025-06,Test,0,1,SIL-Instore
2,3475322,5e2c1e3a-9125-4c2a-89a3-2740361686da,60834753220017,0.04274664294952835,2025-06-03 11:09:19,2025-06-03,2025-06,Test,0,1,SIL-Instore
3,3475310,38debe77-0cab-4047-9aad-a987d7ac7f52,60834753100013,0.06545998825310823,2025-06-03 11:12:07,2025-06-03,2025-06,Test,1,1,SIL-Instore
4,2886147,91ebd6b6-7ee0-4af1-b682-7cce20771ca3,60828861470024,0.13266878766917292,2025-06-03 15:56:44,2025-06-03,2025-06,Test,0,1,SIL-Instore


In [85]:
df1 = dfd.copy()

## Train

In [86]:
sq = """ 
  with modelname as 
  (  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fpd30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1949206,f1208bd6-2849-4a1e-b45f-4cf7ce462a9b,60819492060011,0.121783,2023-03-18 19:40:16,2023-03-18,2023-03,Train,0,1,SIL-Instore
1,2140301,030f6b03-b9b3-4e5f-999b-e53e68b8870f,60821403010012,0.123566,2023-07-16 18:11:53,2023-07-16,2023-07,Train,0,1,SIL-Instore
2,1937519,4f834e9a-5dd3-4cfb-a2fc-131b580ead74,60819375190012,0.203629,2023-03-10 17:10:12,2023-03-10,2023-03,Train,1,1,SIL-Instore
3,2067235,07d76a82-c37e-45ba-8370-f6cfd550f3c6,60820672350011,0.070214,2023-05-29 16:05:45,2023-05-29,2023-05,Train,0,1,SIL-Instore
4,1958097,534d661c-7e22-47c1-9a05-94feed085410,60819580970015,0.024638,2023-03-24 11:55:24,2023-03-24,2023-03,Train,0,1,SIL-Instore


In [87]:
df2 = dfd.copy()

In [88]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167483 entries, 0 to 167482
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             167483 non-null  object        
 1   digitalLoanAccountId   167483 non-null  object        
 2   loanAccountNumber      167483 non-null  object        
 3   Sil_Alpha_Stack_score  167483 non-null  object        
 4   appln_submit_datetime  167483 non-null  datetime64[us]
 5   disbursementdate       167483 non-null  dbdate        
 6   Application_month      167483 non-null  object        
 7   Data_selection         167483 non-null  object        
 8   deffpd30               167483 non-null  Int64         
 9   flg_mature_fpd30       167483 non-null  Int64         
 10  new_loan_type          167483 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 14.4+ MB


In [89]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Sil_Alpha_Stack_score', 'deffpd30', 'FPD30')

gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [90]:
f2=gini_results.copy()

In [91]:
f2.head(10)

,start_date,end_date,Sil_Alpha_Stack_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.358431,Month,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
1,2023-01-09,2023-01-15,0.718033,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
2,2023-01-16,2023-01-22,0.789474,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
3,2023-01-23,2023-01-29,0.107143,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
4,2023-01-30,2023-02-05,0.181818,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
5,2023-02-01,2023-02-28,0.181795,Month,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
6,2023-02-06,2023-02-12,0.419355,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
7,2023-02-13,2023-02-19,-0.110512,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
8,2023-02-20,2023-02-26,0.009615,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall
9,2023-02-27,2023-03-05,0.872340,Week,Sil_Alpha_Stack_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [92]:
sq = """ 
with modelname as 
(
  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fspd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,3382591,6566b436-ca32-4ae6-9029-bf173849ab96,60833825910013,0.11847333586030578,2025-04-16 10:52:27,2025-04-16,2025-04,Test,0,1,SIL ZERO
1,3382963,f9880e7d-c858-4a5e-8d2d-5ecb9c649163,60833829630011,0.1124632682482417,2025-04-16 13:26:02,2025-04-16,2025-04,Test,1,1,SIL-Instore
2,3382819,81195987-deb3-4be7-9b40-57bf8154eee1,60833828190014,0.15621805284724496,2025-04-16 12:28:59,2025-04-16,2025-04,Test,0,1,SIL-Instore
3,3382828,42a1ed04-ef15-4f32-8e14-76f227000e07,60833828280015,0.13342903146529214,2025-04-16 12:32:56,2025-04-16,2025-04,Test,1,1,SIL-Instore
4,3382884,ab712a43-45de-4854-b539-c127dac4c9b2,60833828840013,0.1546292076389961,2025-04-16 12:53:40,2025-04-16,2025-04,Test,1,1,SIL-Instore


In [93]:
df1 = dfd.copy()

## Train

In [94]:
sq = """ 
  with modelname as 
  (  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fspd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2041292,c2c51457-ddcc-4b12-a73f-2827c5dd3eab,60820412920016,0.258804,2023-05-12 17:41:42,2023-05-12,2023-05,Train,0,1,SIL-Instore
1,1989803,46b29c1e-0f20-4071-beb6-bddbb80b79f6,60819898030016,0.029149,2023-04-11 19:03:01,2023-04-11,2023-04,Train,0,1,SIL-Instore
2,2006944,a3ddc8c3-11d6-4ac0-a9f0-f03435522fe8,60820069440019,0.241838,2023-04-21 12:30:33,2023-04-21,2023-04,Train,0,1,SIL-Instore
3,2031230,97c904f8-3bf0-4127-9d74-31ed006642c1,60820312300017,0.066751,2023-05-06 10:40:59,2023-05-06,2023-05,Train,1,1,SIL-Instore
4,2087977,7e0613d1-f8f7-4571-9be8-32f8e76da036,60820879770012,0.027225,2023-06-11 16:36:25,2023-06-11,2023-06,Train,0,1,SIL-Instore


In [95]:
df2 = dfd.copy()

In [96]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158529 entries, 0 to 158528
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             158529 non-null  object        
 1   digitalLoanAccountId   158529 non-null  object        
 2   loanAccountNumber      158529 non-null  object        
 3   Sil_Alpha_Stack_score  158529 non-null  object        
 4   appln_submit_datetime  158529 non-null  datetime64[us]
 5   disbursementdate       158529 non-null  dbdate        
 6   Application_month      158529 non-null  object        
 7   Data_selection         158529 non-null  object        
 8   deffspd30              158529 non-null  Int64         
 9   flg_mature_fspd_30     158529 non-null  Int64         
 10  new_loan_type          158529 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 13.6+ MB


In [97]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Sil_Alpha_Stack_score', 'deffspd30', 'FSPD30')

gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [98]:
f3=gini_results.copy()

In [99]:
f3.head(10)

,start_date,end_date,Sil_Alpha_Stack_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.258788,Month,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
1,2023-01-09,2023-01-15,0.418103,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
2,2023-01-16,2023-01-22,0.789474,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
3,2023-01-23,2023-01-29,0.032051,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
4,2023-01-30,2023-02-05,0.143631,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
5,2023-02-01,2023-02-28,0.117886,Month,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
6,2023-02-06,2023-02-12,0.362963,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
7,2023-02-13,2023-02-19,-0.080610,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
8,2023-02-20,2023-02-26,-0.020000,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall
9,2023-02-27,2023-03-05,0.449612,Week,Sil_Alpha_Stack_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [100]:
sq = """ 
with modelname as 
(
  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()



,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,3406604,0639704e-695e-48bd-9816-ea97f0ae25ac,60834066040013,0.08721469555713367,2025-04-28 10:44:03,2025-04-28,2025-04,Test,0,1,SIL-Instore
1,3406567,0f277365-6230-4153-83c5-e4af6deb64da,60834065670015,0.1035175162094335,2025-04-28 10:21:05,2025-04-28,2025-04,Test,0,1,SIL-Instore
2,3406633,5ef6f81d-f1f9-4839-8aec-b791bd964e30,60834066330018,0.021049370441415725,2025-04-28 10:54:27,2025-04-28,2025-04,Test,0,1,SIL-Instore
3,3407307,d7d4d053-472f-4901-9aa8-0558d67a18ec,60834073070011,0.060192178798550926,2025-04-28 16:14:08,2025-04-28,2025-04,Test,0,1,SIL-Instore
4,3407103,2ee178a7-073f-4140-b8a6-b50e74c2d61c,60834071030011,0.13462299061559377,2025-04-28 14:25:40,2025-04-28,2025-04,Test,0,1,SIL-Instore


In [101]:
df1 = dfd.copy()

## Train

In [102]:
sq = """ 
  with modelname as 
  (  SELECT
    customerId,digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Alpha - StackingModel', 'alpha_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Sil_Alpha_Stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Sil_Alpha_Stack_score is not null
  and del.flg_mature_fstpd_30 = 1
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,Sil_Alpha_Stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2148390,e4af41fe-8104-4e20-95d1-76c68086d003,60821483900011,0.078951,2023-07-21 11:59:44,2023-07-21,2023-07,Train,0,1,SIL-Instore
1,2061957,9f4760f5-6e05-4773-8f9a-6043c200bb92,60820619570014,0.083750,2023-05-26 13:37:18,2023-05-26,2023-05,Train,0,1,SIL-Instore
2,2032788,ef2c958f-670d-4a1f-8fd9-3449cddb264f,60820327880014,0.045306,2023-05-07 11:02:41,2023-05-07,2023-05,Train,0,1,SIL-Instore
3,2115283,8d44bc9e-2ff8-4422-b401-a168ae5e5f06,60821152830012,0.183377,2023-07-01 17:07:39,2023-07-01,2023-07,Train,0,1,SIL-Instore
4,2098557,690eb3c6-ab2e-4e40-8d64-ad4f3a929e90,60820985570015,0.052654,2023-06-18 15:34:38,2023-06-18,2023-06,Train,0,1,SIL-Instore


In [103]:
df2 = dfd.copy()

In [104]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148228 entries, 0 to 148227
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             148228 non-null  object        
 1   digitalLoanAccountId   148228 non-null  object        
 2   loanAccountNumber      148228 non-null  object        
 3   Sil_Alpha_Stack_score  148228 non-null  object        
 4   appln_submit_datetime  148228 non-null  datetime64[us]
 5   disbursementdate       148228 non-null  dbdate        
 6   Application_month      148228 non-null  object        
 7   Data_selection         148228 non-null  object        
 8   deffstpd30             148228 non-null  Int64         
 9   flg_mature_fstpd_30    148228 non-null  Int64         
 10  new_loan_type          148228 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 12.7+ MB


In [105]:
df_concat['Sil_Alpha_Stack_score'] = pd.to_numeric(df_concat['Sil_Alpha_Stack_score'], errors='coerce')
# gini_results = calculate_periodic_gini(df_concat, 'Sil_Alpha_Stack_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Sil_Alpha_Stack_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [106]:
f4=gini_results.copy()

In [107]:
f4.head(10)

,start_date,end_date,Sil_Alpha_Stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.265602,Month,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
1,2023-01-09,2023-01-15,0.302479,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
2,2023-01-16,2023-01-22,0.575758,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
3,2023-01-23,2023-01-29,0.157895,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
4,2023-01-30,2023-02-05,0.324561,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
5,2023-02-01,2023-02-28,0.202272,Month,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
6,2023-02-06,2023-02-12,0.393103,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
7,2023-02-13,2023-02-19,0.028000,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
8,2023-02-20,2023-02-26,-0.036458,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall
9,2023-02-27,2023-03-05,0.347561,Week,Sil_Alpha_Stack_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [108]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type','bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'Sil_Alpha_Stack_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'Sil_Alpha_Stack_score_FPD10_gini',
       'Sil_Alpha_Stack_score_FPD30_gini',
       'Sil_Alpha_Stack_score_FSPD30_gini',
       'Sil_Alpha_Stack_score_FSTPD30_gini'], dtype=object)

In [109]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type','bad_rate','Sil_Alpha_Stack_score_FPD0_gini','Sil_Alpha_Stack_score_FPD10_gini','Sil_Alpha_Stack_score_FPD30_gini',  'Sil_Alpha_Stack_score_FSPD30_gini', 'Sil_Alpha_Stack_score_FSTPD30_gini']].copy()
final_df['Trench_category'] = 'All'
final_df['Model_display_name'] = 'alpha_stack_model_sil'
final_df['Product_type'] = 'SIL'
final_df.dtypes

start_date                            datetime64[ns]
end_date                              datetime64[ns]
period                                        object
Model_Name                                    object
version                                       object
loan_type                                     object
bad_rate                                      object
Sil_Alpha_Stack_score_FPD0_gini              float64
Sil_Alpha_Stack_score_FPD10_gini             float64
Sil_Alpha_Stack_score_FPD30_gini             float64
Sil_Alpha_Stack_score_FSPD30_gini            float64
Sil_Alpha_Stack_score_FSTPD30_gini           float64
Trench_category                               object
Model_display_name                            object
Product_type                                  object
dtype: object

In [110]:
final_df.head() 

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,Sil_Alpha_Stack_score_FPD0_gini,Sil_Alpha_Stack_score_FPD10_gini,Sil_Alpha_Stack_score_FPD30_gini,Sil_Alpha_Stack_score_FSPD30_gini,Sil_Alpha_Stack_score_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,Sil_Alpha_Stack_score,1.1.0,Overall,FPD0,0.252502,NaN,NaN,NaN,NaN,All,alpha_stack_model_sil,SIL
1,2023-01-09,2023-01-15,Week,Sil_Alpha_Stack_score,1.1.0,Overall,FPD0,0.343407,NaN,NaN,NaN,NaN,All,alpha_stack_model_sil,SIL
2,2023-01-16,2023-01-22,Week,Sil_Alpha_Stack_score,1.1.0,Overall,FPD0,0.660714,NaN,NaN,NaN,NaN,All,alpha_stack_model_sil,SIL
3,2023-01-23,2023-01-29,Week,Sil_Alpha_Stack_score,1.1.0,Overall,FPD0,0.113514,NaN,NaN,NaN,NaN,All,alpha_stack_model_sil,SIL
4,2023-01-30,2023-02-05,Week,Sil_Alpha_Stack_score,1.1.0,Overall,FPD0,0.182724,NaN,NaN,NaN,NaN,All,alpha_stack_model_sil,SIL


In [111]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.Sil_Alpha_Stack_scorev1_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=828d941d-2534-40a1-8aae-0f2d857578e3>

# Beta Sil App Score

## FPD10

## Test

In [112]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3758187,f0807726-adc1-4089-9455-893102ea791c,60837581870018,0.465695,2025-10-21 11:08:33,2025-10-21,2025-10,Test,0,1,SIL Competitor
1,3758237,aeb121ad-91e7-404b-b1cd-14ef4a0b490e,60837582370011,0.446200,2025-10-21 11:49:25,2025-10-21,2025-10,Test,0,1,SIL-Instore
2,3758250,90913a53-9b5f-4ac1-8d32-c38457d13b31,60837582500018,0.591178,2025-10-21 11:25:16,2025-10-21,2025-10,Test,0,1,SIL Competitor
3,3758271,2299f220-25f9-473d-9b98-5aeda67f3b84,60837582710013,0.443234,2025-10-21 11:33:12,2025-10-21,2025-10,Test,0,1,SIL-Instore
4,3758288,56bd9ff3-486a-4039-8e7e-66ec80cb15d3,60837582880012,0.527983,2025-10-21 11:37:39,2025-10-21,2025-10,Test,0,1,SIL-Instore


In [113]:
df1 = dfd.copy()

## Train

In [114]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd0 = 1
  ;

"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1905068,06cfaff6-ba84-4a46-9441-cf6d8193c252,60819050680013,0.586266,2023-02-17 17:44:14,2023-02-17,2023-02,Train,0,1,SIL-Instore
1,2076951,ce9dc6af-1c75-4aff-97fb-b5b9430011e1,60820769510013,0.353031,2023-06-04 13:02:53,2023-06-04,2023-06,Train,0,1,SIL-Instore
2,2098128,a13f28fa-b817-4b1a-b84d-a8a676bfb32e,60820981280018,0.539320,2023-06-18 10:22:15,2023-06-18,2023-06,Train,0,1,SIL-Instore
3,1893351,069d1475-2f92-4d3e-9706-704dcffbc0de,60818933510014,0.525320,2023-02-09 15:08:11,2023-02-09,2023-02,Train,0,1,SIL-Instore
4,1953780,721610a6-94bb-45f4-8926-dce7b76a4fef,60819537800017,0.286241,2023-03-21 19:12:34,2023-03-21,2023-03,Train,0,1,SIL-Instore


In [115]:
df2 = dfd.copy()

In [116]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245200 entries, 0 to 245199
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             245200 non-null  object        
 1   digitalLoanAccountId   245200 non-null  object        
 2   loanAccountNumber      245200 non-null  object        
 3   sil_beta_app_score     245200 non-null  float64       
 4   appln_submit_datetime  245200 non-null  datetime64[us]
 5   disbursementdate       245200 non-null  dbdate        
 6   Application_month      245200 non-null  object        
 7   Data_selection         245200 non-null  object        
 8   deffpd0                245200 non-null  Int64         
 9   flg_mature_fpd0        245200 non-null  Int64         
 10  new_loan_type          245200 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(6)
memory usage: 21.0+ MB


In [117]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_app_score'] = pd.to_numeric(df_concat['sil_beta_app_score'], errors='coerce')

In [118]:
# df_concat.to_csv(r"sil_beta_app_score.csv")

In [119]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_app_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_app_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [120]:
f0 = gini_results.copy()

In [121]:
f0.head(10)

,start_date,end_date,sil_beta_app_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.156053,Month,sil_beta_app_score,1.1.0,FPD0,Overall
1,2023-01-02,2023-01-08,0.181917,Week,sil_beta_app_score,1.1.0,FPD0,Overall
2,2023-01-09,2023-01-15,0.297158,Week,sil_beta_app_score,1.1.0,FPD0,Overall
3,2023-01-16,2023-01-22,0.170139,Week,sil_beta_app_score,1.1.0,FPD0,Overall
4,2023-01-23,2023-01-29,0.083217,Week,sil_beta_app_score,1.1.0,FPD0,Overall
5,2023-01-30,2023-02-05,0.297507,Week,sil_beta_app_score,1.1.0,FPD0,Overall
6,2023-02-01,2023-02-28,0.293496,Month,sil_beta_app_score,1.1.0,FPD0,Overall
7,2023-02-06,2023-02-12,0.413246,Week,sil_beta_app_score,1.1.0,FPD0,Overall
8,2023-02-13,2023-02-19,0.164119,Week,sil_beta_app_score,1.1.0,FPD0,Overall
9,2023-02-20,2023-02-26,0.302374,Week,sil_beta_app_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [122]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd10 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3406547,d58d4c78-e14a-40ed-b7bd-019caaa51f1f,60834065470011,0.460089,2025-04-28 10:14:30,2025-04-28,2025-04,Test,0,1,SIL-Instore
1,3406567,0f277365-6230-4153-83c5-e4af6deb64da,60834065670015,0.549969,2025-04-28 10:21:05,2025-04-28,2025-04,Test,0,1,SIL-Instore
2,3406565,96a719f2-4b37-4504-96fd-32cbfd63be7a,60834065650013,0.509632,2025-04-28 10:27:57,2025-04-28,2025-04,Test,0,1,SIL-Instore
3,3406604,0639704e-695e-48bd-9816-ea97f0ae25ac,60834066040013,0.567517,2025-04-28 10:44:03,2025-04-28,2025-04,Test,0,1,SIL-Instore
4,3406633,5ef6f81d-f1f9-4839-8aec-b791bd964e30,60834066330018,0.373547,2025-04-28 10:54:27,2025-04-28,2025-04,Test,0,1,SIL-Instore


In [123]:
df1 = dfd.copy()

## Train

In [124]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd10 = 1
  ;

"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1931275,442f40c1-73b9-4c4a-9df0-e39b56b1e97d,60819312750017,0.592172,2023-03-06 17:45:53,2023-03-06,2023-03,Train,0,1,SIL-Instore
1,1945706,1a778b21-2436-4ef6-8f93-e76044ead6ca,60819457060011,0.509239,2023-03-16 13:37:24,2023-03-16,2023-03,Train,0,1,SIL-Instore
2,2067357,7457e87c-8c66-43c3-beb3-743257a094bc,60820673570011,0.549414,2023-05-29 17:12:24,2023-05-29,2023-05,Train,0,1,SIL-Instore
3,1889099,1e34a166-5e8f-4e65-af26-b9bac17a8b65,60818890990011,0.479318,2023-02-06 17:16:58,2023-02-06,2023-02,Train,0,1,SIL-Instore
4,1941384,d5c499a9-a818-4b42-a0e7-1e5e4248223a,60819413840016,0.589654,2023-03-13 14:54:45,2023-03-13,2023-03,Train,0,1,SIL-Instore


In [125]:
df2 = dfd.copy()

In [126]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241261 entries, 0 to 241260
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             241261 non-null  object        
 1   digitalLoanAccountId   241261 non-null  object        
 2   loanAccountNumber      241261 non-null  object        
 3   sil_beta_app_score     241261 non-null  float64       
 4   appln_submit_datetime  241261 non-null  datetime64[us]
 5   disbursementdate       241261 non-null  dbdate        
 6   Application_month      241261 non-null  object        
 7   Data_selection         241261 non-null  object        
 8   deffpd10               241261 non-null  Int64         
 9   flg_mature_fpd10       241261 non-null  Int64         
 10  new_loan_type          241261 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(6)
memory usage: 20.7+ MB


In [127]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_app_score'] = pd.to_numeric(df_concat['sil_beta_app_score'], errors='coerce')

In [128]:
# df_concat.to_csv(r"sil_beta_app_scorefpd10.csv")

In [129]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_app_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_app_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [130]:
f1 = gini_results.copy()

In [131]:
f1.head(10)

,start_date,end_date,sil_beta_app_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.225850,Month,sil_beta_app_score,1.1.0,FPD10,Overall
1,2023-01-02,2023-01-08,0.196935,Week,sil_beta_app_score,1.1.0,FPD10,Overall
2,2023-01-09,2023-01-15,0.381288,Week,sil_beta_app_score,1.1.0,FPD10,Overall
3,2023-01-16,2023-01-22,0.210526,Week,sil_beta_app_score,1.1.0,FPD10,Overall
4,2023-01-23,2023-01-29,0.191667,Week,sil_beta_app_score,1.1.0,FPD10,Overall
5,2023-01-30,2023-02-05,0.411765,Week,sil_beta_app_score,1.1.0,FPD10,Overall
6,2023-02-01,2023-02-28,0.275626,Month,sil_beta_app_score,1.1.0,FPD10,Overall
7,2023-02-06,2023-02-12,0.378472,Week,sil_beta_app_score,1.1.0,FPD10,Overall
8,2023-02-13,2023-02-19,0.037940,Week,sil_beta_app_score,1.1.0,FPD10,Overall
9,2023-02-20,2023-02-26,0.318296,Week,sil_beta_app_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [132]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,3341273,263984d1-0806-447b-8c5a-9c3443adfb62,60833412730018,0.374347,2025-03-26 12:02:52,2025-03-26,2025-03,Test,1,1,SIL-Instore
1,3341375,32bfa693-7018-412e-978b-0e5370f218e1,60833413750013,0.532252,2025-03-26 12:50:50,2025-03-26,2025-03,Test,0,1,SIL-Instore
2,3341392,32acec8d-2500-4792-85b6-b65fa3de1744,60833413920014,0.487486,2025-03-26 12:56:59,2025-03-26,2025-03,Test,0,1,SIL-Instore
3,3341022,4a4a7ffa-ff23-43c2-bd1b-7c002af9f3dd,60833410220011,0.309667,2025-03-26 09:24:30,2025-03-26,2025-03,Test,0,1,SIL-Instore
4,3341053,1d39bf79-2fde-48b4-86e8-80049b658186,60833410530018,0.405284,2025-03-26 09:40:03,2025-03-26,2025-03,Test,0,1,SIL-Instore


In [133]:
df1 = dfd.copy()

## Train

In [134]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fpd30 = 1
  ;

"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2059837,5f9a2b85-6cfa-4200-9188-97b81a9df0b9,60820598370014,0.359386,2023-05-24 12:35:39,2023-05-24,2023-05,Train,0,1,SIL-Instore
1,1241712,4721a817-dab4-4962-a5a2-cd4927c8533e,60812417120011,0.429813,2023-04-28 18:24:57,2023-04-28,2023-04,Train,0,1,SIL-Instore
2,1857151,b39d23a1-615d-495b-8c84-01af828efc98,60818571510014,0.416184,2023-01-10 14:57:13,2023-01-10,2023-01,Train,0,1,SIL-Instore
3,1887740,c3f237fc-4f29-4eae-a4fe-f898894b90db,60818877400014,0.506146,2023-02-05 17:33:24,2023-02-05,2023-02,Train,0,1,SIL-Instore
4,2005455,7f334f99-ab07-4d67-afb5-22f522dcdfc8,60820054550013,0.377260,2023-04-20 14:22:50,2023-04-20,2023-04,Train,0,1,SIL-Instore


In [135]:
df2 = dfd.copy()

In [136]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232582 entries, 0 to 232581
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             232582 non-null  object        
 1   digitalLoanAccountId   232582 non-null  object        
 2   loanAccountNumber      232582 non-null  object        
 3   sil_beta_app_score     232582 non-null  float64       
 4   appln_submit_datetime  232582 non-null  datetime64[us]
 5   disbursementdate       232582 non-null  dbdate        
 6   Application_month      232582 non-null  object        
 7   Data_selection         232582 non-null  object        
 8   deffpd30               232582 non-null  Int64         
 9   flg_mature_fpd30       232582 non-null  Int64         
 10  new_loan_type          232582 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(6)
memory usage: 20.0+ MB


In [137]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_app_score'] = pd.to_numeric(df_concat['sil_beta_app_score'], errors='coerce')

In [138]:
# df_concat.to_csv(r"sil_beta_app_scorefpd30.csv")

In [139]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_app_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_app_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [140]:
f2 = gini_results.copy()

In [141]:
f2.head(10)

,start_date,end_date,sil_beta_app_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.269790,Month,sil_beta_app_score,1.1.0,FPD30,Overall
1,2023-01-02,2023-01-08,0.257051,Week,sil_beta_app_score,1.1.0,FPD30,Overall
2,2023-01-09,2023-01-15,0.555556,Week,sil_beta_app_score,1.1.0,FPD30,Overall
3,2023-01-16,2023-01-22,0.210526,Week,sil_beta_app_score,1.1.0,FPD30,Overall
4,2023-01-23,2023-01-29,0.130579,Week,sil_beta_app_score,1.1.0,FPD30,Overall
5,2023-01-30,2023-02-05,0.496911,Week,sil_beta_app_score,1.1.0,FPD30,Overall
6,2023-02-01,2023-02-28,0.305164,Month,sil_beta_app_score,1.1.0,FPD30,Overall
7,2023-02-06,2023-02-12,0.334583,Week,sil_beta_app_score,1.1.0,FPD30,Overall
8,2023-02-13,2023-02-19,0.096774,Week,sil_beta_app_score,1.1.0,FPD30,Overall
9,2023-02-20,2023-02-26,0.437681,Week,sil_beta_app_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [142]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fspd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,3357402,1feeb407-bdc1-4b68-8065-24c67aa5cf0e,60833574020018,0.471515,2025-04-04 10:24:39,2025-04-04,2025-04,Test,0,1,SIL-Instore
1,3358464,285ea65e-6126-4912-8397-c00849fd01cb,60833584640016,0.541531,2025-04-04 09:19:04,2025-04-04,2025-04,Test,0,1,SIL-Instore
2,3358589,320b5979-1b06-4f10-b382-cc31f5a3472a,60833585890018,0.433471,2025-04-04 10:26:56,2025-04-04,2025-04,Test,0,1,SIL-Instore
3,3358386,3b83a8cd-b5a5-446d-a623-e09593ea773d,60833583860019,0.606133,2025-04-04 08:11:50,2025-04-04,2025-04,Test,0,1,SIL ZERO
4,3358979,748ae494-ae8b-4137-a437-47e8b71d1bef,60833589790018,0.529379,2025-04-04 13:32:54,2025-04-04,2025-04,Test,0,1,SIL-Instore


In [143]:
df1 = dfd.copy()

## Train

In [144]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fspd_30 = 1
  ;

"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2077017,2b0786f2-4b06-4209-9fc0-cf2506d2306c,60820770170018,0.384938,2023-06-04 14:03:38,2023-06-04,2023-06,Train,0,1,SIL-Instore
1,2045160,5ef813a9-3988-4794-923a-1ec31bf5dd5d,60820451600011,0.458062,2023-05-15 13:29:54,2023-05-15,2023-05,Train,0,1,SIL-Instore
2,2001916,d5d7b371-90ce-4c2a-94ea-decd695b7821,60820019160015,0.448562,2023-04-18 16:13:10,2023-04-18,2023-04,Train,0,1,SIL-Instore
3,2043626,289133d3-be50-49b1-a6e1-587d531dae0d,60820436260016,0.435227,2023-05-14 13:11:53,2023-05-14,2023-05,Train,0,1,SIL-Instore
4,1938948,2308a14f-3812-403f-9df8-74d58797e522,60819389480013,0.301886,2023-03-11 17:21:19,2023-03-11,2023-03,Train,0,1,SIL-Instore


In [145]:
df2 = dfd.copy()

In [146]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220858 entries, 0 to 220857
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             220858 non-null  object        
 1   digitalLoanAccountId   220858 non-null  object        
 2   loanAccountNumber      220858 non-null  object        
 3   sil_beta_app_score     220858 non-null  float64       
 4   appln_submit_datetime  220858 non-null  datetime64[us]
 5   disbursementdate       220858 non-null  dbdate        
 6   Application_month      220858 non-null  object        
 7   Data_selection         220858 non-null  object        
 8   deffspd30              220858 non-null  Int64         
 9   flg_mature_fspd_30     220858 non-null  Int64         
 10  new_loan_type          220858 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(6)
memory usage: 19.0+ MB


In [147]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_app_score'] = pd.to_numeric(df_concat['sil_beta_app_score'], errors='coerce')

In [148]:
# df_concat.to_csv(r"sil_beta_app_scorefspd30.csv")

In [149]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_app_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_app_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [150]:
f3 = gini_results.copy()

In [151]:
f3.head(10)

,start_date,end_date,sil_beta_app_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.247928,Month,sil_beta_app_score,1.1.0,FSPD30,Overall
1,2023-01-02,2023-01-08,0.124845,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
2,2023-01-09,2023-01-15,0.399069,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
3,2023-01-16,2023-01-22,0.364341,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
4,2023-01-23,2023-01-29,0.246377,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
5,2023-01-30,2023-02-05,0.419042,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
6,2023-02-01,2023-02-28,0.330061,Month,sil_beta_app_score,1.1.0,FSPD30,Overall
7,2023-02-06,2023-02-12,0.419753,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
8,2023-02-13,2023-02-19,0.188025,Week,sil_beta_app_score,1.1.0,FSPD30,Overall
9,2023-02-20,2023-02-26,0.318043,Week,sil_beta_app_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [152]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,3560178,b766741a-4cda-4cdc-96c9-c70f7150041d,60835601780018,0.482862,2025-07-16 13:38:18,2025-07-16,2025-07,Test,0,1,SIL-Instore
1,3560112,6ae8a77c-e3ee-4cc8-885c-8e8758fcb4b9,60835601120011,0.607262,2025-07-16 13:27:41,2025-07-16,2025-07,Test,0,1,SIL-Instore
2,3560145,b3a13d0e-85e5-46d4-8a3b-5ec748a2e0ba,60835601450019,0.428847,2025-07-16 13:39:12,2025-07-16,2025-07,Test,0,1,SIL Competitor
3,2753068,e23d87a7-d1a8-4b43-8e4c-9efb25acc70f,60827530680031,0.472483,2025-07-16 12:25:24,2025-07-16,2025-07,Test,0,1,SIL Competitor
4,3019236,65b37025-cf83-4e59-9531-7171d97752d5,60830192360012,0.504957,2025-07-16 12:10:27,2025-07-16,2025-07,Test,0,1,SIL Competitor


In [153]:
df1 = dfd.copy()

## Train

In [154]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score, 
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;

"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_app_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,1981457,a650369a-fc2c-435e-90b4-17b00371214c,60819814570016,0.295080,2023-04-06 19:13:24,2023-04-06,2023-04,Train,0,1,SIL-Instore
1,1942684,33f65398-1f03-489f-bb5e-f82f271d6ab2,60819426840019,0.558468,2023-03-14 13:59:16,2023-03-14,2023-03,Train,0,1,SIL-Instore
2,1854837,765c9c03-7557-44c8-90f3-7571acfeea73,60818548370015,0.528773,2023-01-08 15:23:41,2023-01-08,2023-01,Train,1,1,SIL-Instore
3,2044923,7878aed8-480b-4ddc-ab20-51026e510d33,60820449230016,0.538888,2023-05-15 11:29:36,2023-05-17,2023-05,Train,0,1,SIL-Instore
4,1987476,78a94fca-d337-4246-b218-954723a1c6bf,60819874760013,0.558539,2023-04-10 16:49:34,2023-04-10,2023-04,Train,0,1,SIL-Instore


In [155]:
df2 = dfd.copy()

In [156]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207466 entries, 0 to 207465
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             207466 non-null  object        
 1   digitalLoanAccountId   207466 non-null  object        
 2   loanAccountNumber      207466 non-null  object        
 3   sil_beta_app_score     207466 non-null  float64       
 4   appln_submit_datetime  207466 non-null  datetime64[us]
 5   disbursementdate       207466 non-null  dbdate        
 6   Application_month      207466 non-null  object        
 7   Data_selection         207466 non-null  object        
 8   deffstpd30             207466 non-null  Int64         
 9   flg_mature_fstpd_30    207466 non-null  Int64         
 10  new_loan_type          207466 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(6)
memory usage: 17.8+ MB


In [157]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_app_score'] = pd.to_numeric(df_concat['sil_beta_app_score'], errors='coerce')

In [158]:
# df_concat.to_csv(r"sil_beta_app_scorefstpd30.csv")

In [159]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_app_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_app_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [160]:
f4 = gini_results.copy()

In [161]:
f4.head(10)

,start_date,end_date,sil_beta_app_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.260324,Month,sil_beta_app_score,1.1.0,FSTPD30,Overall
1,2023-01-02,2023-01-08,0.145503,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
2,2023-01-09,2023-01-15,0.362537,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
3,2023-01-16,2023-01-22,0.261324,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
4,2023-01-23,2023-01-29,0.308780,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
5,2023-01-30,2023-02-05,0.436764,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
6,2023-02-01,2023-02-28,0.356994,Month,sil_beta_app_score,1.1.0,FSTPD30,Overall
7,2023-02-06,2023-02-12,0.503968,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
8,2023-02-13,2023-02-19,0.242287,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall
9,2023-02-20,2023-02-26,0.238095,Week,sil_beta_app_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [162]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sil_beta_app_score_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'sil_beta_app_score_FPD10_gini', 'sil_beta_app_score_FPD30_gini',
       'sil_beta_app_score_FSPD30_gini',
       'sil_beta_app_score_FSTPD30_gini'], dtype=object)

In [163]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type','bad_rate','sil_beta_app_score_FPD0_gini','sil_beta_app_score_FPD10_gini', 'sil_beta_app_score_FPD30_gini',
       'sil_beta_app_score_FSPD30_gini',
       'sil_beta_app_score_FSTPD30_gini']].copy()
final_df['Trench_category'] = 'All'
final_df['Model_display_name'] = 'apps_score_model_sil'
final_df['Product_type'] = 'SIL'
final_df.dtypes

start_date                         datetime64[ns]
end_date                           datetime64[ns]
period                                     object
Model_Name                                 object
version                                    object
loan_type                                  object
bad_rate                                   object
sil_beta_app_score_FPD0_gini              float64
sil_beta_app_score_FPD10_gini             float64
sil_beta_app_score_FPD30_gini             float64
sil_beta_app_score_FSPD30_gini            float64
sil_beta_app_score_FSTPD30_gini           float64
Trench_category                            object
Model_display_name                         object
Product_type                               object
dtype: object

In [164]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,sil_beta_app_score_FPD0_gini,sil_beta_app_score_FPD10_gini,sil_beta_app_score_FPD30_gini,sil_beta_app_score_FSPD30_gini,sil_beta_app_score_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,sil_beta_app_score,1.1.0,Overall,FPD0,0.156053,NaN,NaN,NaN,NaN,All,apps_score_model_sil,SIL
1,2023-01-02,2023-01-08,Week,sil_beta_app_score,1.1.0,Overall,FPD0,0.181917,NaN,NaN,NaN,NaN,All,apps_score_model_sil,SIL
2,2023-01-09,2023-01-15,Week,sil_beta_app_score,1.1.0,Overall,FPD0,0.297158,NaN,NaN,NaN,NaN,All,apps_score_model_sil,SIL
3,2023-01-16,2023-01-22,Week,sil_beta_app_score,1.1.0,Overall,FPD0,0.170139,NaN,NaN,NaN,NaN,All,apps_score_model_sil,SIL
4,2023-01-23,2023-01-29,Week,sil_beta_app_score,1.1.0,Overall,FPD0,0.083217,NaN,NaN,NaN,NaN,All,apps_score_model_sil,SIL


In [165]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.sil_beta_app_score_v1_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6e557a66-b8d5-4462-8d0c-904c2c368b1c>

# Beta SIL Demo Score

## FPD0

## Test

In [166]:
sq = """WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd0 = 1
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2266708,0ffca36d-ce46-4a25-998b-3a001316c21a,60822667080011,0.0826999994,2025-07-22 16:31:18,2025-07-22,2025-07,Test,0,1,SIL-Instore
1,2433529,fa7cba2f-faa4-4e87-a15c-ccd000ed8b23,60824335290031,0.1289146069,2025-07-22 11:06:00,2025-07-22,2025-07,Test,0,1,SIL ZERO
2,2439467,77d297d4-07d5-4817-af02-db137931355a,60824394670021,0.1176743528,2025-07-22 10:34:46,2025-07-22,2025-07,Test,0,1,SIL ZERO
3,2519452,f5b5ed4b-201d-4f20-a8ef-a9d768001835,60825194520029,0.1514659868,2025-07-22 14:01:30,2025-07-22,2025-07,Test,0,1,SIL-Instore
4,2598736,e82db18f-5daf-4a5b-8b18-dec88ad64463,60825987360016,0.0985097359,2025-07-22 18:19:00,2025-07-22,2025-07,Test,0,1,SIL-Instore


In [167]:
df1 = dfd.copy()

## Train

In [168]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd0 = 1
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2111260,4cf3316d-8b4d-4bb0-9bb4-7574da398a09,60821112600019,0.060562,2023-06-28 19:19:32,2023-06-28,2023-06,Train,0,1,SIL-Instore
1,2018024,a344b014-ae5e-44d3-b4ed-4928488a520f,60820180240011,0.098842,2023-04-28 17:51:35,2023-04-28,2023-04,Train,0,1,SIL-Instore
2,2056550,ed7d1987-3404-4049-867f-9291f1c228d7,60820565500011,0.072240,2023-05-22 17:42:40,2023-05-22,2023-05,Train,0,1,SIL-Instore
3,2083859,a551e203-402b-4de0-8baf-200655a2f1bf,60820838590011,0.132479,2023-06-08 16:41:36,2023-06-08,2023-06,Train,0,1,SIL-Instore
4,2009090,3f71179e-494a-405d-87ec-f21f576e9da8,60820090900011,0.212695,2023-04-22 18:10:02,2023-04-22,2023-04,Train,0,1,SIL-Instore


In [169]:
df2 = dfd.copy()

In [170]:
df2 = dfd.copy()

In [171]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273188 entries, 0 to 273187
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             273188 non-null  object        
 1   digitalLoanAccountId   273188 non-null  object        
 2   loanAccountNumber      273188 non-null  object        
 3   sil_beta_demo_score    273188 non-null  object        
 4   appln_submit_datetime  273188 non-null  datetime64[us]
 5   disbursementdate       273188 non-null  dbdate        
 6   Application_month      273188 non-null  object        
 7   Data_selection         273188 non-null  object        
 8   deffpd0                273188 non-null  Int64         
 9   flg_mature_fpd0        273188 non-null  Int64         
 10  new_loan_type          273188 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 23.4+ MB


In [172]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [173]:
# df_concat.to_csv(r"sil_beta_demo_scorefpd0.csv")

In [174]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_demo_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [175]:
f0 = gini_results.copy()

In [176]:
f0.head(10)

,start_date,end_date,sil_beta_demo_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.141958,Month,sil_beta_demo_score,1.1.0,FPD0,Overall
1,2023-01-02,2023-01-08,0.279249,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
2,2023-01-09,2023-01-15,0.118033,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
3,2023-01-16,2023-01-22,0.153020,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
4,2023-01-23,2023-01-29,-0.006286,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
5,2023-01-30,2023-02-05,0.196649,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
6,2023-02-01,2023-02-28,0.150587,Month,sil_beta_demo_score,1.1.0,FPD0,Overall
7,2023-02-06,2023-02-12,0.030928,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
8,2023-02-13,2023-02-19,0.379882,Week,sil_beta_demo_score,1.1.0,FPD0,Overall
9,2023-02-20,2023-02-26,0.056522,Week,sil_beta_demo_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [177]:
sq = """WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd10 = 1
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3672405,0312a551-0f07-4dc6-a182-5fbd2ae1f256,60836724050019,0.0529363595,2025-09-08 14:59:54,2025-09-08,2025-09,Test,0,1,SIL Competitor
1,3672482,0474e5f6-5f35-4276-afbe-93df1a5e174f,60836724820012,0.0600415228,2025-09-08 15:42:51,2025-09-08,2025-09,Test,0,1,SIL-Instore
2,3672373,04c23807-4f2c-4bf6-87f0-e8a79bc4f9dd,60836723730018,0.0746722081,2025-09-08 14:45:01,2025-09-08,2025-09,Test,1,1,SIL Competitor
3,3672463,0cbce494-250c-49fc-a8b9-92f6df3ab16e,60836724630019,0.065527793,2025-09-08 15:20:28,2025-09-08,2025-09,Test,1,1,SIL-Instore
4,3672475,0ce4ade3-df6c-4d3c-a4dd-bbed98585c00,60836724750013,0.0301412371,2025-09-08 15:21:20,2025-09-08,2025-09,Test,0,1,SIL Competitor


In [178]:
df1 = dfd.copy()

## Train

In [179]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd10 = 1
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1876799,bc6e9842-ba61-41c3-b0bb-e615a511ac89,60818767990014,0.096743,2023-01-28 12:01:50,2023-01-28,2023-01,Train,0,1,SIL-Instore
1,1852592,23addc1e-56e1-47d6-a5ce-51d24bae4b28,60818525920015,0.138242,2023-01-06 16:26:12,2023-01-06,2023-01,Train,0,1,SIL-Instore
2,1860210,4fed22b6-9fd7-4491-9d68-89e439fbedcf,60818602100015,0.135410,2023-01-13 13:28:33,2023-01-13,2023-01,Train,1,1,SIL-Instore
3,1975441,7560be92-3b5c-435a-b7d5-3d4ef0145106,60819754410019,0.052540,2023-04-03 11:54:22,2023-04-03,2023-04,Train,0,1,SIL-Instore
4,1979374,0ec4d43a-9225-4227-8a8d-158ffd47834a,60819793740011,0.248188,2023-04-05 14:38:56,2023-04-05,2023-04,Train,0,1,SIL-Instore


In [180]:
df2 = dfd.copy()

In [181]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268738 entries, 0 to 268737
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             268738 non-null  object        
 1   digitalLoanAccountId   268738 non-null  object        
 2   loanAccountNumber      268738 non-null  object        
 3   sil_beta_demo_score    268738 non-null  object        
 4   appln_submit_datetime  268738 non-null  datetime64[us]
 5   disbursementdate       268738 non-null  dbdate        
 6   Application_month      268738 non-null  object        
 7   Data_selection         268738 non-null  object        
 8   deffpd10               268738 non-null  Int64         
 9   flg_mature_fpd10       268738 non-null  Int64         
 10  new_loan_type          268738 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 23.1+ MB


In [182]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [183]:
# df_concat.to_csv(r"sil_beta_demo_scorefpd10.csv")

In [184]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_demo_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [185]:
f1 = gini_results.copy()

In [186]:
f1.head(10)

,start_date,end_date,sil_beta_demo_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.231433,Month,sil_beta_demo_score,1.1.0,FPD10,Overall
1,2023-01-02,2023-01-08,0.410532,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
2,2023-01-09,2023-01-15,0.113578,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
3,2023-01-16,2023-01-22,0.485714,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
4,2023-01-23,2023-01-29,0.102941,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
5,2023-01-30,2023-02-05,0.042026,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
6,2023-02-01,2023-02-28,0.113077,Month,sil_beta_demo_score,1.1.0,FPD10,Overall
7,2023-02-06,2023-02-12,0.170380,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
8,2023-02-13,2023-02-19,0.384306,Week,sil_beta_demo_score,1.1.0,FPD10,Overall
9,2023-02-20,2023-02-26,-0.136752,Week,sil_beta_demo_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [187]:
sq = """WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd30 = 1
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,3358980,0a74e0cb-e566-4059-9a39-1322dc549fe1,60833589800011,0.0604700334,2025-04-04 13:40:59,2025-04-04,2025-04,Test,0,1,SIL-Instore
1,3357402,1feeb407-bdc1-4b68-8065-24c67aa5cf0e,60833574020018,0.0765522071,2025-04-04 10:24:39,2025-04-04,2025-04,Test,0,1,SIL-Instore
2,3358464,285ea65e-6126-4912-8397-c00849fd01cb,60833584640016,0.0615415523,2025-04-04 09:19:04,2025-04-04,2025-04,Test,0,1,SIL-Instore
3,3358589,320b5979-1b06-4f10-b382-cc31f5a3472a,60833585890018,0.0448939705,2025-04-04 10:26:56,2025-04-04,2025-04,Test,0,1,SIL-Instore
4,3358386,3b83a8cd-b5a5-446d-a623-e09593ea773d,60833583860019,0.037863578,2025-04-04 08:11:50,2025-04-04,2025-04,Test,0,1,SIL ZERO


In [188]:
df1 = dfd.copy()

## Train

In [189]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd30 = 1
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1870166,80eb165d-f9b9-494e-a10d-b40c3883e5ab,60818701660013,0.062163,2023-01-22 15:43:18,2023-01-22,2023-01,Train,0,1,SIL-Instore
1,1975596,0ee5cba8-f9fa-4bb3-a760-2f31ca4b72a7,60819755960017,0.132763,2023-04-03 13:06:55,2023-04-03,2023-04,Train,0,1,SIL-Instore
2,1918137,9c15abff-aba5-403c-b214-73edcb8166f7,60819181370016,0.223498,2023-03-18 11:29:29,2023-03-18,2023-03,Train,0,1,SIL-Instore
3,1854926,45b11393-802c-4f39-8524-1e86dc478ae1,60818549260015,0.036906,2023-01-08 16:12:32,2023-01-08,2023-01,Train,0,1,SIL-Instore
4,1916608,ca180aac-8401-4829-a354-88a409b989d9,60819166080018,0.103376,2023-02-25 12:18:39,2023-02-25,2023-02,Train,0,1,SIL-Instore


In [190]:
df2 = dfd.copy()

In [191]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258928 entries, 0 to 258927
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             258928 non-null  object        
 1   digitalLoanAccountId   258928 non-null  object        
 2   loanAccountNumber      258928 non-null  object        
 3   sil_beta_demo_score    258928 non-null  object        
 4   appln_submit_datetime  258928 non-null  datetime64[us]
 5   disbursementdate       258928 non-null  dbdate        
 6   Application_month      258928 non-null  object        
 7   Data_selection         258928 non-null  object        
 8   deffpd30               258928 non-null  Int64         
 9   flg_mature_fpd30       258928 non-null  Int64         
 10  new_loan_type          258928 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 22.2+ MB


In [192]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [193]:
# df_concat.to_csv(r"sil_beta_demo_scorefpd30.csv")

In [194]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_demo_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [195]:
f2 = gini_results.copy()

In [196]:
f2.head(10)

,start_date,end_date,sil_beta_demo_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.286133,Month,sil_beta_demo_score,1.1.0,FPD30,Overall
1,2023-01-02,2023-01-08,0.382369,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
2,2023-01-09,2023-01-15,0.290358,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
3,2023-01-16,2023-01-22,0.485714,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
4,2023-01-23,2023-01-29,0.119526,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
5,2023-01-30,2023-02-05,0.090598,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
6,2023-02-01,2023-02-28,0.133196,Month,sil_beta_demo_score,1.1.0,FPD30,Overall
7,2023-02-06,2023-02-12,0.153846,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
8,2023-02-13,2023-02-19,0.417967,Week,sil_beta_demo_score,1.1.0,FPD30,Overall
9,2023-02-20,2023-02-26,-0.047710,Week,sil_beta_demo_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [197]:
sq = """WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fspd_30 = 1
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,3037897,a95f350b-e8a3-4c43-8e1e-b028dc35b96b,60830378970023,0.148720346,2025-06-10 11:15:13,2025-06-10,2025-06,Test,0,1,SIL-Instore
1,3083350,5a13269d-c09b-4448-b18e-f4eda8e93fce,60830833500022,0.1209918852,2025-06-10 12:20:15,2025-06-10,2025-06,Test,1,1,SIL-Instore
2,3488608,d756fb29-d3f1-4ef2-8760-b4a405024ee4,60834886080015,0.0582132504,2025-06-10 10:45:01,2025-06-10,2025-06,Test,0,1,SIL-Instore
3,3488625,762b695b-a9c7-4c71-b913-66f9f417c0f4,60834886250016,0.0570834443,2025-06-10 10:55:35,2025-06-10,2025-06,Test,0,1,SIL-Instore
4,3488648,cf89e1df-3799-4a6c-9cc4-a44e60c04bdd,60834886480013,0.1249459568,2025-06-10 11:22:12,2025-06-10,2025-06,Test,0,1,SIL-Instore


In [198]:
df1 = dfd.copy()

## Train

In [199]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fspd_30 = 1
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,1862251,9157dc31-ebf0-4f33-b3db-e73c4d56aa28,60818622510012,0.041945,2023-01-15 11:02:55,2023-01-15,2023-01,Train,0,1,SIL-Instore
1,2087590,a093c6ba-6b74-42c0-9a4f-647e068cb48b,60820875900017,0.049937,2023-06-11 12:27:59,2023-06-11,2023-06,Train,0,1,SIL-Instore
2,2050535,0fb64866-441f-40d1-adc7-c78002b6f640,60820505350017,0.149591,2023-05-18 17:31:37,2023-05-18,2023-05,Train,0,1,SIL-Instore
3,2041246,eeff27c4-ff94-41d1-a429-d04e477b336c,60820412460011,0.070063,2023-05-12 17:11:42,2023-05-12,2023-05,Train,1,1,SIL-Instore
4,2007393,224766b6-a69e-4760-8166-1154cd725411,60820073930014,0.117972,2023-04-21 16:33:31,2023-04-21,2023-04,Train,0,1,SIL-Instore


In [200]:
df2 = dfd.copy()

In [201]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245665 entries, 0 to 245664
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             245665 non-null  object        
 1   digitalLoanAccountId   245665 non-null  object        
 2   loanAccountNumber      245665 non-null  object        
 3   sil_beta_demo_score    245665 non-null  object        
 4   appln_submit_datetime  245665 non-null  datetime64[us]
 5   disbursementdate       245665 non-null  dbdate        
 6   Application_month      245665 non-null  object        
 7   Data_selection         245665 non-null  object        
 8   deffspd30              245665 non-null  Int64         
 9   flg_mature_fspd_30     245665 non-null  Int64         
 10  new_loan_type          245665 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 21.1+ MB


In [202]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [203]:
# df_concat.to_csv(r"sil_beta_demo_scorefspd30.csv")

In [204]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_demo_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_demo_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)                                                             

In [205]:
f3 = gini_results.copy()

In [206]:
f3.head(10)

,start_date,end_date,sil_beta_demo_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.250960,Month,sil_beta_demo_score,1.1.0,FSPD30,Overall
1,2023-01-02,2023-01-08,0.355795,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
2,2023-01-09,2023-01-15,0.235256,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
3,2023-01-16,2023-01-22,0.371812,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
4,2023-01-23,2023-01-29,0.131773,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
5,2023-01-30,2023-02-05,0.066116,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
6,2023-02-01,2023-02-28,0.120064,Month,sil_beta_demo_score,1.1.0,FSPD30,Overall
7,2023-02-06,2023-02-12,0.150435,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
8,2023-02-13,2023-02-19,0.309559,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall
9,2023-02-20,2023-02-26,-0.036571,Week,sil_beta_demo_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [207]:
sq = """WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()


,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,3390438,def6e4c5-de59-4d2e-b4ea-8692ff122bc2,60833904380012,0.0376010203,2025-04-20 10:20:50,2025-04-23,2025-04,Test,0,1,SIL-Instore
1,3390448,3a1d0848-8fcf-472f-a965-db58540c9230,60833904480014,0.1482116117,2025-04-20 10:31:07,2025-04-20,2025-04,Test,0,1,SIL-Instore
2,3390776,657bff20-70e5-4a9a-8ab8-9a257f64d6d0,60833907760017,0.1918704632,2025-04-20 12:43:37,2025-04-20,2025-04,Test,0,1,SIL-Instore
3,3390824,4c7bfc9b-4b39-4264-abb2-f8ee1fd1e3a6,60833908240018,0.0376889123,2025-04-20 12:58:38,2025-04-20,2025-04,Test,0,1,SIL-Instore
4,3390562,bcab7f3b-b855-4c37-88a1-9dc60871ae22,60833905620015,0.2421042693,2025-04-20 11:16:34,2025-04-20,2025-04,Test,0,1,SIL-Instore


In [208]:
df1 = dfd.copy()

## Train

In [209]:
sq = """ 
WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2099758,3c0df6db-9965-4660-bce2-350c947ceaf5,60820997580016,0.108052,2023-06-19 15:08:47,2023-06-19,2023-06,Train,0,1,SIL-Instore
1,1854827,1dad0aec-3d5d-44c6-9c2e-1aadc1ab7278,60818548270013,0.055964,2023-01-08 15:08:17,2023-01-08,2023-01,Train,0,1,SIL-Instore
2,2023085,8289480f-dfeb-4642-ba79-35034502ee6f,60820230850019,0.115756,2023-05-01 16:01:25,2023-05-01,2023-05,Train,0,1,SIL-Instore
3,2086653,d34207a6-69b7-45fe-9e9a-ae85de26fd75,60820866530011,0.095296,2023-06-10 17:24:05,2023-06-10,2023-06,Train,0,1,SIL-Instore
4,2010529,2f78776a-7dfa-4548-819e-a936f28eb16b,60820105290019,0.098040,2023-04-23 18:07:21,2023-04-23,2023-04,Train,0,1,SIL-Instore


In [210]:
df2 = dfd.copy()

In [211]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230343 entries, 0 to 230342
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             230343 non-null  object        
 1   digitalLoanAccountId   230343 non-null  object        
 2   loanAccountNumber      230343 non-null  object        
 3   sil_beta_demo_score    230343 non-null  object        
 4   appln_submit_datetime  230343 non-null  datetime64[us]
 5   disbursementdate       230343 non-null  dbdate        
 6   Application_month      230343 non-null  object        
 7   Data_selection         230343 non-null  object        
 8   deffstpd30             230343 non-null  Int64         
 9   flg_mature_fstpd_30    230343 non-null  Int64         
 10  new_loan_type          230343 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 19.8+ MB


In [212]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [213]:
# df_concat.to_csv(r"sil_beta_demo_scorefstpd30.csv")

In [214]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_demo_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_demo_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [215]:
f4 = gini_results.copy()

In [216]:
f4.head(10)

,start_date,end_date,sil_beta_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.253371,Month,sil_beta_demo_score,1.1.0,FSTPD30,Overall
1,2023-01-02,2023-01-08,0.335695,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
2,2023-01-09,2023-01-15,0.219488,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
3,2023-01-16,2023-01-22,0.363712,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
4,2023-01-23,2023-01-29,0.146972,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
5,2023-01-30,2023-02-05,0.121693,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
6,2023-02-01,2023-02-28,0.128934,Month,sil_beta_demo_score,1.1.0,FSTPD30,Overall
7,2023-02-06,2023-02-12,0.118855,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
8,2023-02-13,2023-02-19,0.343576,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall
9,2023-02-20,2023-02-26,-0.047750,Week,sil_beta_demo_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [217]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sil_beta_demo_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'sil_beta_demo_score_FPD10_gini', 'sil_beta_demo_score_FPD30_gini',
       'sil_beta_demo_score_FSPD30_gini',
       'sil_beta_demo_score_FSTPD30_gini'], dtype=object)

In [218]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','sil_beta_demo_score_FPD0_gini','sil_beta_demo_score_FPD10_gini', 'sil_beta_demo_score_FPD30_gini',
       'sil_beta_demo_score_FSPD30_gini',
       'sil_beta_demo_score_FSTPD30_gini']].copy()
final_df['Trench_category'] = 'All'
final_df['Model_display_name'] = 'beta_demo_model_sil'
final_df['Product_type'] = 'SIL'
final_df.dtypes

start_date                          datetime64[ns]
end_date                            datetime64[ns]
period                                      object
Model_Name                                  object
version                                     object
loan_type                                   object
bad_rate                                    object
sil_beta_demo_score_FPD0_gini              float64
sil_beta_demo_score_FPD10_gini             float64
sil_beta_demo_score_FPD30_gini             float64
sil_beta_demo_score_FSPD30_gini            float64
sil_beta_demo_score_FSTPD30_gini           float64
Trench_category                             object
Model_display_name                          object
Product_type                                object
dtype: object

In [219]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,sil_beta_demo_score_FPD0_gini,sil_beta_demo_score_FPD10_gini,sil_beta_demo_score_FPD30_gini,sil_beta_demo_score_FSPD30_gini,sil_beta_demo_score_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,sil_beta_demo_score,1.1.0,Overall,FPD0,0.141958,NaN,NaN,NaN,NaN,All,beta_demo_model_sil,SIL
1,2023-01-02,2023-01-08,Week,sil_beta_demo_score,1.1.0,Overall,FPD0,0.279249,NaN,NaN,NaN,NaN,All,beta_demo_model_sil,SIL
2,2023-01-09,2023-01-15,Week,sil_beta_demo_score,1.1.0,Overall,FPD0,0.118033,NaN,NaN,NaN,NaN,All,beta_demo_model_sil,SIL
3,2023-01-16,2023-01-22,Week,sil_beta_demo_score,1.1.0,Overall,FPD0,0.153020,NaN,NaN,NaN,NaN,All,beta_demo_model_sil,SIL
4,2023-01-23,2023-01-29,Week,sil_beta_demo_score,1.1.0,Overall,FPD0,-0.006286,NaN,NaN,NaN,NaN,All,beta_demo_model_sil,SIL


In [220]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.sil_beta_demo_score_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=61a7d6ec-2599-483a-8119-4c2bfdd0d430>

# Beta SIL STACK Score Model

## FPD0

## Test

In [221]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd0 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2693919,c7f044d3-bb9c-41eb-8589-ec39e6ed088f,60826939190017,0.17907621460154732,2025-10-04 11:31:56,2025-10-04,2025-10,Test,0,1,SIL-Instore
1,3126517,96702cba-a54d-4874-99a7-f244e351505f,60831265170017,0.11619971900923692,2025-10-04 11:27:23,2025-10-04,2025-10,Test,0,1,SIL-Instore
2,3132041,fb9b64fe-08c1-4602-8c49-8f5ea9231c6c,60831320410029,0.13691971553649196,2025-10-04 11:14:14,2025-10-04,2025-10,Test,0,1,SIL-Instore
3,3451823,6415bd20-c879-4f81-895e-ecf0b7600f0c,60834518230017,0.07925914529695802,2025-10-04 11:36:31,2025-10-04,2025-10,Test,0,1,SIL ZERO
4,3722004,4f752a10-cfb7-4a63-b8aa-6f39e2b84728,60837220040017,0.030606823567872612,2025-10-04 10:35:48,2025-10-04,2025-10,Test,0,1,SIL Competitor


In [222]:
df1 = dfd.copy()

## Train

In [223]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd0 = 1
  ;

"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2089656,742a531d-9c43-4a55-8301-3acd1fa8c937,60820896560016,0.148851,2023-06-12 18:34:09,2023-06-12,2023-06,Train,0,1,SIL-Instore
1,2080302,8600beb4-68fb-4da9-8ecc-42f8c69b5d14,60820803020017,0.051647,2023-06-06 13:29:16,2023-06-06,2023-06,Train,0,1,SIL-Instore
2,1904533,e7719c24-697d-4d53-a794-69c90b1e0996,60819045330013,0.020308,2023-02-17 12:07:34,2023-02-17,2023-02,Train,0,1,SIL-Instore
3,2075939,e62af1f3-a2f3-4681-81dc-1ddfd0d2c3ad,60820759390013,0.100153,2023-06-03 18:32:32,2023-06-03,2023-06,Train,0,1,SIL-Instore
4,1576563,064ae1f0-49be-4340-9893-f6b5c819d1fa,60815765630018,0.190517,2023-01-15 11:13:44,2023-01-15,2023-01,Train,0,1,SIL-Instore


In [224]:
df2 = dfd.copy()

In [225]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273188 entries, 0 to 273187
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             273188 non-null  object        
 1   digitalLoanAccountId   273188 non-null  object        
 2   loanAccountNumber      273188 non-null  object        
 3   sil_beta_stack_score   273188 non-null  object        
 4   appln_submit_datetime  273188 non-null  datetime64[us]
 5   disbursementdate       273188 non-null  dbdate        
 6   Application_month      273188 non-null  object        
 7   Data_selection         273188 non-null  object        
 8   deffpd0                273188 non-null  Int64         
 9   flg_mature_fpd0        273188 non-null  Int64         
 10  new_loan_type          273188 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 23.4+ MB


In [226]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_stack_score'] = pd.to_numeric(df_concat['sil_beta_stack_score'], errors='coerce')

In [227]:
# df_concat.to_csv(r"sil_beta_stack_scorefpd0.csv")

In [228]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_stack_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_stack_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [229]:
f0 = gini_results.copy()

In [230]:
f0.head(10)

,start_date,end_date,sil_beta_stack_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.207885,Month,sil_beta_stack_score,1.1.0,FPD0,Overall
1,2023-01-02,2023-01-08,0.330022,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
2,2023-01-09,2023-01-15,0.260013,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
3,2023-01-16,2023-01-22,0.234899,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
4,2023-01-23,2023-01-29,0.045143,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
5,2023-01-30,2023-02-05,0.348225,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
6,2023-02-01,2023-02-28,0.288730,Month,sil_beta_stack_score,1.1.0,FPD0,Overall
7,2023-02-06,2023-02-12,0.316416,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
8,2023-02-13,2023-02-19,0.318935,Week,sil_beta_stack_score,1.1.0,FPD0,Overall
9,2023-02-20,2023-02-26,0.257971,Week,sil_beta_stack_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [231]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd10 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3632133,013dfd00-63d2-4f7c-bca4-9afd22bf63ac,60836321330014,0.08940994761840597,2025-08-20 12:43:08,2025-08-20,2025-08,Test,0,1,SIL-Instore
1,2531597,05df84b3-a153-45a5-b603-bd52e0c3be62,60825315970023,0.05173408534461259,2025-08-20 14:13:36,2025-08-20,2025-08,Test,0,1,SIL Competitor
2,3632096,0f7bfabd-9276-40bb-b838-07bf82e44540,60836320960016,0.45048844581065267,2025-08-20 12:37:31,2025-08-20,2025-08,Test,1,1,SIL-Instore
3,3631972,1c4553fd-7a34-45ec-a811-26c63b0e4773,60836319720011,0.0514574072268597,2025-08-20 12:38:37,2025-08-20,2025-08,Test,0,1,SIL Competitor
4,3632127,21be63ee-b40a-45f6-abcc-889c1288b594,60836321270016,0.06119608154876716,2025-08-20 13:04:06,2025-08-20,2025-08,Test,0,1,SIL Competitor


In [232]:
df1 = dfd.copy()

## Train

In [234]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd10 = 1
  ;

"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2000434,0f9397e4-f1ec-4405-8394-cd2c9c62f30e,60820004340018,0.069159,2023-04-17 19:11:56,2023-04-17,2023-04,Train,0,1,SIL-Instore
1,2065846,7f181edd-fc8a-45cf-acaf-4eee2a5fccb4,60820658460014,0.021803,2023-05-28 20:03:52,2023-05-28,2023-05,Train,0,1,SIL-Instore
2,1895182,e19439d2-c58e-47b4-819b-5a232bcd3a5d,60818951820013,0.079350,2023-02-10 19:46:35,2023-02-10,2023-02,Train,0,1,SIL-Instore
3,2064099,9b360a84-8344-4cd1-910d-353539d7cd20,60820640990019,0.108226,2023-05-27 17:31:36,2023-05-27,2023-05,Train,0,1,SIL-Instore
4,1897229,ac44427e-1874-402b-bfcb-59c3f57582c9,60818972290019,0.110595,2023-02-12 11:22:49,2023-02-12,2023-02,Train,0,1,SIL-Instore


In [235]:
df2 = dfd.copy()

In [236]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268738 entries, 0 to 268737
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             268738 non-null  object        
 1   digitalLoanAccountId   268738 non-null  object        
 2   loanAccountNumber      268738 non-null  object        
 3   sil_beta_stack_score   268738 non-null  object        
 4   appln_submit_datetime  268738 non-null  datetime64[us]
 5   disbursementdate       268738 non-null  dbdate        
 6   Application_month      268738 non-null  object        
 7   Data_selection         268738 non-null  object        
 8   deffpd10               268738 non-null  Int64         
 9   flg_mature_fpd10       268738 non-null  Int64         
 10  new_loan_type          268738 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 23.1+ MB


In [237]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_stack_score'] = pd.to_numeric(df_concat['sil_beta_stack_score'], errors='coerce')

In [238]:
# df_concat.to_csv(r"sil_beta_stack_scorefpd10.csv")

In [239]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_stack_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_stack_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [240]:
f1 = gini_results.copy()

In [241]:
f1.head(10)

,start_date,end_date,sil_beta_stack_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.295238,Month,sil_beta_stack_score,1.1.0,FPD10,Overall
1,2023-01-02,2023-01-08,0.390082,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
2,2023-01-09,2023-01-15,0.302997,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
3,2023-01-16,2023-01-22,0.475325,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
4,2023-01-23,2023-01-29,0.189446,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
5,2023-01-30,2023-02-05,0.308190,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
6,2023-02-01,2023-02-28,0.269499,Month,sil_beta_stack_score,1.1.0,FPD10,Overall
7,2023-02-06,2023-02-12,0.338139,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
8,2023-02-13,2023-02-19,0.272636,Week,sil_beta_stack_score,1.1.0,FPD10,Overall
9,2023-02-20,2023-02-26,0.201709,Week,sil_beta_stack_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [242]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2935708,5feacb0e-b295-427a-b8e1-5e64e51b2e2c,60829357080022,0.16911975470863502,2025-08-31 14:58:22,2025-08-31,2025-08,Test,0,1,SIL-Instore
1,3158907,c8398f8c-90e6-4667-93b2-d39e071f7cd2,60831589070011,0.04024126977077865,2025-08-31 14:35:53,2025-08-31,2025-08,Test,0,1,SIL Competitor
2,3349760,f837a96b-b357-4ab7-a5ec-df39b40b6890,60833497600011,0.02261857530060947,2025-08-31 13:55:39,2025-08-31,2025-08,Test,0,1,SIL-Instore
3,3378851,8786eef0-6d20-4dd2-9fa1-984618815ab5,60833788510013,0.22349075437917312,2025-08-31 13:33:11,2025-08-31,2025-08,Test,0,1,SIL Competitor
4,3653885,74361219-88eb-46fc-ba82-7c9e3e2fe2b5,60836538850011,0.04312675898569908,2025-08-31 13:21:24,2025-08-31,2025-08,Test,0,1,SIL-Instore


In [243]:
df1 = dfd.copy()

## Train

In [244]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fpd30 = 1
  ;

"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2076709,8f2ff649-fee9-4806-90be-4aac76a22da9,60820767090015,0.384236,2023-06-04 10:26:37,2023-06-04,2023-06,Train,0,1,SIL-Instore
1,2046673,33990289-05ad-4c50-bb6f-97d307b9cadc,60820466730015,0.053931,2023-05-16 10:55:02,2023-05-16,2023-05,Train,0,1,SIL-Instore
2,1972724,ab30cefb-127e-423a-9c25-c6e3a61bc845,60819727240015,0.048634,2023-04-01 17:30:45,2023-04-01,2023-04,Train,0,1,SIL-Instore
3,1967217,6dbbdec5-aa87-49fa-9093-5ea3ed9f51c4,60819672170016,0.120325,2023-03-29 18:02:56,2023-03-29,2023-03,Train,0,1,SIL-Instore
4,2106124,01010a51-837d-4ad3-8321-9bdb2446e3a6,60821061240017,0.197371,2023-06-24 13:49:44,2023-06-24,2023-06,Train,0,1,SIL-Instore


In [245]:
df2 = dfd.copy()

In [246]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258928 entries, 0 to 258927
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             258928 non-null  object        
 1   digitalLoanAccountId   258928 non-null  object        
 2   loanAccountNumber      258928 non-null  object        
 3   sil_beta_stack_score   258928 non-null  object        
 4   appln_submit_datetime  258928 non-null  datetime64[us]
 5   disbursementdate       258928 non-null  dbdate        
 6   Application_month      258928 non-null  object        
 7   Data_selection         258928 non-null  object        
 8   deffpd30               258928 non-null  Int64         
 9   flg_mature_fpd30       258928 non-null  Int64         
 10  new_loan_type          258928 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 22.2+ MB


In [247]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_stack_score'] = pd.to_numeric(df_concat['sil_beta_stack_score'], errors='coerce')

In [248]:
# df_concat.to_csv(r"sil_beta_stack_scorefpd30.csv")

In [249]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_stack_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_stack_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [250]:
f2 = gini_results.copy()

In [251]:
f2.head(10)

,start_date,end_date,sil_beta_stack_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.353518,Month,sil_beta_stack_score,1.1.0,FPD30,Overall
1,2023-01-02,2023-01-08,0.418182,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
2,2023-01-09,2023-01-15,0.563636,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
3,2023-01-16,2023-01-22,0.475325,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
4,2023-01-23,2023-01-29,0.133212,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
5,2023-01-30,2023-02-05,0.349288,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
6,2023-02-01,2023-02-28,0.284919,Month,sil_beta_stack_score,1.1.0,FPD30,Overall
7,2023-02-06,2023-02-12,0.282517,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
8,2023-02-13,2023-02-19,0.288865,Week,sil_beta_stack_score,1.1.0,FPD30,Overall
9,2023-02-20,2023-02-26,0.332061,Week,sil_beta_stack_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [252]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,3354990,ffd3ba3d-5a6e-4a2f-ba72-64a659d1ca54,60833549900017,0.057728939638435985,2025-04-02 12:22:23,2025-04-02,2025-04,Test,0,1,SIL-Instore
1,3355084,7a04ce41-2ed7-4e1c-a147-938a7314edbf,60833550840016,0.056980784240014,2025-04-02 13:07:01,2025-04-02,2025-04,Test,0,1,SIL-Instore
2,3231246,b7ed213e-2608-4838-bc65-cf7603a0210c,60832312460014,0.10867777966084624,2025-04-02 12:08:19,2025-04-02,2025-04,Test,0,1,SIL-Instore
3,2497801,40bcd42e-ece4-413c-9492-233e73f7a39d,60824978010026,0.02287316476734115,2025-04-02 12:17:07,2025-04-02,2025-04,Test,0,1,SIL-Instore
4,3354788,8d4f97f1-cabb-49a8-9543-0896f54bac99,60833547880017,0.0607531805984611,2025-04-02 10:30:13,2025-04-02,2025-04,Test,0,1,SIL-Instore


In [253]:
df1 = dfd.copy()

## Train

In [254]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  ;

"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2041292,c2c51457-ddcc-4b12-a73f-2827c5dd3eab,60820412920016,0.167527,2023-05-12 17:41:42,2023-05-12,2023-05,Train,0,1,SIL-Instore
1,1972324,aa1cb9b7-6485-4802-a96a-388ff39fa19f,60819723240013,0.021552,2023-04-01 13:57:42,2023-04-01,2023-04,Train,0,1,SIL-Instore
2,1881056,8fe56690-cea4-44ea-8a2f-e0406653ac08,60818810560017,0.178551,2023-01-31 18:15:24,2023-01-31,2023-01,Train,0,1,SIL-Instore
3,1989803,46b29c1e-0f20-4071-beb6-bddbb80b79f6,60819898030016,0.054083,2023-04-11 19:03:01,2023-04-11,2023-04,Train,0,1,SIL-Instore
4,2006944,a3ddc8c3-11d6-4ac0-a9f0-f03435522fe8,60820069440019,0.180432,2023-04-21 12:30:33,2023-04-21,2023-04,Train,0,1,SIL-Instore


In [255]:
df2 = dfd.copy()

In [256]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245665 entries, 0 to 245664
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             245665 non-null  object        
 1   digitalLoanAccountId   245665 non-null  object        
 2   loanAccountNumber      245665 non-null  object        
 3   sil_beta_stack_score   245665 non-null  object        
 4   appln_submit_datetime  245665 non-null  datetime64[us]
 5   disbursementdate       245665 non-null  dbdate        
 6   Application_month      245665 non-null  object        
 7   Data_selection         245665 non-null  object        
 8   deffspd30              245665 non-null  Int64         
 9   flg_mature_fspd_30     245665 non-null  Int64         
 10  new_loan_type          245665 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 21.1+ MB


In [257]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_stack_score'] = pd.to_numeric(df_concat['sil_beta_stack_score'], errors='coerce')

In [258]:
# df_concat.to_csv(r"sil_beta_stack_scorefspd30.csv")

In [259]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_stack_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_stack_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [260]:
f3 = gini_results.copy()

In [261]:
f3.head(10)

,start_date,end_date,sil_beta_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.310982,Month,sil_beta_stack_score,1.1.0,FSPD30,Overall
1,2023-01-02,2023-01-08,0.309075,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
2,2023-01-09,2023-01-15,0.405128,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
3,2023-01-16,2023-01-22,0.485906,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
4,2023-01-23,2023-01-29,0.200000,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
5,2023-01-30,2023-02-05,0.309917,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
6,2023-02-01,2023-02-28,0.300713,Month,sil_beta_stack_score,1.1.0,FSPD30,Overall
7,2023-02-06,2023-02-12,0.366957,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
8,2023-02-13,2023-02-19,0.311029,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall
9,2023-02-20,2023-02-26,0.225143,Week,sil_beta_stack_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [262]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  ),
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2847730,3ca12099-24dc-4fa9-97a5-8fb89297ec18,60828477300027,0.07090413190727753,2025-04-01 16:31:52,2025-04-01,2025-04,Test,0,1,SIL-Instore
1,3467716,888c3b16-3bb8-422a-bdfb-6916926e26c1,60834677160014,0.052493762951970085,2025-05-30 15:24:23,2025-05-30,2025-05,Test,0,1,SIL-Instore
2,3346550,5d42d7b6-c035-4f9e-86d3-2f8302e71bbd,60833465500011,0.04510482463194055,2025-03-29 11:29:37,2025-03-31,2025-03,Test,0,1,SIL-Instore
3,3414960,981eb920-5924-446d-a472-74719e58ad83,60834149600017,0.05419306258620756,2025-05-02 13:18:33,2025-05-02,2025-05,Test,0,1,SIL-Instore
4,3350570,7d43f7fc-edaf-482f-a1d3-4421c1e9dce4,60833505700015,0.0944055894400065,2025-03-31 10:12:42,2025-03-31,2025-03,Test,0,1,SIL-Instore


In [263]:
df1 = dfd.copy()

## Train

In [264]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  and modelVersionId = 'v1'
  ),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_stack_score
  from cleaned
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.sil_beta_stack_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  ;

"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,sil_beta_stack_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2111260,4cf3316d-8b4d-4bb0-9bb4-7574da398a09,60821112600019,0.013553,2023-06-28 19:19:32,2023-06-28,2023-06,Train,0,1,SIL-Instore
1,2018024,a344b014-ae5e-44d3-b4ed-4928488a520f,60820180240011,0.077267,2023-04-28 17:51:35,2023-04-28,2023-04,Train,0,1,SIL-Instore
2,2056550,ed7d1987-3404-4049-867f-9291f1c228d7,60820565500011,0.087474,2023-05-22 17:42:40,2023-05-22,2023-05,Train,0,1,SIL-Instore
3,2083859,a551e203-402b-4de0-8baf-200655a2f1bf,60820838590011,0.117142,2023-06-08 16:41:36,2023-06-08,2023-06,Train,0,1,SIL-Instore
4,2009090,3f71179e-494a-405d-87ec-f21f576e9da8,60820090900011,0.160525,2023-04-22 18:10:02,2023-04-22,2023-04,Train,0,1,SIL-Instore


In [265]:
df2 = dfd.copy()

In [266]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230343 entries, 0 to 230342
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   customerId             230343 non-null  object        
 1   digitalLoanAccountId   230343 non-null  object        
 2   loanAccountNumber      230343 non-null  object        
 3   sil_beta_stack_score   230343 non-null  object        
 4   appln_submit_datetime  230343 non-null  datetime64[us]
 5   disbursementdate       230343 non-null  dbdate        
 6   Application_month      230343 non-null  object        
 7   Data_selection         230343 non-null  object        
 8   deffstpd30             230343 non-null  Int64         
 9   flg_mature_fstpd_30    230343 non-null  Int64         
 10  new_loan_type          230343 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(7)
memory usage: 19.8+ MB


In [267]:
df_concat['Application_month'].value_counts(dropna=False).sort_index()
df_concat['sil_beta_stack_score'] = pd.to_numeric(df_concat['sil_beta_stack_score'], errors='coerce')

In [268]:
# df_concat.to_csv(r"sil_beta_stack_scorefstpd30.csv")

In [269]:
# gini_results = calculate_periodic_gini(df_concat, 'sil_beta_stack_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'sil_beta_stack_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [270]:
f4 = gini_results.copy()

In [271]:
f4.head(10)

,start_date,end_date,sil_beta_stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.317218,Month,sil_beta_stack_score,1.1.0,FSTPD30,Overall
1,2023-01-02,2023-01-08,0.304604,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
2,2023-01-09,2023-01-15,0.367139,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
3,2023-01-16,2023-01-22,0.373654,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
4,2023-01-23,2023-01-29,0.292769,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
5,2023-01-30,2023-02-05,0.362257,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
6,2023-02-01,2023-02-28,0.325214,Month,sil_beta_stack_score,1.1.0,FSTPD30,Overall
7,2023-02-06,2023-02-12,0.435070,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
8,2023-02-13,2023-02-19,0.326577,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall
9,2023-02-20,2023-02-26,0.167126,Week,sil_beta_stack_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [272]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sil_beta_stack_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'sil_beta_stack_score_FPD10_gini',
       'sil_beta_stack_score_FPD30_gini',
       'sil_beta_stack_score_FSPD30_gini',
       'sil_beta_stack_score_FSTPD30_gini'], dtype=object)

In [273]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type','bad_rate','sil_beta_stack_score_FPD0_gini','sil_beta_stack_score_FPD10_gini',
       'sil_beta_stack_score_FPD30_gini',
       'sil_beta_stack_score_FSPD30_gini',
       'sil_beta_stack_score_FSTPD30_gini']].copy()
final_df['Trench_category'] = 'All'
final_df['Model_display_name'] = 'beta_stack_model_sil'
final_df['Product_type'] = 'SIL'
final_df.dtypes

start_date                           datetime64[ns]
end_date                             datetime64[ns]
period                                       object
Model_Name                                   object
version                                      object
loan_type                                    object
bad_rate                                     object
sil_beta_stack_score_FPD0_gini              float64
sil_beta_stack_score_FPD10_gini             float64
sil_beta_stack_score_FPD30_gini             float64
sil_beta_stack_score_FSPD30_gini            float64
sil_beta_stack_score_FSTPD30_gini           float64
Trench_category                              object
Model_display_name                           object
Product_type                                 object
dtype: object

In [274]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,sil_beta_stack_score_FPD0_gini,sil_beta_stack_score_FPD10_gini,sil_beta_stack_score_FPD30_gini,sil_beta_stack_score_FSPD30_gini,sil_beta_stack_score_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,sil_beta_stack_score,1.1.0,Overall,FPD0,0.207885,NaN,NaN,NaN,NaN,All,beta_stack_model_sil,SIL
1,2023-01-02,2023-01-08,Week,sil_beta_stack_score,1.1.0,Overall,FPD0,0.330022,NaN,NaN,NaN,NaN,All,beta_stack_model_sil,SIL
2,2023-01-09,2023-01-15,Week,sil_beta_stack_score,1.1.0,Overall,FPD0,0.260013,NaN,NaN,NaN,NaN,All,beta_stack_model_sil,SIL
3,2023-01-16,2023-01-22,Week,sil_beta_stack_score,1.1.0,Overall,FPD0,0.234899,NaN,NaN,NaN,NaN,All,beta_stack_model_sil,SIL
4,2023-01-23,2023-01-29,Week,sil_beta_stack_score,1.1.0,Overall,FPD0,0.045143,NaN,NaN,NaN,NaN,All,beta_stack_model_sil,SIL


In [275]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.sil_beta_stack_score_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5931a7c5-a6fa-49be-a45e-00192c8934fd>

# Alpha-Cash-CIC-Model

## Trench 1

## FPD0

## Test

In [276]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3760011,f6061dee-d6ca-4379-8b39-619687cd9169,60837600110018,0.41030006546307835,Trench 1,2025-10-22 09:34:02,2025-10-22,2025-10,Test,0,1,Quick
1,3760325,b8e92633-80c3-44b3-8e2d-46164691a22e,60837603250013,0.42385541057934334,Trench 1,2025-10-22 12:02:07,2025-10-24,2025-10,Test,0,1,Quick
2,3760248,2f385822-d3d4-4c8c-856b-9972040eb1be,60837602480017,0.5056282103081561,Trench 1,2025-10-22 11:23:32,2025-10-22,2025-10,Test,0,1,Quick
3,3760577,2b9cfdc3-e5cc-4363-898b-dc6050cd4a84,60837605770011,0.4799418478751456,Trench 1,2025-10-22 13:39:40,2025-10-22,2025-10,Test,0,1,Quick
4,3760632,6e4fb40d-a378-41a1-abc3-d698f19317ef,60837606320011,0.47239918585227825,Trench 1,2025-10-22 14:06:45,2025-10-22,2025-10,Test,0,1,Quick


In [277]:
df1 = dfd.copy()

## Train

In [278]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2843049,bf71016f-c356-4367-bd72-54b2699cad14,60828430490013,0.606567,Trench 1,2024-09-13 17:51:48,2024-09-13,2024-09,Train,1,1,Quick
1,2824056,a0e4ae3f-0010-4da1-bcd6-e1bc6152fba0,60828240560016,0.450209,Trench 1,2024-09-06 05:27:43,2024-09-07,2024-09,Train,1,1,Quick
2,2885434,368276ec-8bf6-477d-9874-cd6d875da29d,60828854340016,0.609640,Trench 1,2024-09-27 12:54:01,2024-09-27,2024-09,Train,0,1,Quick
3,2792070,57c8cc5e-b74f-484c-81a1-4c2923dbe82d,60827920700015,0.436697,Trench 1,2024-09-07 15:48:30,2024-09-07,2024-09,Train,1,1,Quick
4,2860206,afbf2bdc-0b46-4a5c-b6ad-e8eda9ae1f9f,60828602060016,0.479148,Trench 1,2024-09-19 12:56:38,2024-09-20,2024-09,Train,1,1,Quick


In [279]:
df2 = dfd.copy()

In [280]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17490 entries, 0 to 17489
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             17490 non-null  object        
 1   digitalLoanAccountId   17490 non-null  object        
 2   loanAccountNumber      17490 non-null  object        
 3   aCicScore              17490 non-null  object        
 4   trenchCategory         17490 non-null  object        
 5   appln_submit_datetime  17490 non-null  datetime64[us]
 6   disbursementdate       17490 non-null  dbdate        
 7   Application_month      17490 non-null  object        
 8   Data_selection         17490 non-null  object        
 9   deffpd0                17490 non-null  Int64         
 10  flg_mature_fpd0        17490 non-null  Int64         
 11  new_loan_type          17490 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [281]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [282]:
# df_concat.to_csv(r"aCicScorefpd0.csv")

In [283]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [284]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aCicScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,-0.170940,Week,aCicScore,1.1.0,FPD0,Overall
1,2024-09-01,2024-09-30,0.217332,Month,aCicScore,1.1.0,FPD0,Overall
2,2024-09-02,2024-09-08,0.187550,Week,aCicScore,1.1.0,FPD0,Overall
3,2024-09-09,2024-09-15,0.286166,Week,aCicScore,1.1.0,FPD0,Overall
4,2024-09-16,2024-09-22,0.271593,Week,aCicScore,1.1.0,FPD0,Overall


In [285]:
f0['loan_type'].value_counts()

loan_type
Overall    76
Quick      76
Name: count, dtype: int64

## FPD10

## Test

In [286]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3713545,3d4efb65-52d5-4ff2-8740-8f499b6491c7,60837135450011,0.4424019440049859,Trench 1,2025-09-29 19:36:21,2025-09-30,2025-09,Test,0,1,Quick
1,3714449,d1de89c0-b375-4500-8618-d63ea29d2bcf,60837144490015,0.4771895198564867,Trench 1,2025-09-30 12:35:10,2025-10-24,2025-09,Test,0,1,Quick
2,3714015,577e2c33-6a05-4f5d-ac37-de55ae801b98,60837140150013,0.41111921722304673,Trench 1,2025-09-30 09:03:02,2025-09-30,2025-09,Test,0,1,Quick
3,3713908,ff29594e-6d08-4439-9aff-12debc426602,60837139080017,0.3957903295529901,Trench 1,2025-09-30 17:11:30,2025-09-30,2025-09,Test,1,1,Quick
4,3714868,bdda830c-3985-463e-b8bf-a4d02bd00a03,60837148680011,0.42478178955732115,Trench 1,2025-09-30 15:32:10,2025-10-02,2025-09,Test,0,1,Quick


In [287]:
df1 = dfd.copy()

## Train

In [288]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2893884,5a2736b8-3421-44d3-bd45-f4b423a04ee4,60828938840015,0.491640,Trench 1,2024-09-30 23:15:42,2024-10-01,2024-09,Train,0,1,Quick
1,2842024,dad56691-9e39-4d16-b142-1065b19d7ea1,60828420240011,0.375594,Trench 1,2024-09-13 12:41:54,2024-09-13,2024-09,Train,1,1,Quick
2,2818670,904ac9a1-23c6-46f1-8f9e-2f75184393f8,60828186700013,0.400789,Trench 1,2024-09-03 03:15:45,2024-09-03,2024-09,Train,0,1,Quick
3,2789572,16e043b5-af24-498f-8833-155f75916e0b,60827895720015,0.544169,Trench 1,2024-09-01 13:51:02,2024-09-01,2024-09,Train,0,1,Quick
4,2822204,4bee3aff-bb39-4e04-bae9-c93faa93cf8c,60828222040012,0.478889,Trench 1,2024-09-04 17:52:32,2024-09-04,2024-09,Train,0,1,Quick


In [289]:
df2 = dfd.copy()

In [290]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17006 entries, 0 to 17005
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             17006 non-null  object        
 1   digitalLoanAccountId   17006 non-null  object        
 2   loanAccountNumber      17006 non-null  object        
 3   aCicScore              17006 non-null  object        
 4   trenchCategory         17006 non-null  object        
 5   appln_submit_datetime  17006 non-null  datetime64[us]
 6   disbursementdate       17006 non-null  dbdate        
 7   Application_month      17006 non-null  object        
 8   Data_selection         17006 non-null  object        
 9   deffpd10               17006 non-null  Int64         
 10  flg_mature_fpd10       17006 non-null  Int64         
 11  new_loan_type          17006 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [291]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [292]:
# df_concat.to_csv(r"aCicScorefpd10.csv")

In [293]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [294]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aCicScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.036585,Week,aCicScore,1.1.0,FPD10,Overall
1,2024-09-01,2024-09-30,0.236056,Month,aCicScore,1.1.0,FPD10,Overall
2,2024-09-02,2024-09-08,0.183358,Week,aCicScore,1.1.0,FPD10,Overall
3,2024-09-09,2024-09-15,0.334879,Week,aCicScore,1.1.0,FPD10,Overall
4,2024-09-16,2024-09-22,0.278113,Week,aCicScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [295]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [296]:
df1 = dfd.copy()

## Train

In [297]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2818243,a45a005d-1090-43bc-955a-ead1aea55a57,60828182430018,0.637890,Trench 1,2024-09-02 20:28:30,2024-09-02,2024-09,Train,1,1,Quick
1,2894648,210d09c6-279e-46bd-9eb4-7addb306f63b,60828946480019,0.382421,Trench 1,2024-09-29 20:26:49,2024-10-04,2024-09,Train,0,1,Quick
2,2885713,2d1d6a84-eb7c-4dbb-a4da-e0cb93ee89c7,60828857130011,0.403504,Trench 1,2024-09-27 14:03:33,2024-09-27,2024-09,Train,0,1,Quick
3,2886700,e26fae2f-57b3-42e1-9a61-9221286067ef,60828867000019,0.418806,Trench 1,2024-09-27 19:02:13,2024-09-27,2024-09,Train,0,1,Quick
4,2857266,148190bd-b77f-47cc-b0a1-8de93402c027,60828572660011,0.469935,Trench 1,2024-09-18 12:17:26,2024-09-18,2024-09,Train,0,1,Quick


In [298]:
df2 = dfd.copy()

In [299]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16215 entries, 0 to 16214
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16215 non-null  Int64         
 1   digitalLoanAccountId   16215 non-null  object        
 2   loanAccountNumber      16215 non-null  object        
 3   aCicScore              16215 non-null  float64       
 4   trenchCategory         16215 non-null  object        
 5   appln_submit_datetime  16215 non-null  datetime64[us]
 6   disbursementdate       16215 non-null  dbdate        
 7   Application_month      16215 non-null  object        
 8   Data_selection         16215 non-null  object        
 9   deffpd30               16215 non-null  Int64         
 10  flg_mature_fpd30       16215 non-null  Int64         
 11  new_loan_type          16215 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [300]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [301]:
# df_concat.to_csv(r"aCicScorefpd30.csv")

In [302]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [303]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aCicScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.036585,Week,aCicScore,1.1.0,FPD30,Overall
1,2024-09-01,2024-09-30,0.265266,Month,aCicScore,1.1.0,FPD30,Overall
2,2024-09-02,2024-09-08,0.241210,Week,aCicScore,1.1.0,FPD30,Overall
3,2024-09-09,2024-09-15,0.338013,Week,aCicScore,1.1.0,FPD30,Overall
4,2024-09-16,2024-09-22,0.292500,Week,aCicScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [304]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fpd30,new_loan_type


In [305]:
df1 = dfd.copy()

## Train

In [306]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2862332,3bc102c9-8893-4aa8-9eac-4b6fc90ab029,60828623320019,0.285362,Trench 1,2024-09-20 09:24:52,2024-09-20,2024-09,Train,1,1,Quick
1,2896323,9fc404b5-d6e3-4ec5-a03f-6e5ce11e6ae8,60828963230019,0.459477,Trench 1,2024-09-30 14:41:06,2024-09-30,2024-09,Train,1,1,Quick
2,2845559,5af2dbea-8398-47ac-b63b-da11a440832f,60828455590018,0.585309,Trench 1,2024-09-14 17:02:07,2024-09-14,2024-09,Train,0,1,Quick
3,2873858,db083354-fdaa-497b-85f8-32af97a5d09f,60828738580013,0.464543,Trench 1,2024-09-23 14:55:47,2024-09-23,2024-09,Train,0,1,Quick
4,2853136,fd9f5eb9-2113-4427-9636-0c2c5837944c,60828531360016,0.515183,Trench 1,2024-09-22 00:28:27,2024-09-22,2024-09,Train,1,1,Quick


In [307]:
df2 = dfd.copy()

In [308]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15042 entries, 0 to 15041
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             15042 non-null  Int64         
 1   digitalLoanAccountId   15042 non-null  object        
 2   loanAccountNumber      15042 non-null  object        
 3   aCicScore              15042 non-null  float64       
 4   trenchCategory         15042 non-null  object        
 5   appln_submit_datetime  15042 non-null  datetime64[us]
 6   disbursementdate       15042 non-null  dbdate        
 7   Application_month      15042 non-null  object        
 8   Data_selection         15042 non-null  object        
 9   deffspd30              15042 non-null  Int64         
 10  flg_mature_fspd_30     15042 non-null  Int64         
 11  new_loan_type          15042 non-null  object        
 12  flg_mature_fpd30       0 non-null      object        
dtypes

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [309]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [310]:
# df_concat.to_csv(r"aCicScorefspd30.csv")

In [311]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [312]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aCicScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.182432,Week,aCicScore,1.1.0,FSPD30,Overall
1,2024-09-01,2024-09-30,0.254053,Month,aCicScore,1.1.0,FSPD30,Overall
2,2024-09-02,2024-09-08,0.292380,Week,aCicScore,1.1.0,FSPD30,Overall
3,2024-09-09,2024-09-15,0.265693,Week,aCicScore,1.1.0,FSPD30,Overall
4,2024-09-16,2024-09-22,0.260664,Week,aCicScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [313]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [314]:
df1 = dfd.copy()

## Train

In [315]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2818354,c77b9833-353c-416d-8299-33a41c5a2612,60828183540014,0.476919,Trench 1,2024-09-02 21:14:57,2024-09-02,2024-09,Train,0,1,Quick
1,2898106,c6b84a2c-da75-4a29-9093-a2a022faecec,60828981060011,0.530989,Trench 1,2024-09-30 18:25:56,2024-09-30,2024-09,Train,0,1,Quick
2,2849475,d3ca678f-8e8c-4b9f-9736-2fb90a14ddb3,60828494750011,0.442424,Trench 1,2024-09-15 22:55:20,2024-09-16,2024-09,Train,0,1,Quick
3,2813700,d2b0c365-893b-4042-8d8f-49b4c438e7c0,60828137000012,0.421165,Trench 1,2024-09-01 12:27:27,2024-09-01,2024-09,Train,0,1,Quick
4,2850790,86f01f62-c1f4-4f58-953b-7e3321156f52,60828507900018,0.465728,Trench 1,2024-09-16 14:12:59,2024-09-17,2024-09,Train,0,1,Quick


In [316]:
df2 = dfd.copy()

In [317]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             13892 non-null  Int64         
 1   digitalLoanAccountId   13892 non-null  object        
 2   loanAccountNumber      13892 non-null  object        
 3   aCicScore              13892 non-null  float64       
 4   trenchCategory         13892 non-null  object        
 5   appln_submit_datetime  13892 non-null  datetime64[us]
 6   disbursementdate       13892 non-null  dbdate        
 7   Application_month      13892 non-null  object        
 8   Data_selection         13892 non-null  object        
 9   deffstpd30             13892 non-null  Int64         
 10  flg_mature_fstpd_30    13892 non-null  Int64         
 11  new_loan_type          13892 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [318]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [319]:
# df_concat.to_csv(r"aCicScorefstpd30.csv")

In [320]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [321]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aCicScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.288770,Week,aCicScore,1.1.0,FSTPD30,Overall
1,2024-09-01,2024-09-30,0.254391,Month,aCicScore,1.1.0,FSTPD30,Overall
2,2024-09-02,2024-09-08,0.311464,Week,aCicScore,1.1.0,FSTPD30,Overall
3,2024-09-09,2024-09-15,0.243421,Week,aCicScore,1.1.0,FSTPD30,Overall
4,2024-09-16,2024-09-22,0.285890,Week,aCicScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [322]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aCicScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aCicScore_FPD10_gini', 'aCicScore_FPD30_gini',
       'aCicScore_FSPD30_gini', 'aCicScore_FSTPD30_gini'], dtype=object)

In [323]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','aCicScore_FPD0_gini','aCicScore_FPD10_gini',
       'aCicScore_FPD30_gini', 'aCicScore_FSPD30_gini',
       'aCicScore_FSTPD30_gini']].copy()
final_df['Model_display_name'] = 'cic_model_cash'
final_df['Trench_category'] = 'Trench 1'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                datetime64[ns]
end_date                  datetime64[ns]
period                            object
Model_Name                        object
version                           object
loan_type                         object
bad_rate                          object
aCicScore_FPD0_gini              float64
aCicScore_FPD10_gini             float64
aCicScore_FPD30_gini             float64
aCicScore_FSPD30_gini            float64
aCicScore_FSTPD30_gini           float64
Model_display_name                object
Trench_category                   object
Product_type                      object
dtype: object

In [324]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aCicScore_FPD0_gini,aCicScore_FPD10_gini,aCicScore_FPD30_gini,aCicScore_FSPD30_gini,aCicScore_FSTPD30_gini,Model_display_name,Trench_category,Product_type
0,2024-08-26,2024-09-01,Week,aCicScore,1.1.0,Overall,FPD0,-0.170940,NaN,NaN,NaN,NaN,cic_model_cash,Trench 1,CASH
1,2024-09-01,2024-09-30,Month,aCicScore,1.1.0,Overall,FPD0,0.217332,NaN,NaN,NaN,NaN,cic_model_cash,Trench 1,CASH
2,2024-09-02,2024-09-08,Week,aCicScore,1.1.0,Overall,FPD0,0.187550,NaN,NaN,NaN,NaN,cic_model_cash,Trench 1,CASH
3,2024-09-09,2024-09-15,Week,aCicScore,1.1.0,Overall,FPD0,0.286166,NaN,NaN,NaN,NaN,cic_model_cash,Trench 1,CASH
4,2024-09-16,2024-09-22,Week,aCicScore,1.1.0,Overall,FPD0,0.271593,NaN,NaN,NaN,NaN,cic_model_cash,Trench 1,CASH


In [325]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7a26e03b-b2f5-413e-a795-8eac306d59b7>

## Trench 2

## FPD0

## Test

In [326]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1945496,a069a1f0-c467-4c7e-945f-609d46f5df01,60819454960027,0.464424079095343,Trench 2,2025-10-12 08:09:23,2025-10-12,2025-10,Test,0,1,Quick
1,3508511,ce128857-2a6c-4c41-9c1c-b9ee1c15ace9,60835085110013,0.6333372996201331,Trench 2,2025-10-12 08:37:51,2025-10-12,2025-10,Test,0,1,Quick
2,2245137,ea96ebc8-11fd-4213-bc3c-bf5ed20153e0,60822451370014,0.44314478332315793,Trench 2,2025-10-12 11:47:52,2025-10-12,2025-10,Test,0,1,Quick
3,3520612,ded06602-1a34-4ac0-bedb-145da1a50ee4,60835206120015,0.4668304394761966,Trench 2,2025-10-12 11:21:55,2025-10-12,2025-10,Test,0,1,Quick
4,3552643,c04c60df-c968-432d-9f9e-3f6349d45727,60835526430023,0.3956867386380188,Trench 2,2025-10-12 12:19:09,2025-10-12,2025-10,Test,0,1,Quick


In [327]:
df1 = dfd.copy()

## Train

In [328]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2546031,d7f1a565-b482-4c92-a8ef-027a90a96fd0,60825460310017,0.456065,Trench 2,2024-09-03 12:04:42,2024-09-03,2024-09,Train,1,1,Quick
1,2409718,a42b4d99-0772-4549-9f04-0691703cdaae,60824097180017,0.464905,Trench 2,2024-09-18 15:31:44,2024-09-18,2024-09,Train,1,1,Quick
2,2218790,b0dc60c7-dd11-4fa4-8d00-f9576bc87127,60822187900014,0.420073,Trench 2,2024-09-24 19:24:35,2024-09-24,2024-09,Train,1,1,Quick
3,1282912,8d230527-f7ce-4910-a3e3-56cc4c6b5ec0,60812829120011,0.419544,Trench 2,2024-09-14 00:30:23,2024-09-14,2024-09,Train,0,1,Quick
4,2640880,f21e3fa1-40d2-48cb-8f2e-68eb55e2d985,60826408800012,0.378444,Trench 2,2024-09-30 14:57:36,2024-09-30,2024-09,Train,0,1,Quick


In [329]:
df2 = dfd.copy()

In [330]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12790 entries, 0 to 12789
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12790 non-null  object        
 1   digitalLoanAccountId   12790 non-null  object        
 2   loanAccountNumber      12790 non-null  object        
 3   aCicScore              12790 non-null  object        
 4   trenchCategory         12790 non-null  object        
 5   appln_submit_datetime  12790 non-null  datetime64[us]
 6   disbursementdate       12790 non-null  dbdate        
 7   Application_month      12790 non-null  object        
 8   Data_selection         12790 non-null  object        
 9   deffpd0                12790 non-null  Int64         
 10  flg_mature_fpd0        12790 non-null  Int64         
 11  new_loan_type          12790 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [331]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [332]:
# df_concat.to_csv(r"aCicScoretrench2fpd0.csv")

In [333]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [334]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aCicScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.495652,Week,aCicScore,1.1.0,FPD0,Overall
1,2024-09-01,2024-09-30,0.290750,Month,aCicScore,1.1.0,FPD0,Overall
2,2024-09-02,2024-09-08,0.285824,Week,aCicScore,1.1.0,FPD0,Overall
3,2024-09-09,2024-09-15,0.301728,Week,aCicScore,1.1.0,FPD0,Overall
4,2024-09-16,2024-09-22,0.224814,Week,aCicScore,1.1.0,FPD0,Overall


## FPD10

## Test

In [335]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1223831,a5fcdd74-1730-47dc-8548-b259d7454717,60812238310105,0.43211544875013963,Trench 2,2025-10-08 10:06:10,2025-10-08,2025-10,Test,0,1,Quick
1,2370906,18609e3a-3593-400c-a7d5-62b2917a08b1,60823709060062,0.4320550107106068,Trench 2,2025-10-08 10:55:19,2025-10-08,2025-10,Test,0,1,Quick
2,2940729,d40291db-f66b-47d9-a1e4-e47741b97e86,60829407290014,0.5975933517764157,Trench 2,2025-10-08 08:07:28,2025-10-08,2025-10,Test,1,1,Quick
3,3304854,253cf8b1-5ac4-43f4-9636-5791b8a5189c,60833048540015,0.376410602412713,Trench 2,2025-10-08 09:26:05,2025-10-08,2025-10,Test,0,1,Quick
4,3339838,69aee249-629e-4705-8660-8f497eb56da9,60833398380011,0.3653901187839602,Trench 2,2025-10-08 09:38:33,2025-10-08,2025-10,Test,0,1,Quick


In [336]:
df1 = dfd.copy()

## Train

In [337]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')\
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2430269,4a88275a-e444-43be-bfc1-f24b501e72f7,60824302690012,0.718481,Trench 2,2024-09-01 00:54:36,2024-09-02,2024-09,Train,1,1,Quick
1,2311384,0ac25837-ef6a-4470-b351-31812f255dcc,60823113840012,0.476179,Trench 2,2024-09-29 12:51:58,2024-09-29,2024-09,Train,0,1,Quick
2,2200293,e1146aba-010b-4ec2-992d-fcaf7f3a8142,60822002930015,0.503469,Trench 2,2024-09-21 11:23:04,2024-09-21,2024-09,Train,0,1,Quick
3,1477120,cb442e76-617d-435f-995c-604e1dbe8c81,60814771200021,0.470829,Trench 2,2024-09-02 06:36:01,2024-09-02,2024-09,Train,0,1,Quick
4,2489643,320f47de-7bcc-4bc6-85aa-5c2914e2de01,60824896430015,0.468150,Trench 2,2024-09-21 10:09:06,2024-09-21,2024-09,Train,1,1,Quick


In [338]:
df2 = dfd.copy()

In [339]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12600 non-null  object        
 1   digitalLoanAccountId   12600 non-null  object        
 2   loanAccountNumber      12600 non-null  object        
 3   aCicScore              12600 non-null  object        
 4   trenchCategory         12600 non-null  object        
 5   appln_submit_datetime  12600 non-null  datetime64[us]
 6   disbursementdate       12600 non-null  dbdate        
 7   Application_month      12600 non-null  object        
 8   Data_selection         12600 non-null  object        
 9   deffpd10               12600 non-null  Int64         
 10  flg_mature_fpd10       12600 non-null  Int64         
 11  new_loan_type          12600 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [340]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [341]:
# df_concat.to_csv(r"aCicScoretrench2fpd10.csv")

In [342]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [343]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aCicScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.495652,Week,aCicScore,1.1.0,FPD10,Overall
1,2024-09-01,2024-09-30,0.291819,Month,aCicScore,1.1.0,FPD10,Overall
2,2024-09-02,2024-09-08,0.298729,Week,aCicScore,1.1.0,FPD10,Overall
3,2024-09-09,2024-09-15,0.277186,Week,aCicScore,1.1.0,FPD10,Overall
4,2024-09-16,2024-09-22,0.284906,Week,aCicScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [344]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [345]:
df1 = dfd.copy()

## Train

In [346]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2755527,f156ee81-b1f8-4fd0-a77b-f972180e7d0a,60827555270019,0.446896,Trench 2,2024-09-26 08:10:01,2024-09-27,2024-09,Train,0,1,Quick
1,2255250,0479edc5-2696-4373-b04e-055a28024e0f,60822552500019,0.440517,Trench 2,2024-09-28 13:02:26,2024-09-28,2024-09,Train,1,1,Quick
2,2527352,30684c76-937e-41da-9172-b5c880c4e293,60825273520015,0.468855,Trench 2,2024-09-07 18:12:51,2024-09-08,2024-09,Train,1,1,Quick
3,2303572,77693497-36ed-44f3-8e9e-03a7533dd64e,60823035720017,0.505655,Trench 2,2024-09-21 14:31:40,2024-09-21,2024-09,Train,1,1,Quick
4,2561847,4eea0397-7fdf-461e-930e-0924986e7b0a,60825618470019,0.754098,Trench 2,2024-09-20 23:25:40,2024-09-21,2024-09,Train,1,1,Quick


In [347]:
df2 = dfd.copy()

In [348]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12245 entries, 0 to 12244
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12245 non-null  Int64         
 1   digitalLoanAccountId   12245 non-null  object        
 2   loanAccountNumber      12245 non-null  object        
 3   aCicScore              12245 non-null  float64       
 4   trenchCategory         12245 non-null  object        
 5   appln_submit_datetime  12245 non-null  datetime64[us]
 6   disbursementdate       12245 non-null  dbdate        
 7   Application_month      12245 non-null  object        
 8   Data_selection         12245 non-null  object        
 9   deffpd30               12245 non-null  Int64         
 10  flg_mature_fpd30       12245 non-null  Int64         
 11  new_loan_type          12245 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [349]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [350]:
# df_concat.to_csv(r"aCicScoretrench2fpd30.csv")

In [351]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [352]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aCicScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.495652,Week,aCicScore,1.1.0,FPD30,Overall
1,2024-09-01,2024-09-30,0.318568,Month,aCicScore,1.1.0,FPD30,Overall
2,2024-09-02,2024-09-08,0.291139,Week,aCicScore,1.1.0,FPD30,Overall
3,2024-09-09,2024-09-15,0.285386,Week,aCicScore,1.1.0,FPD30,Overall
4,2024-09-16,2024-09-22,0.317084,Week,aCicScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [353]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fpd30,new_loan_type


In [354]:
df1 = dfd.copy()

## Train

In [355]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2524455,96c72d7e-9954-472b-87ad-4212658842ef,60825244550019,0.382210,Trench 2,2024-09-02 03:47:14,2024-09-03,2024-09,Train,1,1,Quick
1,2463235,00c9fdfb-3806-47f8-95d7-b30337d17921,60824632350019,0.353224,Trench 2,2024-09-11 15:22:57,2024-09-11,2024-09,Train,1,1,Quick
2,1380291,1ef2b0ca-ce57-40c6-affd-ed88c37fadae,60813802910071,0.443603,Trench 2,2024-09-24 11:18:55,2024-09-24,2024-09,Train,0,1,Quick
3,2729762,0724524a-5a77-4e93-afc2-fc6402947f0a,60827297620019,0.575914,Trench 2,2024-09-11 13:51:06,2024-09-22,2024-09,Train,1,1,Quick
4,1963254,0c6d8927-caa1-4d1d-a282-ae13c79f78d4,60819632540015,0.468998,Trench 2,2024-09-05 16:39:36,2024-09-05,2024-09,Train,0,1,Quick


In [356]:
df2 = dfd.copy()

In [357]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11567 entries, 0 to 11566
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11567 non-null  Int64         
 1   digitalLoanAccountId   11567 non-null  object        
 2   loanAccountNumber      11567 non-null  object        
 3   aCicScore              11567 non-null  float64       
 4   trenchCategory         11567 non-null  object        
 5   appln_submit_datetime  11567 non-null  datetime64[us]
 6   disbursementdate       11567 non-null  dbdate        
 7   Application_month      11567 non-null  object        
 8   Data_selection         11567 non-null  object        
 9   deffspd30              11567 non-null  Int64         
 10  flg_mature_fspd_30     11567 non-null  Int64         
 11  new_loan_type          11567 non-null  object        
 12  flg_mature_fpd30       0 non-null      object        
dtypes

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [358]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [359]:
# df_concat.to_csv(r"aCicScoretrench2fspd30.csv")

In [360]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [361]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aCicScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.636364,Week,aCicScore,1.1.0,FSPD30,Overall
1,2024-09-01,2024-09-30,0.330574,Month,aCicScore,1.1.0,FSPD30,Overall
2,2024-09-02,2024-09-08,0.342180,Week,aCicScore,1.1.0,FSPD30,Overall
3,2024-09-09,2024-09-15,0.325465,Week,aCicScore,1.1.0,FSPD30,Overall
4,2024-09-16,2024-09-22,0.320003,Week,aCicScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [362]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [363]:
df1 = dfd.copy()

## Train

In [364]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2419490,11e00d5a-570d-4bc0-833a-9903762a19b5,60824194900011,0.423030,Trench 2,2024-09-13 09:30:16,2024-09-13,2024-09,Train,0,1,Quick
1,1562414,85fe52aa-0b67-480f-93a6-2c3720d645f9,60815624140015,0.619431,Trench 2,2024-09-23 01:23:09,2024-09-30,2024-09,Train,1,1,Quick
2,2073189,90dacded-6757-48a3-9d9b-58f85efdaa1e,60820731890011,0.636984,Trench 2,2024-09-11 11:00:36,2024-09-11,2024-09,Train,1,1,Quick
3,2585243,336cd386-57b8-40f9-8524-67b975eac293,60825852430013,0.716632,Trench 2,2024-09-22 18:11:32,2024-09-22,2024-09,Train,1,1,Quick
4,2588412,8780c78f-f43f-4490-b5ce-74078bac53df,60825884120014,0.522872,Trench 2,2024-09-21 01:21:45,2024-09-22,2024-09,Train,0,1,Quick


In [365]:
df2 = dfd.copy()

In [366]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10832 entries, 0 to 10831
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10832 non-null  Int64         
 1   digitalLoanAccountId   10832 non-null  object        
 2   loanAccountNumber      10832 non-null  object        
 3   aCicScore              10832 non-null  float64       
 4   trenchCategory         10832 non-null  object        
 5   appln_submit_datetime  10832 non-null  datetime64[us]
 6   disbursementdate       10832 non-null  dbdate        
 7   Application_month      10832 non-null  object        
 8   Data_selection         10832 non-null  object        
 9   deffstpd30             10832 non-null  Int64         
 10  flg_mature_fstpd_30    10832 non-null  Int64         
 11  new_loan_type          10832 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [367]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [368]:
# df_concat.to_csv(r"aCicScoretrench2fstpd30.csv")

In [369]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [370]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aCicScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.426901,Week,aCicScore,1.1.0,FSTPD30,Overall
1,2024-09-01,2024-09-30,0.291161,Month,aCicScore,1.1.0,FSTPD30,Overall
2,2024-09-02,2024-09-08,0.308345,Week,aCicScore,1.1.0,FSTPD30,Overall
3,2024-09-09,2024-09-15,0.262134,Week,aCicScore,1.1.0,FSTPD30,Overall
4,2024-09-16,2024-09-22,0.303910,Week,aCicScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [371]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aCicScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aCicScore_FPD10_gini', 'aCicScore_FPD30_gini',
       'aCicScore_FSPD30_gini', 'aCicScore_FSTPD30_gini'], dtype=object)

In [372]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','aCicScore_FPD0_gini','aCicScore_FPD10_gini',
       'aCicScore_FPD30_gini', 'aCicScore_FSPD30_gini',
       'aCicScore_FSTPD30_gini']].copy()
final_df.rename(columns={'aCicScore_FPD0_gini':'aCicScore_t2_FPD0_gini', 'aCicScore_FPD10_gini':'aCicScore_t2_FPD10_gini', 'aCicScore_FPD30_gini':'aCicScore_t2_FPD30_gini', 'aCicScore_FSPD30_gini':'aCicScore_t2_FSPD30_gini'
                        , 'aCicScore_FSPD30_gini':'aCicScore_t2_FSPD30_gini', 'aCicScore_FSTPD30_gini':'aCicScore_t2_FSTPD30_gini'
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 2'
final_df['Model_display_name'] = 'cic_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                   datetime64[ns]
end_date                     datetime64[ns]
period                               object
Model_Name                           object
version                              object
loan_type                            object
bad_rate                             object
aCicScore_t2_FPD0_gini              float64
aCicScore_t2_FPD10_gini             float64
aCicScore_t2_FPD30_gini             float64
aCicScore_t2_FSPD30_gini            float64
aCicScore_t2_FSTPD30_gini           float64
Trench_category                      object
Model_display_name                   object
Product_type                         object
dtype: object

In [373]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aCicScore_t2_FPD0_gini,aCicScore_t2_FPD10_gini,aCicScore_t2_FPD30_gini,aCicScore_t2_FSPD30_gini,aCicScore_t2_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2024-08-26,2024-09-01,Week,aCicScore,1.1.0,Overall,FPD0,0.495652,NaN,NaN,NaN,NaN,Trench 2,cic_model_cash,CASH
1,2024-09-01,2024-09-30,Month,aCicScore,1.1.0,Overall,FPD0,0.290750,NaN,NaN,NaN,NaN,Trench 2,cic_model_cash,CASH
2,2024-09-02,2024-09-08,Week,aCicScore,1.1.0,Overall,FPD0,0.285824,NaN,NaN,NaN,NaN,Trench 2,cic_model_cash,CASH
3,2024-09-09,2024-09-15,Week,aCicScore,1.1.0,Overall,FPD0,0.301728,NaN,NaN,NaN,NaN,Trench 2,cic_model_cash,CASH
4,2024-09-16,2024-09-22,Week,aCicScore,1.1.0,Overall,FPD0,0.224814,NaN,NaN,NaN,NaN,Trench 2,cic_model_cash,CASH


In [374]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_t2_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=fd70ddaa-d1d1-4bd1-ad9c-afd4ae407f14>

## Trench 3

## FPD0

## Test

In [375]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1854457,6b538e30-1be2-40e2-b371-549757d18202,60818544570041,0.49680031457194324,Trench 3,2025-10-22 09:30:41,2025-10-23,2025-10,Test,0,1,Quick
1,3669928,aa6a90bc-1e7b-4e67-966d-46a0f647bd2a,60836699280022,0.4119239543949854,Trench 3,2025-10-22 10:17:16,2025-10-23,2025-10,Test,0,1,Quick
2,3377316,b396dd00-d1b5-4ff3-ac79-d87d8a3ce8ea,60833773160029,0.41893599548559146,Trench 3,2025-10-22 11:52:05,2025-10-22,2025-10,Test,0,1,Quick
3,3390733,da7ee5b6-edf6-44d3-b94c-9694521894cc,60833907330024,0.658135015454353,Trench 3,2025-10-22 11:57:10,2025-10-22,2025-10,Test,0,1,Quick
4,2999661,6817fc81-b474-4083-aea1-ff966a076586,60829996610038,0.3128773259015523,Trench 3,2025-10-22 10:37:26,2025-10-24,2025-10,Test,0,1,Quick


In [376]:
df1 = dfd.copy()

## Train

In [377]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2756002,2b165ad7-5d86-4ec6-b95b-f67ae2c82aa8,60827560020027,0.447945,Trench 3,2024-09-24 14:02:31,2024-10-22,2024-09,Train,0,1,Quick
1,1676290,9d35f338-3807-4e4d-8fd8-6223e93abbbe,60816762900051,0.493883,Trench 3,2024-09-03 10:04:48,2024-09-03,2024-09,Train,1,1,Quick
2,1772915,b46887cf-fe0a-4d69-a00b-dac93daa8bda,60817729150034,0.392878,Trench 3,2024-09-23 13:05:32,2024-09-23,2024-09,Train,0,1,Quick
3,1992646,58b7e8b6-d4ff-4772-8f87-bfed089144b3,60819926460034,0.368966,Trench 3,2024-09-15 08:25:16,2024-09-15,2024-09,Train,0,1,Quick
4,2785688,5052efef-6cef-4c96-8750-2f8f2315d2f0,60827856880028,0.433623,Trench 3,2024-09-03 04:34:37,2024-09-03,2024-09,Train,0,1,Quick


In [378]:
df2 = dfd.copy()

In [379]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11586 entries, 0 to 11585
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11586 non-null  object        
 1   digitalLoanAccountId   11586 non-null  object        
 2   loanAccountNumber      11586 non-null  object        
 3   aCicScore              11586 non-null  object        
 4   trenchCategory         11586 non-null  object        
 5   appln_submit_datetime  11586 non-null  datetime64[us]
 6   disbursementdate       11586 non-null  dbdate        
 7   Application_month      11586 non-null  object        
 8   Data_selection         11586 non-null  object        
 9   deffpd0                11586 non-null  Int64         
 10  flg_mature_fpd0        11586 non-null  Int64         
 11  new_loan_type          11586 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [380]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [381]:
# df_concat.to_csv(r"aCicScoretrench3fpd0.csv")

In [382]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [383]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aCicScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.214286,Week,aCicScore,1.1.0,FPD0,Overall
1,2024-09-01,2024-09-30,0.168616,Month,aCicScore,1.1.0,FPD0,Overall
2,2024-09-02,2024-09-08,0.256825,Week,aCicScore,1.1.0,FPD0,Overall
3,2024-09-09,2024-09-15,0.140000,Week,aCicScore,1.1.0,FPD0,Overall
4,2024-09-16,2024-09-22,0.108178,Week,aCicScore,1.1.0,FPD0,Overall


## FPD10

## Test

In [384]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1333757,e8ec43bf-1c27-4e64-85dd-7a7e5792f0ab,60813337570046,0.3983057328524946,Trench 3,2025-10-17 10:39:27,2025-10-17,2025-10,Test,0,1,Quick
1,1419506,5de90a47-4100-48aa-8984-310792a44bd7,60814195060031,0.4316538950151025,Trench 3,2025-10-17 16:17:22,2025-10-17,2025-10,Test,0,1,Quick
2,2079185,717b3a8e-9ba1-49ee-bccb-ef606e0fabb5,60820791850045,0.41059708306422843,Trench 3,2025-10-18 07:52:21,2025-10-18,2025-10,Test,0,1,Quick
3,2901087,9a5f5a79-3600-444a-ae69-cc38b0fa0ca3,60829010870025,0.42682465979079715,Trench 3,2025-10-17 08:40:50,2025-10-17,2025-10,Test,0,1,Quick
4,3246519,44944778-eac7-4df4-97c9-8ed2e6175ed1,60832465190049,0.39645910759989345,Trench 3,2025-10-17 10:18:12,2025-10-17,2025-10,Test,0,1,Quick


In [385]:
df1 = dfd.copy()

## Train

In [386]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1973030,aa17042f-b93c-4beb-a440-2018a566aaea,60819730300024,0.443920,Trench 3,2024-09-28 00:35:55,2024-09-28,2024-09,Train,0,1,Quick
1,2385760,88773f1f-33fd-49cb-a674-ab8ef0770d7c,60823857600025,0.447614,Trench 3,2024-09-09 13:57:25,2024-09-10,2024-09,Train,0,1,Quick
2,1956571,ca30862c-1c52-46d1-8c62-bb041f0b4199,60819565710038,0.485643,Trench 3,2024-09-27 16:10:41,2024-09-27,2024-09,Train,0,1,Quick
3,2705309,d6ba6ff9-11fa-404d-a87b-919608c0facd,60827053090032,0.443874,Trench 3,2024-09-16 15:38:10,2024-09-16,2024-09,Train,0,1,Quick
4,1541414,33842c2f-9580-405e-b1f1-027b151e6c41,60815414140045,0.378005,Trench 3,2024-09-30 13:41:26,2024-09-30,2024-09,Train,0,1,Quick


In [387]:
df2 = dfd.copy()

In [388]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11430 non-null  object        
 1   digitalLoanAccountId   11430 non-null  object        
 2   loanAccountNumber      11430 non-null  object        
 3   aCicScore              11430 non-null  object        
 4   trenchCategory         11430 non-null  object        
 5   appln_submit_datetime  11430 non-null  datetime64[us]
 6   disbursementdate       11430 non-null  dbdate        
 7   Application_month      11430 non-null  object        
 8   Data_selection         11430 non-null  object        
 9   deffpd10               11430 non-null  Int64         
 10  flg_mature_fpd10       11430 non-null  Int64         
 11  new_loan_type          11430 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [389]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [390]:
# df_concat.to_csv(r"aCicScoretrench3fpd10.csv")

In [391]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [392]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aCicScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,0.055556,Week,aCicScore,1.1.0,FPD10,Overall
1,2024-09-01,2024-09-30,0.191107,Month,aCicScore,1.1.0,FPD10,Overall
2,2024-09-02,2024-09-08,0.098118,Week,aCicScore,1.1.0,FPD10,Overall
3,2024-09-09,2024-09-15,0.497942,Week,aCicScore,1.1.0,FPD10,Overall
4,2024-09-16,2024-09-22,0.091880,Week,aCicScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [393]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [394]:
df1 = dfd.copy()

## Train

In [395]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1096918,b7856f03-f20c-4fcb-a487-b3851ec34f32,60810969180031,0.372927,Trench 3,2024-09-09 02:21:44,2024-09-30,2024-09,Train,0,1,Quick
1,1865751,8e4ca842-1e35-4de5-8622-792dbda82728,60818657510035,0.410226,Trench 3,2024-09-05 12:23:42,2024-09-05,2024-09,Train,0,1,Quick
2,1980223,08e97080-984e-43b0-82da-7629c75cc9cc,60819802230032,0.459555,Trench 3,2024-09-20 12:47:48,2024-09-20,2024-09,Train,0,1,Quick
3,2186677,a646ee54-96c1-45d1-a32e-def39ee59bef,60821866770023,0.444575,Trench 3,2024-09-29 21:12:19,2024-09-29,2024-09,Train,0,1,Quick
4,2266581,c95d0ef4-efa4-48e8-b953-e6d411992682,60822665810031,0.409603,Trench 3,2024-09-14 06:06:34,2024-09-14,2024-09,Train,0,1,Quick


In [396]:
df2 = dfd.copy()

In [397]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10946 entries, 0 to 10945
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10946 non-null  Int64         
 1   digitalLoanAccountId   10946 non-null  object        
 2   loanAccountNumber      10946 non-null  object        
 3   aCicScore              10946 non-null  float64       
 4   trenchCategory         10946 non-null  object        
 5   appln_submit_datetime  10946 non-null  datetime64[us]
 6   disbursementdate       10946 non-null  dbdate        
 7   Application_month      10946 non-null  object        
 8   Data_selection         10946 non-null  object        
 9   deffpd30               10946 non-null  Int64         
 10  flg_mature_fpd30       10946 non-null  Int64         
 11  new_loan_type          10946 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [398]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [399]:
# df_concat.to_csv(r"aCicScoretrench3fpd30.csv")

In [400]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [401]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aCicScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,-0.230769,Week,aCicScore,1.1.0,FPD30,Overall
1,2024-09-01,2024-09-30,0.210121,Month,aCicScore,1.1.0,FPD30,Overall
2,2024-09-02,2024-09-08,0.168794,Week,aCicScore,1.1.0,FPD30,Overall
3,2024-09-09,2024-09-15,0.332143,Week,aCicScore,1.1.0,FPD30,Overall
4,2024-09-16,2024-09-22,0.455556,Week,aCicScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [402]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fpd30,new_loan_type


In [403]:
df1 = dfd.copy()

## Train

In [404]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2234990,9905b4db-5c01-4058-b9a8-f92c2284323e,60822349900048,0.431447,Trench 3,2024-09-15 09:53:13,2024-09-15,2024-09,Train,0,1,Quick
1,2281107,2d3a69ce-6d21-43bf-b190-a70f1e2feb08,60822811070021,0.419928,Trench 3,2024-09-07 07:42:41,2024-09-07,2024-09,Train,0,1,Quick
2,2768998,e5fb2a99-e2ac-4774-9475-8f91d88263f8,60827689980021,0.407185,Trench 3,2024-09-19 12:01:56,2024-09-19,2024-09,Train,0,1,Quick
3,2796037,936207e7-6147-480f-9813-eb99ba770e47,60827960370028,0.357994,Trench 3,2024-09-22 09:10:58,2024-09-22,2024-09,Train,0,1,Quick
4,2253646,6099c2b6-a901-47fb-941d-c88c91512489,60822536460025,0.502753,Trench 3,2024-09-03 01:37:07,2024-09-03,2024-09,Train,0,1,Quick


In [405]:
df2 = dfd.copy()

In [406]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9755 entries, 0 to 9754
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9755 non-null   Int64         
 1   digitalLoanAccountId   9755 non-null   object        
 2   loanAccountNumber      9755 non-null   object        
 3   aCicScore              9755 non-null   float64       
 4   trenchCategory         9755 non-null   object        
 5   appln_submit_datetime  9755 non-null   datetime64[us]
 6   disbursementdate       9755 non-null   dbdate        
 7   Application_month      9755 non-null   object        
 8   Data_selection         9755 non-null   object        
 9   deffspd30              9755 non-null   Int64         
 10  flg_mature_fspd_30     9755 non-null   Int64         
 11  new_loan_type          9755 non-null   object        
 12  flg_mature_fpd30       0 non-null      object        
dtypes: 

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [407]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [408]:
# df_concat.to_csv(r"aCicScoretrench2fspd30.csv")

In [409]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [410]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aCicScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,-0.230769,Week,aCicScore,1.1.0,FSPD30,Overall
1,2024-09-01,2024-09-30,0.276687,Month,aCicScore,1.1.0,FSPD30,Overall
2,2024-09-02,2024-09-08,0.234417,Week,aCicScore,1.1.0,FSPD30,Overall
3,2024-09-09,2024-09-15,0.499065,Week,aCicScore,1.1.0,FSPD30,Overall
4,2024-09-16,2024-09-22,0.316923,Week,aCicScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [411]:
sq = r"""
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
--REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aCicScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [412]:
df1 = dfd.copy()

## Train

In [413]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aCicScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aCicScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aCicScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aCicScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2156231,98931b72-923b-4467-ac5b-24d9b754ea03,60821562310022,0.616977,Trench 3,2024-09-09 11:17:13,2024-09-09,2024-09,Train,0,1,Quick
1,1131179,f4c050d2-616c-4ca7-8f4a-abd16daf2894,60811311790044,0.452935,Trench 3,2024-09-27 08:17:02,2024-09-27,2024-09,Train,0,1,Quick
2,2275836,ad6b5415-d289-466c-b3aa-f4053145cd05,60822758360027,0.391020,Trench 3,2024-09-06 17:11:21,2024-09-06,2024-09,Train,0,1,Quick
3,2061528,b86bb9f7-8317-41ac-af12-e16cb9e872e6,60820615280025,0.347930,Trench 3,2024-09-27 09:01:58,2024-09-27,2024-09,Train,0,1,Quick
4,1683832,ae266131-585e-4732-848b-93ae0828d841,60816838320035,0.452418,Trench 3,2024-09-04 11:00:48,2024-09-04,2024-09,Train,1,1,Quick


In [414]:
df2 = dfd.copy()

In [415]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8521 entries, 0 to 8520
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8521 non-null   Int64         
 1   digitalLoanAccountId   8521 non-null   object        
 2   loanAccountNumber      8521 non-null   object        
 3   aCicScore              8521 non-null   float64       
 4   trenchCategory         8521 non-null   object        
 5   appln_submit_datetime  8521 non-null   datetime64[us]
 6   disbursementdate       8521 non-null   dbdate        
 7   Application_month      8521 non-null   object        
 8   Data_selection         8521 non-null   object        
 9   deffstpd30             8521 non-null   Int64         
 10  flg_mature_fstpd_30    8521 non-null   Int64         
 11  new_loan_type          8521 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [416]:
df_concat['aCicScore'] = pd.to_numeric(df_concat['aCicScore'], errors='coerce')

In [417]:
# df_concat.to_csv(r"aCicScoretrench3fstpd30.csv")

In [418]:
# gini_results = calculate_periodic_gini(df_concat, 'aCicScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aCicScore', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [419]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aCicScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-08-26,2024-09-01,-0.272727,Week,aCicScore,1.1.0,FSTPD30,Overall
1,2024-09-01,2024-09-30,0.283827,Month,aCicScore,1.1.0,FSTPD30,Overall
2,2024-09-02,2024-09-08,0.234417,Week,aCicScore,1.1.0,FSTPD30,Overall
3,2024-09-09,2024-09-15,0.495425,Week,aCicScore,1.1.0,FSTPD30,Overall
4,2024-09-16,2024-09-22,0.307951,Week,aCicScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [420]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aCicScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aCicScore_FPD10_gini', 'aCicScore_FPD30_gini',
       'aCicScore_FSPD30_gini', 'aCicScore_FSTPD30_gini'], dtype=object)

In [421]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','aCicScore_FPD0_gini','aCicScore_FPD10_gini',
       'aCicScore_FPD30_gini', 'aCicScore_FSPD30_gini',
       'aCicScore_FSTPD30_gini']].copy()
final_df.rename(columns={'aCicScore_FPD0_gini':'aCicScore_t3_FPD0_gini', 'aCicScore_FPD10_gini':'aCicScore_t3_FPD10_gini', 'aCicScore_FPD30_gini':'aCicScore_t3_FPD30_gini', 'aCicScore_FSPD30_gini':'aCicScore_t3_FSPD30_gini'
                        , 'aCicScore_FSPD30_gini':'aCicScore_t3_FSPD30_gini', 'aCicScore_FSTPD30_gini':'aCicScore_t3_FSTPD30_gini'
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 3'
final_df['Model_display_name'] = 'cic_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                   datetime64[ns]
end_date                     datetime64[ns]
period                               object
Model_Name                           object
version                              object
loan_type                            object
bad_rate                             object
aCicScore_t3_FPD0_gini              float64
aCicScore_t3_FPD10_gini             float64
aCicScore_t3_FPD30_gini             float64
aCicScore_t3_FSPD30_gini            float64
aCicScore_t3_FSTPD30_gini           float64
Trench_category                      object
Model_display_name                   object
Product_type                         object
dtype: object

In [422]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aCicScore_t3_FPD0_gini,aCicScore_t3_FPD10_gini,aCicScore_t3_FPD30_gini,aCicScore_t3_FSPD30_gini,aCicScore_t3_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2024-08-26,2024-09-01,Week,aCicScore,1.1.0,Overall,FPD0,0.214286,NaN,NaN,NaN,NaN,Trench 3,cic_model_cash,CASH
1,2024-09-01,2024-09-30,Month,aCicScore,1.1.0,Overall,FPD0,0.168616,NaN,NaN,NaN,NaN,Trench 3,cic_model_cash,CASH
2,2024-09-02,2024-09-08,Week,aCicScore,1.1.0,Overall,FPD0,0.256825,NaN,NaN,NaN,NaN,Trench 3,cic_model_cash,CASH
3,2024-09-09,2024-09-15,Week,aCicScore,1.1.0,Overall,FPD0,0.140000,NaN,NaN,NaN,NaN,Trench 3,cic_model_cash,CASH
4,2024-09-16,2024-09-22,Week,aCicScore,1.1.0,Overall,FPD0,0.108178,NaN,NaN,NaN,NaN,Trench 3,cic_model_cash,CASH


In [423]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_t3_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=407a10cb-101c-47c4-b819-168dc669cea2>

# Alpha-Cash-Stack-Model

# Trench 1

## FPD0

## Test

In [424]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3736314,0bafdbc6-b864-43b6-97f9-ff6dc3e03235,60837363140019,0.4248711219827766,Trench 1,2025-10-11 08:08:41,2025-10-11,2025-10,Test,0,1,Quick
1,3736322,cede4e7b-ebbb-47e8-bb28-19198b0c2909,60837363220019,0.45220222545720534,Trench 1,2025-10-11 08:13:07,2025-10-11,2025-10,Test,0,1,Quick
2,3736374,4d57bea4-9aa3-4388-b81c-b92af0d0b484,60837363740011,0.30390012004651235,Trench 1,2025-10-11 09:18:55,2025-10-11,2025-10,Test,0,1,Quick
3,3736439,e7caf63c-ca94-4e46-abe0-d31b731c603b,60837364390011,0.2829745737805767,Trench 1,2025-10-11 09:58:54,2025-10-11,2025-10,Test,1,1,Quick
4,3736028,1a898f1d-bc2d-45f3-9316-017416e73bd6,60837360280016,0.38529446722167304,Trench 1,2025-10-11 11:15:20,2025-10-11,2025-10,Test,0,1,Quick


In [425]:
df1 = dfd.copy()

## Train

In [426]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2930426,d70cda1e-dcff-4153-aa23-4202795b1106,60829304260017,0.538985,Trench 1,2024-10-15 10:01:17,2024-10-15,2024-10,Train,0,1,Quick
1,2986097,d537e5b9-0df8-48fa-b9fb-531e6d9aa42e,60829860970019,0.554756,Trench 1,2024-10-30 22:39:35,2024-10-31,2024-10,Train,0,1,Quick
2,2951637,25cef540-1681-447b-8bde-66f381a17ed1,60829516370011,0.727970,Trench 1,2024-10-25 14:16:19,2024-10-26,2024-10,Train,1,1,Quick
3,2956722,04998e07-91b6-4e64-bd43-fdf0b84cc550,60829567220016,0.815600,Trench 1,2024-10-20 09:51:53,2024-10-20,2024-10,Train,1,1,Quick
4,2927084,42daaf03-b729-42bb-910c-4c939f5d3865,60829270840018,0.625830,Trench 1,2024-10-10 10:19:36,2024-10-10,2024-10,Train,1,1,Quick


In [427]:
df2 = dfd.copy()

In [428]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16046 entries, 0 to 16045
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16046 non-null  object        
 1   digitalLoanAccountId   16046 non-null  object        
 2   loanAccountNumber      16046 non-null  object        
 3   aStackScore            16046 non-null  object        
 4   trenchCategory         16046 non-null  object        
 5   appln_submit_datetime  16046 non-null  datetime64[us]
 6   disbursementdate       16046 non-null  dbdate        
 7   Application_month      16046 non-null  object        
 8   Data_selection         16046 non-null  object        
 9   deffpd0                16046 non-null  Int64         
 10  flg_mature_fpd0        16046 non-null  Int64         
 11  new_loan_type          16046 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [429]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [430]:
# df_concat.to_csv(r"aStackScoretrench1fpd0.csv")

In [431]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [432]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aStackScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.492727,Week,aStackScore,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.394791,Month,aStackScore,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,0.379712,Week,aStackScore,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.359015,Week,aStackScore,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.380435,Week,aStackScore,1.1.0,FPD0,Overall


## FPD10

## Test

In [433]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3704941,8c979bb5-6c9f-490a-b0da-ee86863eebc4,60837049410017,0.3663243805082073,Trench 1,2025-09-25 19:31:42,2025-09-26,2025-09,Test,0,1,Quick
2,3734829,78ac4e9f-c623-4ef6-8348-014c8102ccdd,60837348290013,0.3290684613883727,Trench 1,2025-10-10 11:58:26,2025-10-10,2025-10,Test,1,1,Quick
3,3729658,da613289-ff12-4c81-aecd-4956b289b27c,60837296580017,0.3841893599931154,Trench 1,2025-10-10 12:05:46,2025-10-10,2025-10,Test,0,1,Quick
4,3734503,d76f4bec-573a-404b-82e7-7e97f08dee91,60837345030014,0.2833193516969392,Trench 1,2025-10-10 08:16:52,2025-10-10,2025-10,Test,0,1,Quick
5,3734511,1eb7892a-863e-48ca-ae43-1446856ad64b,60837345110014,0.4478539678574054,Trench 1,2025-10-10 08:33:05,2025-10-10,2025-10,Test,0,1,Quick


In [434]:
df1 = dfd.copy()

## Train

In [435]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2951581,bb4e9677-026e-4c0a-b6a9-cf29d22dab56,60829515810011,0.652590,Trench 1,2024-10-18 15:28:38,2024-10-18,2024-10,Train,0,1,Quick
1,2922203,96d99c43-79f0-486b-9cd2-fc5ca11d4950,60829222030017,0.583011,Trench 1,2024-10-08 14:50:54,2024-10-08,2024-10,Train,0,1,Quick
2,2965881,817732f6-e191-458f-b2c9-c7e0930fa455,60829658810011,0.281811,Trench 1,2024-10-23 11:11:48,2024-10-23,2024-10,Train,0,1,Quick
3,2935031,31a0425e-6649-46bc-b219-c2dfaa8b7fbf,60829350310012,0.736851,Trench 1,2024-10-13 01:58:03,2024-10-14,2024-10,Train,1,1,Quick
4,2981553,4f5936c8-1eff-47b0-857d-7aa79fd301fb,60829815530012,0.871518,Trench 1,2024-10-29 12:19:47,2024-10-29,2024-10,Train,1,1,Quick


In [436]:
df2 = dfd.copy()

In [437]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15562 entries, 0 to 15561
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             15562 non-null  object        
 1   digitalLoanAccountId   15562 non-null  object        
 2   loanAccountNumber      15562 non-null  object        
 3   aStackScore            15562 non-null  object        
 4   trenchCategory         15562 non-null  object        
 5   appln_submit_datetime  15562 non-null  datetime64[us]
 6   disbursementdate       15562 non-null  dbdate        
 7   Application_month      15562 non-null  object        
 8   Data_selection         15562 non-null  object        
 9   deffpd10               15562 non-null  Int64         
 10  flg_mature_fpd10       15562 non-null  Int64         
 11  new_loan_type          15562 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [438]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [439]:
# df_concat.to_csv(r"aStackScoretrench1fpd10.csv")

In [440]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [441]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aStackScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.544336,Week,aStackScore,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.549608,Month,aStackScore,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.585794,Week,aStackScore,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.518239,Week,aStackScore,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.562636,Week,aStackScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [442]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [443]:
df1 = dfd.copy()

## Train

In [444]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2970129,b186d9a8-6888-46dc-8918-2e9b02fafe9b,60829701290029,0.699538,Trench 1,2024-10-25 11:08:27,2024-10-25,2024-10,Train,0,1,Quick
1,2955221,08b01ba9-4178-4775-94a2-f29263c9bbb4,60829552210016,0.582693,Trench 1,2024-10-19 17:07:47,2024-10-19,2024-10,Train,0,1,Quick
2,2962511,734df7ec-824a-4812-bfa6-9fc0d02cd6d0,60829625110016,0.187757,Trench 1,2024-10-21 22:30:44,2024-10-22,2024-10,Train,0,1,Quick
3,2913903,06f72b1d-fbff-47f4-9859-483221a959e1,60829139030017,0.412660,Trench 1,2024-10-05 21:05:59,2024-10-05,2024-10,Train,0,1,Quick
4,2978010,4a22b0a6-c0b6-41d8-aea2-35114e933d49,60829780100019,0.490274,Trench 1,2024-10-27 18:37:22,2024-10-27,2024-10,Train,1,1,Quick


In [445]:
df2 = dfd.copy()

In [446]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14772 entries, 0 to 14771
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             14772 non-null  Int64         
 1   digitalLoanAccountId   14772 non-null  object        
 2   loanAccountNumber      14772 non-null  object        
 3   aStackScore            14772 non-null  float64       
 4   trenchCategory         14772 non-null  object        
 5   appln_submit_datetime  14772 non-null  datetime64[us]
 6   disbursementdate       14772 non-null  dbdate        
 7   Application_month      14772 non-null  object        
 8   Data_selection         14772 non-null  object        
 9   deffpd30               14772 non-null  Int64         
 10  flg_mature_fpd30       14772 non-null  Int64         
 11  new_loan_type          14772 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [447]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [448]:
# df_concat.to_csv(r"aStackScoretrench1fpd30.csv")

In [449]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [450]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aStackScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.609023,Week,aStackScore,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.577049,Month,aStackScore,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.592929,Week,aStackScore,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.537136,Week,aStackScore,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.582035,Week,aStackScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [451]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [452]:
df1 = dfd.copy()

## Train

In [453]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2925374,629296c0-7417-45a8-946d-8ea315851b8e,60829253740017,0.714348,Trench 1,2024-10-09 16:14:52,2024-10-09,2024-10,Train,1,1,Quick
1,2932745,3c2d48f6-a762-41b5-8dc7-050d4e8a5be7,60829327450017,0.422585,Trench 1,2024-10-12 11:13:06,2024-10-12,2024-10,Train,1,1,Quick
2,2981118,7ab21daf-8090-4789-b47f-28392c3d917e,60829811180017,0.943890,Trench 1,2024-10-29 19:23:09,2024-10-29,2024-10,Train,1,1,Quick
3,2932088,d2b1d1d8-baaa-4382-8251-a79279200e02,60829320880017,0.618859,Trench 1,2024-10-12 01:17:18,2024-10-12,2024-10,Train,1,1,Quick
4,2986547,ba5601db-435e-495d-82bf-b7b271b6cbbe,60829865470014,0.470276,Trench 1,2024-10-31 08:33:58,2024-10-31,2024-10,Train,0,1,Quick


In [454]:
df2 = dfd.copy()

In [455]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13599 entries, 0 to 13598
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             13599 non-null  Int64         
 1   digitalLoanAccountId   13599 non-null  object        
 2   loanAccountNumber      13599 non-null  object        
 3   aStackScore            13599 non-null  float64       
 4   trenchCategory         13599 non-null  object        
 5   appln_submit_datetime  13599 non-null  datetime64[us]
 6   disbursementdate       13599 non-null  dbdate        
 7   Application_month      13599 non-null  object        
 8   Data_selection         13599 non-null  object        
 9   deffspd30              13599 non-null  Int64         
 10  flg_mature_fspd_30     13599 non-null  Int64         
 11  new_loan_type          13599 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [456]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [457]:
# df_concat.to_csv(r"aStackScoretrench1fspd30.csv")

In [458]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [459]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aStackScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.630674,Week,aStackScore,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.552296,Month,aStackScore,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.589437,Week,aStackScore,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.455286,Week,aStackScore,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.559849,Week,aStackScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [460]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [461]:
df1 = dfd.copy()

## Train

In [462]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2988053,98aae7b9-f8ef-4f68-b582-602281ffa21e,60829880530015,0.629455,Trench 1,2024-10-31 16:34:31,2024-10-31,2024-10,Train,1,1,Quick
1,2931424,380f61d3-b61f-468d-8c52-fa3ef9ea607c,60829314240019,0.797071,Trench 1,2024-10-11 19:01:20,2024-10-11,2024-10,Train,1,1,Quick
2,2900772,5a34a87d-2f84-49fe-91eb-8f26cddccf19,60829007720017,0.813983,Trench 1,2024-10-07 09:06:39,2024-10-07,2024-10,Train,1,1,Quick
3,2943410,1ceb1d4e-433e-4a5e-8b26-f53c31a0442b,60829434100016,0.505643,Trench 1,2024-10-15 20:52:16,2024-10-15,2024-10,Train,1,1,Quick
4,2976137,e767afbc-b10f-4bfb-b1e1-2da8de977057,60829761370015,0.482714,Trench 1,2024-10-27 13:58:29,2024-10-27,2024-10,Train,0,1,Quick


In [463]:
df2 = dfd.copy()

In [464]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12449 entries, 0 to 12448
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12449 non-null  Int64         
 1   digitalLoanAccountId   12449 non-null  object        
 2   loanAccountNumber      12449 non-null  object        
 3   aStackScore            12449 non-null  float64       
 4   trenchCategory         12449 non-null  object        
 5   appln_submit_datetime  12449 non-null  datetime64[us]
 6   disbursementdate       12449 non-null  dbdate        
 7   Application_month      12449 non-null  object        
 8   Data_selection         12449 non-null  object        
 9   deffstpd30             12449 non-null  Int64         
 10  flg_mature_fstpd_30    12449 non-null  Int64         
 11  new_loan_type          12449 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [465]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [466]:
# df_concat.to_csv(r"aStackScoretrench1fstpd30.csv")

In [467]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,  
    'aStackScore', 
    'deffstpd30',   
    'FSTPD30',
    product_column='new_loan_type'
)

In [468]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aStackScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.493301,Week,aStackScore,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.491028,Month,aStackScore,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.500463,Week,aStackScore,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.414163,Week,aStackScore,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.516269,Week,aStackScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [469]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aStackScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aStackScore_FPD10_gini', 'aStackScore_FPD30_gini',
       'aStackScore_FSPD30_gini', 'aStackScore_FSTPD30_gini'],
      dtype=object)

In [470]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','aStackScore_FPD0_gini','aStackScore_FPD10_gini',
       'aStackScore_FPD30_gini', 'aStackScore_FSPD30_gini',
       'aStackScore_FSTPD30_gini']].copy()
final_df.rename(columns={'aStackScore_FPD0_gini':'aStackScore_t1_FPD0_gini'
                         , 'aStackScore_FPD10_gini':'aStackScore_t1_FPD10_gini'
                         , 'aStackScore_FPD30_gini':'aStackScore_t1_FPD30_gini'
                         , 'aStackScore_FSPD30_gini':'aStackScore_t1_FSPD30_gini'
                        , 'aStackScore_FSTPD30_gini':'aStackScore_t1_FSTPD30_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 1'
final_df['Model_display_name'] = 'alpha_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                     datetime64[ns]
end_date                       datetime64[ns]
period                                 object
Model_Name                             object
version                                object
loan_type                              object
bad_rate                               object
aStackScore_t1_FPD0_gini              float64
aStackScore_t1_FPD10_gini             float64
aStackScore_t1_FPD30_gini             float64
aStackScore_t1_FSPD30_gini            float64
aStackScore_t1_FSTPD30_gini           float64
Trench_category                        object
Model_display_name                     object
Product_type                           object
dtype: object

In [471]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aStackScore_t1_FPD0_gini,aStackScore_t1_FPD10_gini,aStackScore_t1_FPD30_gini,aStackScore_t1_FSPD30_gini,aStackScore_t1_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,aStackScore,1.1.0,Overall,FPD0,0.492727,NaN,NaN,NaN,NaN,Trench 1,alpha_stack_model_cash,CASH
1,2024-10-01,2024-10-31,Month,aStackScore,1.1.0,Overall,FPD0,0.394791,NaN,NaN,NaN,NaN,Trench 1,alpha_stack_model_cash,CASH
2,2024-10-07,2024-10-13,Week,aStackScore,1.1.0,Overall,FPD0,0.379712,NaN,NaN,NaN,NaN,Trench 1,alpha_stack_model_cash,CASH
3,2024-10-14,2024-10-20,Week,aStackScore,1.1.0,Overall,FPD0,0.359015,NaN,NaN,NaN,NaN,Trench 1,alpha_stack_model_cash,CASH
4,2024-10-21,2024-10-27,Week,aStackScore,1.1.0,Overall,FPD0,0.380435,NaN,NaN,NaN,NaN,Trench 1,alpha_stack_model_cash,CASH


In [472]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_t1_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=594f5040-d74b-49bc-b572-4df3eb3197fa>

# Trench 2

## FPD0

## Test

In [473]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2953030,0a4a844e-f93a-4d4f-9f4d-acbb5cef6f7c,60829530300013,0.4182113204332962,Trench 2,2025-10-10 11:43:32,2025-10-10,2025-10,Test,0,1,Quick
1,1665724,1bfa7d7a-871f-45f5-87aa-35655c792488,60816657240022,0.1052313698089733,Trench 2,2025-10-10 12:07:37,2025-10-13,2025-10,Test,0,1,Quick
2,3549543,a636b789-6464-4112-923c-58fa6d9969b5,60835495430015,0.4193721763372729,Trench 2,2025-10-10 12:44:06,2025-10-10,2025-10,Test,0,1,Quick
3,3504490,bab68f07-47f8-459e-a20c-85ac99a70ac8,60835044900051,0.23935773097745378,Trench 2,2025-10-10 08:24:07,2025-10-10,2025-10,Test,0,1,Quick
4,3441140,4200b450-7550-4b11-a73e-6e9be00f30da,60834411400012,0.2620348678890976,Trench 2,2025-10-10 10:55:23,2025-10-12,2025-10,Test,1,1,Quick


In [474]:
df1 = dfd.copy()

## Train

In [475]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1255515,90e5fe24-7d42-4b8c-9ec7-bc0e79b932a4,60812555150051,0.120132,Trench 2,2024-10-28 11:58:30,2024-10-30,2024-10,Train,0,1,Quick
1,2834038,4cd71bbd-6e27-44d7-af47-85944fd0c7f1,60828340380019,0.584342,Trench 2,2024-10-11 10:05:25,2024-10-11,2024-10,Train,1,1,Quick
2,2839904,fbeab111-1685-4fad-8230-9484e0a87da1,60828399040016,0.334947,Trench 2,2024-10-28 15:06:41,2024-10-28,2024-10,Train,0,1,Quick
3,2967755,ab758537-cf14-4b04-bebc-58ddad0150f3,60829677550014,0.660579,Trench 2,2024-10-24 00:15:22,2024-10-24,2024-10,Train,1,1,Quick
4,2723775,be26d1ce-f427-4b95-b9e0-82389f29f6aa,60827237750011,0.363181,Trench 2,2024-10-03 09:23:20,2024-10-03,2024-10,Train,0,1,Quick


In [476]:
df2 = dfd.copy()

In [477]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11114 entries, 0 to 11113
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11114 non-null  object        
 1   digitalLoanAccountId   11114 non-null  object        
 2   loanAccountNumber      11114 non-null  object        
 3   aStackScore            11114 non-null  object        
 4   trenchCategory         11114 non-null  object        
 5   appln_submit_datetime  11114 non-null  datetime64[us]
 6   disbursementdate       11114 non-null  dbdate        
 7   Application_month      11114 non-null  object        
 8   Data_selection         11114 non-null  object        
 9   deffpd0                11114 non-null  Int64         
 10  flg_mature_fpd0        11114 non-null  Int64         
 11  new_loan_type          11114 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [478]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [479]:
# df_concat.to_csv(r"aStackScoretrench2fpd0.csv")

In [480]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [481]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aStackScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.465684,Week,aStackScore,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.422023,Month,aStackScore,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,0.472046,Week,aStackScore,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.415392,Week,aStackScore,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.342969,Week,aStackScore,1.1.0,FPD0,Overall


## FPD10

## Test

In [482]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3238233,0c565a70-5dde-41c6-aff6-ee43b08d5f4a,60832382330017,0.3879119136306274,Trench 2,2025-10-13 08:23:04,2025-10-13,2025-10,Test,0,1,Quick
1,1204260,ab052dbc-f7da-49d9-83f5-1697f2c171dc,60812042600023,0.3919261827534121,Trench 2,2025-10-13 09:14:07,2025-10-13,2025-10,Test,0,1,Quick
2,3289023,15c6a2d9-4776-4cf9-9223-727f1a0507fb,60832890230018,0.31093508522901564,Trench 2,2025-10-13 10:17:40,2025-10-13,2025-10,Test,0,1,Quick
3,3521755,39923d59-541b-4ac6-97d9-263979e2a327,60835217550013,0.4280786758769984,Trench 2,2025-10-13 10:14:18,2025-10-13,2025-10,Test,0,1,Quick
4,3289928,e2b2cab1-f5d6-422b-8a89-b64a5b953a19,60832899280011,0.25383193883191735,Trench 2,2025-10-13 12:12:59,2025-10-13,2025-10,Test,0,1,Quick


In [483]:
df1 = dfd.copy()

## Train

In [484]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2499358,f70b5e93-6e8b-4b0f-8429-c7f8bf409490,60824993580018,0.754558,Trench 2,2024-10-09 22:29:45,2024-10-10,2024-10,Train,1,1,Quick
1,1467113,f473ee7f-2b13-462b-b5ee-cca8a66760cb,60814671130069,0.239309,Trench 2,2024-10-04 23:34:48,2024-10-05,2024-10,Train,0,1,Quick
2,2511793,ac3598d4-ecdc-4480-9683-24d8bc9c545a,60825117930017,0.718095,Trench 2,2024-10-15 02:06:48,2024-10-15,2024-10,Train,1,1,Quick
3,2628983,38b26ff0-c363-4b24-9c12-c7fc3f5a91fc,60826289830011,0.881249,Trench 2,2024-10-05 20:22:49,2024-10-05,2024-10,Train,1,1,Quick
4,2439958,c56f9b31-5dfd-4260-a660-fe72040e66f5,60824399580024,0.615813,Trench 2,2024-10-14 03:04:27,2024-10-14,2024-10,Train,0,1,Quick


In [485]:
df2 = dfd.copy()

In [486]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10924 entries, 0 to 10923
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10924 non-null  object        
 1   digitalLoanAccountId   10924 non-null  object        
 2   loanAccountNumber      10924 non-null  object        
 3   aStackScore            10924 non-null  object        
 4   trenchCategory         10924 non-null  object        
 5   appln_submit_datetime  10924 non-null  datetime64[us]
 6   disbursementdate       10924 non-null  dbdate        
 7   Application_month      10924 non-null  object        
 8   Data_selection         10924 non-null  object        
 9   deffpd10               10924 non-null  Int64         
 10  flg_mature_fpd10       10924 non-null  Int64         
 11  new_loan_type          10924 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [487]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [488]:
# df_concat.to_csv(r"aStackScoretrench1fpd10.csv")

In [489]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [490]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aStackScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.615146,Week,aStackScore,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.547853,Month,aStackScore,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.591411,Week,aStackScore,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.555232,Week,aStackScore,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.428945,Week,aStackScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [491]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
)
, 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [492]:
df1 = dfd.copy()

## Train

In [493]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1960827,603801aa-ca4d-4eb9-b135-7c5d27dec643,60819608270018,0.834206,Trench 2,2024-10-26 11:45:02,2024-10-27,2024-10,Train,0,1,Quick
1,2416962,82b5aca9-e9db-4d33-ba9a-ecf79a1435e8,60824169620011,0.188844,Trench 2,2024-10-19 23:22:17,2024-10-20,2024-10,Train,0,1,Quick
2,2591868,3e133ef0-3df9-4ab0-a2a1-1cf9bf6371f4,60825918680019,0.412056,Trench 2,2024-10-01 00:22:57,2024-10-01,2024-10,Train,0,1,Quick
3,2638225,bebda478-3b3c-400a-9ab9-4379aee76324,60826382250014,0.567091,Trench 2,2024-10-12 07:53:33,2024-10-17,2024-10,Train,0,1,Quick
4,2853765,92ac9846-1dbe-4b16-b75a-91533d61580d,60828537650019,0.672325,Trench 2,2024-10-25 22:16:43,2024-10-26,2024-10,Train,0,1,Quick


In [494]:
df2 = dfd.copy()

In [495]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10578 entries, 0 to 10577
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10578 non-null  Int64         
 1   digitalLoanAccountId   10578 non-null  object        
 2   loanAccountNumber      10578 non-null  object        
 3   aStackScore            10578 non-null  float64       
 4   trenchCategory         10578 non-null  object        
 5   appln_submit_datetime  10578 non-null  datetime64[us]
 6   disbursementdate       10578 non-null  dbdate        
 7   Application_month      10578 non-null  object        
 8   Data_selection         10578 non-null  object        
 9   deffpd30               10578 non-null  Int64         
 10  flg_mature_fpd30       10578 non-null  Int64         
 11  new_loan_type          10578 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [496]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [497]:
# df_concat.to_csv(r"aStackScoretrench1fpd30.csv")

In [498]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [499]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aStackScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.672161,Week,aStackScore,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.559772,Month,aStackScore,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.627584,Week,aStackScore,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.529913,Week,aStackScore,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.448268,Week,aStackScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [500]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [501]:
df1 = dfd.copy()

## Train

In [502]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2570660,37827f43-b8e1-4fbf-bf7e-37cff4205fe4,60825706600011,0.738909,Trench 2,2024-10-10 22:48:39,2024-10-11,2024-10,Train,1,1,Quick
1,1021940,0d2ddb0e-c108-45ec-a752-3aa53eaaa1f7,60810219400109,0.074307,Trench 2,2024-10-07 11:40:51,2024-10-07,2024-10,Train,0,1,Quick
2,2180819,4a48e2f4-f87a-478c-9585-9f58f1905dfe,60821808190017,0.549340,Trench 2,2024-10-11 11:06:12,2024-10-11,2024-10,Train,0,1,Quick
3,2509678,c0b6bc88-3161-48b7-89a5-02bdd5a9f95b,60825096780012,0.334382,Trench 2,2024-10-06 11:22:23,2024-10-10,2024-10,Train,0,1,Quick
4,2626626,65d5bdc5-90ae-4279-ab76-63ea37ff4ff3,60826266260014,0.236156,Trench 2,2024-10-15 12:36:04,2024-10-15,2024-10,Train,0,1,Quick


In [503]:
df2 = dfd.copy()

In [504]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9921 non-null   Int64         
 1   digitalLoanAccountId   9921 non-null   object        
 2   loanAccountNumber      9921 non-null   object        
 3   aStackScore            9921 non-null   float64       
 4   trenchCategory         9921 non-null   object        
 5   appln_submit_datetime  9921 non-null   datetime64[us]
 6   disbursementdate       9921 non-null   dbdate        
 7   Application_month      9921 non-null   object        
 8   Data_selection         9921 non-null   object        
 9   deffspd30              9921 non-null   Int64         
 10  flg_mature_fspd_30     9921 non-null   Int64         
 11  new_loan_type          9921 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [505]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [506]:
# df_concat.to_csv(r"aStackScoretrench2fspd30.csv")

In [507]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [508]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aStackScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.583149,Week,aStackScore,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.578261,Month,aStackScore,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.604414,Week,aStackScore,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.615705,Week,aStackScore,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.454545,Week,aStackScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [509]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [510]:
df1 = dfd.copy()

## Train

In [511]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,1551007,acf6f3d9-a94d-4e65-8fdb-76b723d441da,60815510070015,0.327006,Trench 2,2024-10-08 12:32:23,2024-10-08,2024-10,Train,1,1,Quick
1,2908775,50a67783-4c8a-469c-ae2e-cc64f7dcad74,60829087750012,0.509206,Trench 2,2024-10-04 10:53:17,2024-10-04,2024-10,Train,0,1,Quick
2,2100363,2906fd54-eb70-4387-b595-0f0aedb5527f,60821003630016,0.236583,Trench 2,2024-10-09 00:24:21,2024-10-09,2024-10,Train,0,1,Quick
3,2554288,3a03f2bb-1f90-4fac-89fd-04517ec71ece,60825542880017,0.261793,Trench 2,2024-10-08 17:46:31,2024-10-08,2024-10,Train,0,1,Quick
4,2172435,be4dc00d-7065-4333-b7df-0fa28466f4db,60821724350018,0.469755,Trench 2,2024-10-28 05:57:11,2024-10-28,2024-10,Train,1,1,Quick


In [512]:
df2 = dfd.copy()

In [513]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9206 entries, 0 to 9205
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9206 non-null   Int64         
 1   digitalLoanAccountId   9206 non-null   object        
 2   loanAccountNumber      9206 non-null   object        
 3   aStackScore            9206 non-null   float64       
 4   trenchCategory         9206 non-null   object        
 5   appln_submit_datetime  9206 non-null   datetime64[us]
 6   disbursementdate       9206 non-null   dbdate        
 7   Application_month      9206 non-null   object        
 8   Data_selection         9206 non-null   object        
 9   deffstpd30             9206 non-null   Int64         
 10  flg_mature_fstpd_30    9206 non-null   Int64         
 11  new_loan_type          9206 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [514]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [515]:
# df_concat.to_csv(r"aStackScoretrench2fstpd30.csv")

In [516]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype( 
    df_concat, 
    'aStackScore', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [517]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aStackScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.505994,Week,aStackScore,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.530044,Month,aStackScore,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.566274,Week,aStackScore,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.548177,Week,aStackScore,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.449289,Week,aStackScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [518]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aStackScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aStackScore_FPD10_gini', 'aStackScore_FPD30_gini',
       'aStackScore_FSPD30_gini', 'aStackScore_FSTPD30_gini'],
      dtype=object)

In [519]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','aStackScore_FPD0_gini','aStackScore_FPD10_gini',
       'aStackScore_FPD30_gini', 'aStackScore_FSPD30_gini',
       'aStackScore_FSTPD30_gini']].copy()
final_df.rename(columns={'aStackScore_FPD0_gini':'aStackScore_t2_FPD0_gini'
                         , 'aStackScore_FPD10_gini':'aStackScore_t2_FPD10_gini'
                         , 'aStackScore_FPD30_gini':'aStackScore_t2_FPD30_gini'
                         , 'aStackScore_FSPD30_gini':'aStackScore_t2_FSPD30_gini'
                        , 'aStackScore_FSTPD30_gini':'aStackScore_t2_FSTPD30_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 2'
final_df['Model_display_name'] = 'alpha_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                     datetime64[ns]
end_date                       datetime64[ns]
period                                 object
Model_Name                             object
version                                object
loan_type                              object
bad_rate                               object
aStackScore_t2_FPD0_gini              float64
aStackScore_t2_FPD10_gini             float64
aStackScore_t2_FPD30_gini             float64
aStackScore_t2_FSPD30_gini            float64
aStackScore_t2_FSTPD30_gini           float64
Trench_category                        object
Model_display_name                     object
Product_type                           object
dtype: object

In [520]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aStackScore_t2_FPD0_gini,aStackScore_t2_FPD10_gini,aStackScore_t2_FPD30_gini,aStackScore_t2_FSPD30_gini,aStackScore_t2_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,aStackScore,1.1.0,Overall,FPD0,0.465684,NaN,NaN,NaN,NaN,Trench 2,alpha_stack_model_cash,CASH
1,2024-10-01,2024-10-31,Month,aStackScore,1.1.0,Overall,FPD0,0.422023,NaN,NaN,NaN,NaN,Trench 2,alpha_stack_model_cash,CASH
2,2024-10-07,2024-10-13,Week,aStackScore,1.1.0,Overall,FPD0,0.472046,NaN,NaN,NaN,NaN,Trench 2,alpha_stack_model_cash,CASH
3,2024-10-14,2024-10-20,Week,aStackScore,1.1.0,Overall,FPD0,0.415392,NaN,NaN,NaN,NaN,Trench 2,alpha_stack_model_cash,CASH
4,2024-10-21,2024-10-27,Week,aStackScore,1.1.0,Overall,FPD0,0.342969,NaN,NaN,NaN,NaN,Trench 2,alpha_stack_model_cash,CASH


In [521]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_t2_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1b6b9f24-0fc9-429c-a10a-57afb0e4b650>

# Trench 3

## FPD0

## Test

In [522]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2832384,03847a36-0b78-468b-9248-919edac720f1,60828323840024,0.2858757352144149,Trench 3,2025-10-01 08:15:53,2025-10-01,2025-10,Test,0,1,Quick
1,1886292,08ce90dd-2e33-485b-88f3-b684c6bc63ae,60818862920033,0.5490042483794184,Trench 3,2025-10-01 19:08:33,2025-10-01,2025-10,Test,0,1,Quick
2,1080796,27ed2366-6b6c-4c53-b984-ef57da6466bd,60810807960064,0.31933992372334347,Trench 3,2025-10-02 01:05:15,2025-10-02,2025-10,Test,0,1,Quick
3,1809770,7d77f354-adef-4ba9-8ed4-7fd1cb666211,60818097700026,0.49061321060129415,Trench 3,2025-10-10 12:04:33,2025-10-10,2025-10,Test,0,1,Quick
4,2516985,6f56e9d0-e88d-4da6-9ac9-e2adeb1747f5,60825169850039,0.34435611810401856,Trench 3,2025-10-10 11:16:25,2025-10-10,2025-10,Test,0,1,Quick


In [523]:
df1 = dfd.copy()

## Train

In [524]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2884527,dfa615e5-ac4c-4240-9dc5-24f5ddd0dbf8,60828845270024,0.449327,Trench 3,2024-10-12 02:50:33,2024-10-12,2024-10,Train,0,1,Quick
1,2516764,dc287c29-7ca1-4ca3-aa8f-47182cb926f3,60825167640021,0.393564,Trench 3,2024-10-30 19:05:51,2024-10-30,2024-10,Train,0,1,Quick
2,2545037,904dce9a-7e9c-46f8-b58b-9e1c010a2748,60825450370027,0.354074,Trench 3,2024-10-02 11:01:54,2024-10-02,2024-10,Train,0,1,Quick
3,2567612,6c34ead3-8e17-4fbf-9c9f-20c690435ded,60825676120024,0.498791,Trench 3,2024-10-30 14:42:35,2024-10-31,2024-10,Train,0,1,Quick
4,2613981,b7629b1b-270e-47d0-a030-84d5a1c6a6b9,60826139810021,0.718466,Trench 3,2024-10-08 16:59:47,2024-10-08,2024-10,Train,0,1,Quick


In [525]:
df2 = dfd.copy()

In [526]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11069 entries, 0 to 11068
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11069 non-null  object        
 1   digitalLoanAccountId   11069 non-null  object        
 2   loanAccountNumber      11069 non-null  object        
 3   aStackScore            11069 non-null  object        
 4   trenchCategory         11069 non-null  object        
 5   appln_submit_datetime  11069 non-null  datetime64[us]
 6   disbursementdate       11069 non-null  dbdate        
 7   Application_month      11069 non-null  object        
 8   Data_selection         11069 non-null  object        
 9   deffpd0                11069 non-null  Int64         
 10  flg_mature_fpd0        11069 non-null  Int64         
 11  new_loan_type          11069 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [527]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [528]:
# df_concat.to_csv(r"aStackScoretrench3fpd0.csv")

In [529]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [530]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,aStackScore_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.380488,Week,aStackScore,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.365971,Month,aStackScore,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,0.357708,Week,aStackScore,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.459742,Week,aStackScore,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.378547,Week,aStackScore,1.1.0,FPD0,Overall


## FPD10

## Test

In [531]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1935788,7eceb181-3d8f-41d7-8486-2022ca9499a2,60819357880055,0.392885634691008,Trench 3,2025-10-09 11:49:18,2025-10-09,2025-10,Test,0,1,Quick
1,2748366,70222d59-0f75-4089-a828-2e1bfd28d268,60827483660025,0.4462152256154672,Trench 3,2025-10-09 11:47:29,2025-10-09,2025-10,Test,0,1,Quick
2,3056951,ef88f86a-58f8-44a7-8989-6f3fefc444f1,60830569510044,0.23552026234338302,Trench 3,2025-10-09 11:43:56,2025-10-09,2025-10,Test,0,1,Quick
3,2311054,f70db2ce-13de-48e0-8ea3-0ef3279fd38e,60823110540033,0.35970154122733444,Trench 3,2025-10-09 10:48:02,2025-10-09,2025-10,Test,0,1,Quick
4,2863783,218fc028-5b1d-444a-ae6e-ea201d7244a0,60828637830024,0.41328333800398875,Trench 3,2025-10-09 08:22:35,2025-10-09,2025-10,Test,0,1,Quick


In [532]:
df1 = dfd.copy()

## Train

In [533]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2574279,0c4f7047-7ba0-42aa-9ff5-591322734b4c,60825742790024,0.527494,Trench 3,2024-10-26 15:50:34,2024-10-26,2024-10,Train,0,1,Quick
1,1801183,7e0a2cff-240f-4385-8618-9fffeea626a8,60818011830047,0.378225,Trench 3,2024-10-09 18:37:59,2024-10-09,2024-10,Train,0,1,Quick
2,2248972,88dc693d-9889-4716-a92c-810bb0dac757,60822489720021,0.293716,Trench 3,2024-10-28 16:31:08,2024-10-28,2024-10,Train,0,1,Quick
3,1371177,75515663-5030-48d4-a189-bc4eac9f8453,60813711770044,0.281713,Trench 3,2024-10-02 02:40:28,2024-10-02,2024-10,Train,0,1,Quick
4,1251993,8907abbf-8fea-4645-b32b-63324c053d55,60812519930026,0.306299,Trench 3,2024-10-29 19:14:49,2024-10-29,2024-10,Train,0,1,Quick


In [534]:
df2 = dfd.copy()

In [535]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10913 entries, 0 to 10912
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10913 non-null  object        
 1   digitalLoanAccountId   10913 non-null  object        
 2   loanAccountNumber      10913 non-null  object        
 3   aStackScore            10913 non-null  object        
 4   trenchCategory         10913 non-null  object        
 5   appln_submit_datetime  10913 non-null  datetime64[us]
 6   disbursementdate       10913 non-null  dbdate        
 7   Application_month      10913 non-null  object        
 8   Data_selection         10913 non-null  object        
 9   deffpd10               10913 non-null  Int64         
 10  flg_mature_fpd10       10913 non-null  Int64         
 11  new_loan_type          10913 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [536]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [537]:
# df_concat.to_csv(r"aStackScoretrench1fpd10.csv")

In [538]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [539]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,aStackScore_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.610390,Week,aStackScore,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.595177,Month,aStackScore,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.450000,Week,aStackScore,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.759259,Week,aStackScore,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.489115,Week,aStackScore,1.1.0,FPD10,Overall


## FPD30

## Test

In [540]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [541]:
df1 = dfd.copy()

## Train

In [542]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1608893,9f8144b7-adea-43f3-9794-2655cb4a758a,60816088930038,0.139458,Trench 3,2024-10-17 11:52:15,2024-10-17,2024-10,Train,0,1,Quick
1,1465292,80ecaff3-d33d-4bad-88c0-4dd9388bf1d9,60814652920027,0.331273,Trench 3,2024-10-31 03:58:02,2024-11-10,2024-10,Train,0,1,Quick
2,2046237,01e32847-b240-4d33-b694-a466e3c054a3,60820462370027,0.326066,Trench 3,2024-10-04 19:01:58,2024-10-04,2024-10,Train,0,1,Quick
3,2908997,c9ce14d0-0897-4af1-acdb-a3616a588fb3,60829089970031,0.827032,Trench 3,2024-10-12 13:46:37,2024-10-12,2024-10,Train,0,1,Quick
4,2531279,634040f6-6baa-442e-8035-0cfaaab74205,60825312790022,0.478295,Trench 3,2024-10-10 06:34:20,2024-10-10,2024-10,Train,0,1,Quick


In [543]:
df2 = dfd.copy()

In [544]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10433 entries, 0 to 10432
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10433 non-null  Int64         
 1   digitalLoanAccountId   10433 non-null  object        
 2   loanAccountNumber      10433 non-null  object        
 3   aStackScore            10433 non-null  float64       
 4   trenchCategory         10433 non-null  object        
 5   appln_submit_datetime  10433 non-null  datetime64[us]
 6   disbursementdate       10433 non-null  dbdate        
 7   Application_month      10433 non-null  object        
 8   Data_selection         10433 non-null  object        
 9   deffpd30               10433 non-null  Int64         
 10  flg_mature_fpd30       10433 non-null  Int64         
 11  new_loan_type          10433 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [545]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [546]:
# df_concat.to_csv(r"aStackScoretrench3fpd30.csv")

In [547]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [548]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,aStackScore_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.610390,Week,aStackScore,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.627072,Month,aStackScore,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.516509,Week,aStackScore,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.801835,Week,aStackScore,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.457207,Week,aStackScore,1.1.0,FPD30,Overall


## FSPD30

## Test

In [549]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [550]:
df1 = dfd.copy()

## Train

In [551]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2533813,4a9b96a4-0e5d-4c14-9f7c-2156ec93f920,60825338130021,0.501631,Trench 3,2024-10-31 14:37:47,2024-10-31,2024-10,Train,0,1,Quick
1,2350151,75c17f8a-4d8c-4476-948f-f32a32d63273,60823501510021,0.418375,Trench 3,2024-10-04 11:39:25,2024-10-04,2024-10,Train,0,1,Quick
2,1910984,7d67faad-ddea-41c6-adfa-80981c12eac1,60819109840031,0.378440,Trench 3,2024-10-02 11:28:22,2024-10-04,2024-10,Train,0,1,Quick
3,2157119,4128534d-5d98-4945-90ce-9e9a0bf21840,60821571190027,0.342478,Trench 3,2024-10-23 19:32:57,2024-10-23,2024-10,Train,0,1,Quick
4,2504233,2533e000-d526-41a7-b04e-d8de700b1dca,60825042330025,0.588223,Trench 3,2024-10-29 02:08:38,2024-10-29,2024-10,Train,0,1,Quick


In [552]:
df2 = dfd.copy()

In [553]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9249 entries, 0 to 9248
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9249 non-null   Int64         
 1   digitalLoanAccountId   9249 non-null   object        
 2   loanAccountNumber      9249 non-null   object        
 3   aStackScore            9249 non-null   float64       
 4   trenchCategory         9249 non-null   object        
 5   appln_submit_datetime  9249 non-null   datetime64[us]
 6   disbursementdate       9249 non-null   dbdate        
 7   Application_month      9249 non-null   object        
 8   Data_selection         9249 non-null   object        
 9   deffspd30              9249 non-null   Int64         
 10  flg_mature_fspd_30     9249 non-null   Int64         
 11  new_loan_type          9249 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [554]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [555]:
# df_concat.to_csv(r"aStackScoretrench3fspd30.csv")

In [556]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'aStackScore', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [557]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,aStackScore_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.646802,Week,aStackScore,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.611353,Month,aStackScore,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.430312,Week,aStackScore,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.763350,Week,aStackScore,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.527891,Week,aStackScore,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [558]:
sq = r""" 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,requestPayload as requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Alpha-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  p.start_time,
  p.prediction aStackScore,
  coalesce (p.trenchCategory, REGEXP_EXTRACT(m.requestPayload_clean, r"trenchCategory[:=]['\"]?([^'\"]+)['\"]?")) trenchCategory
  from parsed p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [559]:
df1 = dfd.copy()

## Train

In [560]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction aStackScore,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.aStackScore,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.aStackScore is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,aStackScore,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2588018,6a647e1b-8c12-457f-a91b-8dd7028b6280,60825880180026,0.382413,Trench 3,2024-10-28 11:13:14,2024-10-28,2024-10,Train,0,1,Quick
1,2333952,06935c14-06b7-4423-a217-9d63ccee3fa3,60823339520028,0.576660,Trench 3,2024-10-23 05:46:23,2024-10-23,2024-10,Train,1,1,Quick
2,2260129,6fb99481-1d5f-4a76-bc0d-24578a46155c,60822601290056,0.458085,Trench 3,2024-10-10 18:25:46,2024-10-10,2024-10,Train,0,1,Quick
3,2274858,4482f7e2-5673-4321-9b58-5418bb58d421,60822748580029,0.378681,Trench 3,2024-10-30 13:58:31,2024-10-30,2024-10,Train,0,1,Quick
4,2521858,22129122-7027-448f-9a32-41b330812d9d,60825218580038,0.587897,Trench 3,2024-10-28 19:15:45,2024-10-28,2024-10,Train,0,1,Quick


In [561]:
df2 = dfd.copy()

In [562]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8020 non-null   Int64         
 1   digitalLoanAccountId   8020 non-null   object        
 2   loanAccountNumber      8020 non-null   object        
 3   aStackScore            8020 non-null   float64       
 4   trenchCategory         8020 non-null   object        
 5   appln_submit_datetime  8020 non-null   datetime64[us]
 6   disbursementdate       8020 non-null   dbdate        
 7   Application_month      8020 non-null   object        
 8   Data_selection         8020 non-null   object        
 9   deffstpd30             8020 non-null   Int64         
 10  flg_mature_fstpd_30    8020 non-null   Int64         
 11  new_loan_type          8020 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [563]:
df_concat['aStackScore'] = pd.to_numeric(df_concat['aStackScore'], errors='coerce')

In [564]:
# df_concat.to_csv(r"aStackScoretrench3fstpd30.csv")

In [565]:
# gini_results = calculate_periodic_gini(df_concat, 'aStackScore', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'aStackScore', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [566]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,aStackScore_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.543902,Week,aStackScore,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.518087,Month,aStackScore,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.303820,Week,aStackScore,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.772277,Week,aStackScore,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.523652,Week,aStackScore,1.1.0,FSTPD30,Overall


## combining the dataframe

In [567]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'aStackScore_FPD0_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'loan_type',
       'aStackScore_FPD10_gini', 'aStackScore_FPD30_gini',
       'aStackScore_FSPD30_gini', 'aStackScore_FSTPD30_gini'],
      dtype=object)

In [568]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','aStackScore_FPD0_gini','aStackScore_FPD10_gini',
       'aStackScore_FPD30_gini', 'aStackScore_FSPD30_gini',
       'aStackScore_FSTPD30_gini']].copy()
final_df.rename(columns={'aStackScore_FPD0_gini':'aStackScore_t3_FPD0_gini'
                         , 'aStackScore_FPD10_gini':'aStackScore_t3_FPD10_gini'
                         , 'aStackScore_FPD30_gini':'aStackScore_t3_FPD30_gini'
                         , 'aStackScore_FSPD30_gini':'aStackScore_t3_FSPD30_gini'
                        , 'aStackScore_FSTPD30_gini':'aStackScore_t3_FSTPD30_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 3'
final_df['Model_display_name'] = 'alpha_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                     datetime64[ns]
end_date                       datetime64[ns]
period                                 object
Model_Name                             object
version                                object
loan_type                              object
bad_rate                               object
aStackScore_t3_FPD0_gini              float64
aStackScore_t3_FPD10_gini             float64
aStackScore_t3_FPD30_gini             float64
aStackScore_t3_FSPD30_gini            float64
aStackScore_t3_FSTPD30_gini           float64
Trench_category                        object
Model_display_name                     object
Product_type                           object
dtype: object

In [569]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,aStackScore_t3_FPD0_gini,aStackScore_t3_FPD10_gini,aStackScore_t3_FPD30_gini,aStackScore_t3_FSPD30_gini,aStackScore_t3_FSTPD30_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,aStackScore,1.1.0,Overall,FPD0,0.380488,NaN,NaN,NaN,NaN,Trench 3,alpha_stack_model_cash,CASH
1,2024-10-01,2024-10-31,Month,aStackScore,1.1.0,Overall,FPD0,0.365971,NaN,NaN,NaN,NaN,Trench 3,alpha_stack_model_cash,CASH
2,2024-10-07,2024-10-13,Week,aStackScore,1.1.0,Overall,FPD0,0.357708,NaN,NaN,NaN,NaN,Trench 3,alpha_stack_model_cash,CASH
3,2024-10-14,2024-10-20,Week,aStackScore,1.1.0,Overall,FPD0,0.459742,NaN,NaN,NaN,NaN,Trench 3,alpha_stack_model_cash,CASH
4,2024-10-21,2024-10-27,Week,aStackScore,1.1.0,Overall,FPD0,0.378547,NaN,NaN,NaN,NaN,Trench 3,alpha_stack_model_cash,CASH


In [570]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_t3_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c8cd022a-5375-49f1-a271-00120a1b2c51>

# Beta-Cash-Demo-Model

## Trench 1

## Test

In [571]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3735593,629272b1-2f65-4ad7-b659-3a34399830b0,60837355930016,0.42604456591659007,Trench 1,2025-10-10 17:19:13,2025-10-10,2025-10,Test,0,1,Quick
1,3756537,df96be1b-fb0c-4604-9af3-fbfcdfa8846a,60837565370012,0.5562252532790979,Trench 1,2025-10-20 14:19:56,2025-10-20,2025-10,Test,0,1,Quick
2,3749252,78f61824-64c1-4bb6-b0d4-8f8d7e824cfa,60837492520019,0.4789553352350351,Trench 1,2025-10-17 06:02:49,2025-10-17,2025-10,Test,0,1,Quick
3,3759598,0bbf2014-7d2e-4b65-9d08-6da8855f48da,60837595980017,0.4609584799654663,Trench 1,2025-10-21 21:19:13,2025-10-24,2025-10,Test,0,1,Quick
4,3730562,b692d079-2e3d-423f-93ca-e74251b62aa4,60837305620019,0.4747008263718064,Trench 1,2025-10-08 02:34:42,2025-10-11,2025-10,Test,1,1,Quick


In [572]:
df1 = dfd.copy()

## Train

In [573]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
 and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2967705,737d8fbd-4b57-4ffa-ad2d-ea80b8391002,60829677050014,0.463486,Trench 1,2024-10-28 10:45:11,2024-10-28,2024-10,Train,0,1,Quick
1,2909810,91cb284c-fb52-473d-a42b-f6a43abdd4a1,60829098100012,0.528967,Trench 1,2024-10-05 15:06:05,2024-10-05,2024-10,Train,1,1,Quick
2,2979022,4a0d63a8-e9c0-449c-8125-626140e1f2fb,60829790220017,0.462325,Trench 1,2024-10-28 09:46:19,2024-10-28,2024-10,Train,0,1,Quick
3,2952171,c3a54fba-338e-418e-ac76-79c0bc16088e,60829521710011,0.483908,Trench 1,2024-10-18 17:12:03,2024-10-18,2024-10,Train,0,1,Quick
4,2983794,cfc5f1ea-5003-422b-89ec-0cad98c234c7,60829837940015,0.236990,Trench 1,2024-10-30 01:15:17,2024-10-30,2024-10,Train,0,1,Quick


In [574]:
df2 = dfd.copy()

In [575]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16860 non-null  object        
 1   digitalLoanAccountId   16860 non-null  object        
 2   loanAccountNumber      16860 non-null  object        
 3   Beta_Cash_Demo_Score   16860 non-null  object        
 4   trenchCategory         16860 non-null  object        
 5   appln_submit_datetime  16860 non-null  datetime64[us]
 6   disbursementdate       16860 non-null  dbdate        
 7   Application_month      16860 non-null  object        
 8   Data_selection         16860 non-null  object        
 9   deffpd0                16860 non-null  Int64         
 10  flg_mature_fpd0        16860 non-null  Int64         
 11  new_loan_type          16860 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [576]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [577]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd0.csv")

In [578]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [579]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.162108,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.239410,Month,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,0.260402,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.270544,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.261032,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall


## FPD10

## Test

In [580]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3739289,9b6b4616-a5a2-4e09-a557-f31f702b720d,60837392890017,0.41451706044123854,Trench 1,2025-10-12 13:37:22,2025-10-12,2025-10,Test,0,1,Quick
1,3735452,d47893e5-4485-4575-88fc-4b591b7a72d6,60837354520014,0.4611891516407004,Trench 1,2025-10-10 16:29:36,2025-10-10,2025-10,Test,0,1,Quick
2,3726615,f4799043-6449-42a3-8bfb-742c770ffd9d,60837266150014,0.5932798929539214,Trench 1,2025-10-13 14:18:24,2025-10-14,2025-10,Test,0,1,Quick
3,3730453,bab337a1-3044-4584-b4f8-7bfcb567909a,60837304530015,0.45830241485823703,Trench 1,2025-10-07 23:19:12,2025-10-08,2025-10,Test,0,1,Quick
4,3747695,95b1acfc-c809-4196-b3bc-4f2de6874184,60837476950014,0.6173780858754521,Trench 1,2025-10-17 12:10:25,2025-10-17,2025-10,Test,1,1,Quick


In [581]:
df1 = dfd.copy()

## Train

In [582]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2932875,e155e93f-df46-4cd0-9166-3378dd7ad599,60829328750016,0.478085,Trench 1,2024-10-12 12:32:44,2024-10-13,2024-10,Train,0,1,Quick
1,2970312,13abeb5c-87f0-4505-b775-21510b99e01b,60829703120018,0.548129,Trench 1,2024-10-25 07:52:56,2024-10-25,2024-10,Train,1,1,Quick
2,2900142,519b3819-4968-433d-93e8-94ba6fb16adf,60829001420013,0.428623,Trench 1,2024-10-02 14:04:09,2024-10-02,2024-10,Train,0,1,Quick
3,2953793,d8304fe2-4f40-4f3b-a647-2431330af12e,60829537930019,0.597811,Trench 1,2024-10-19 11:06:43,2024-10-19,2024-10,Train,0,1,Quick
4,2959225,8ab9ec30-6392-4740-85c3-e65f1d4ffbe6,60829592250016,0.457122,Trench 1,2024-10-20 19:58:26,2024-10-20,2024-10,Train,0,1,Quick


In [583]:
df2 = dfd.copy()

In [584]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16350 entries, 0 to 16349
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16350 non-null  object        
 1   digitalLoanAccountId   16350 non-null  object        
 2   loanAccountNumber      16350 non-null  object        
 3   Beta_Cash_Demo_Score   16350 non-null  object        
 4   trenchCategory         16350 non-null  object        
 5   appln_submit_datetime  16350 non-null  datetime64[us]
 6   disbursementdate       16350 non-null  dbdate        
 7   Application_month      16350 non-null  object        
 8   Data_selection         16350 non-null  object        
 9   deffpd10               16350 non-null  Int64         
 10  flg_mature_fpd10       16350 non-null  Int64         
 11  new_loan_type          16350 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [585]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [586]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd10.csv")

In [587]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [588]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.235812,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.347381,Month,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.379712,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.319219,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.448780,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall


## FPD30

## Test

In [589]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [590]:
df1 = dfd.copy()

## Train

In [591]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2944414,94372175-e20b-42f5-832b-05d5eebc6ab9,60829444140014,0.500295,Trench 1,2024-10-16 10:17:26,2024-10-16,2024-10,Train,0,1,Quick
1,2963698,ccaf3ead-864f-4af3-84df-3f1674703e7d,60829636980016,0.545326,Trench 1,2024-10-22 13:15:53,2024-10-22,2024-10,Train,1,1,Quick
2,2958664,8664f089-4d4b-4e5f-a6d6-15ba44a58352,60829586640011,0.635146,Trench 1,2024-10-20 17:04:48,2024-10-22,2024-10,Train,0,1,Quick
3,2936482,df6aafd2-ddf0-4779-a443-785e76e7d220,60829364820019,0.596220,Trench 1,2024-10-13 14:14:08,2024-10-15,2024-10,Train,1,1,Quick
4,2955548,b7d08e48-1522-4ec0-ba7a-85388f57b72d,60829555480016,0.612554,Trench 1,2024-10-19 19:03:42,2024-10-19,2024-10,Train,0,1,Quick


In [592]:
df2 = dfd.copy()

In [593]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15515 entries, 0 to 15514
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             15515 non-null  Int64         
 1   digitalLoanAccountId   15515 non-null  object        
 2   loanAccountNumber      15515 non-null  object        
 3   Beta_Cash_Demo_Score   15515 non-null  float64       
 4   trenchCategory         15515 non-null  object        
 5   appln_submit_datetime  15515 non-null  datetime64[us]
 6   disbursementdate       15515 non-null  dbdate        
 7   Application_month      15515 non-null  object        
 8   Data_selection         15515 non-null  object        
 9   deffpd30               15515 non-null  Int64         
 10  flg_mature_fpd30       15515 non-null  Int64         
 11  new_loan_type          15515 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [594]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [595]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd30.csv")

In [596]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [597]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.285632,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.355098,Month,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.323583,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.327721,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.464095,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [598]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [599]:
df1 = dfd.copy()

## Train

In [600]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2907010,59b0e1b8-831c-4d20-ad22-cf8f92eab366,60829070100028,0.502680,Trench 1,2024-10-05 09:42:09,2024-10-05,2024-10,Train,0,1,Quick
1,2941951,f50ff6ed-f872-4b5f-9e11-f83152c9199d,60829419510012,0.525017,Trench 1,2024-10-15 12:28:56,2024-10-15,2024-10,Train,0,1,Quick
2,2943878,278db654-8c5d-4c9c-b3ad-353a5d89aa51,60829438780018,0.608458,Trench 1,2024-10-15 23:20:40,2024-10-16,2024-10,Train,0,1,Quick
3,2986436,e23030c5-cb8e-4d7c-bbe2-cce8580c0753,60829864360018,0.483757,Trench 1,2024-10-31 06:42:40,2024-10-31,2024-10,Train,0,1,Quick
4,2970411,57bcfac6-4b6e-4e5a-befa-809dc9dd03bb,60829704110011,0.502110,Trench 1,2024-10-25 09:18:11,2024-10-25,2024-10,Train,0,1,Quick


In [601]:
df2 = dfd.copy()

In [602]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14299 entries, 0 to 14298
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             14299 non-null  Int64         
 1   digitalLoanAccountId   14299 non-null  object        
 2   loanAccountNumber      14299 non-null  object        
 3   Beta_Cash_Demo_Score   14299 non-null  float64       
 4   trenchCategory         14299 non-null  object        
 5   appln_submit_datetime  14299 non-null  datetime64[us]
 6   disbursementdate       14299 non-null  dbdate        
 7   Application_month      14299 non-null  object        
 8   Data_selection         14299 non-null  object        
 9   deffspd30              14299 non-null  Int64         
 10  flg_mature_fspd_30     14299 non-null  Int64         
 11  new_loan_type          14299 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [603]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [604]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfspd30.csv")

In [605]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [606]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.319646,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.318281,Month,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.332984,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.306730,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.339559,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [607]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [608]:
df1 = dfd.copy()

## Train

In [609]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2946324,6cf8b86f-5b16-4f0f-b2f5-e3dd324db8cd,60829463240011,0.590540,Trench 1,2024-10-17 08:00:14,2024-10-17,2024-10,Train,1,1,Quick
1,2905733,37b5a601-b3e0-4e2e-9cd2-2732fd9b7ae8,60829057330011,0.235840,Trench 1,2024-10-03 10:49:26,2024-10-03,2024-10,Train,0,1,Quick
2,2973750,d0f25537-49fe-40a6-bafa-36c50e3718e7,60829737500018,0.562040,Trench 1,2024-10-29 11:03:32,2024-10-30,2024-10,Train,0,1,Quick
3,2973251,bc2aaf7f-8940-4313-ad2b-3b1d9ab5ec61,60829732510014,0.529040,Trench 1,2024-10-26 10:14:13,2024-10-26,2024-10,Train,0,1,Quick
4,2978632,3b84951d-707c-4792-a25a-5cac2fcf673b,60829786320013,0.574563,Trench 1,2024-10-28 16:21:45,2024-10-28,2024-10,Train,1,1,Quick


In [610]:
df2 = dfd.copy()

In [611]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13106 entries, 0 to 13105
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             13106 non-null  Int64         
 1   digitalLoanAccountId   13106 non-null  object        
 2   loanAccountNumber      13106 non-null  object        
 3   Beta_Cash_Demo_Score   13106 non-null  float64       
 4   trenchCategory         13106 non-null  object        
 5   appln_submit_datetime  13106 non-null  datetime64[us]
 6   disbursementdate       13106 non-null  dbdate        
 7   Application_month      13106 non-null  object        
 8   Data_selection         13106 non-null  object        
 9   deffstpd30             13106 non-null  Int64         
 10  flg_mature_fstpd_30    13106 non-null  Int64         
 11  new_loan_type          13106 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [612]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [613]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfstpd30.csv")

In [614]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [615]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.274895,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.293095,Month,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.326968,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.231325,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.333658,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [616]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'Beta_Cash_Demo_Score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'Beta_Cash_Demo_Score_FPD10_gini',
       'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini'], dtype=object)

In [617]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type','bad_rate','Beta_Cash_Demo_Score_FPD0_gini','Beta_Cash_Demo_Score_FPD10_gini',
      'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_Cash_Demo_Score_FPD0_gini':'Beta_Cash_Demo_Score_FPD0_t1_gini'
                         , 'Beta_Cash_Demo_Score_FPD10_gini':'Beta_Cash_Demo_Score_FPD10_t1_gini'
                         , 'Beta_Cash_Demo_Score_FPD30_gini':'Beta_Cash_Demo_Score_FPD30_t1_gini'
                         , 'Beta_Cash_Demo_Score_FSPD30_gini':'Beta_Cash_Demo_Score_FSPD30_t1_gini'
                        , 'Beta_Cash_Demo_Score_FSTPD30_gini':'Beta_Cash_Demo_Score_FSTPD30_t1_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 1'
final_df['Model_display_name'] = 'beta_demo_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                              datetime64[ns]
end_date                                datetime64[ns]
period                                          object
Model_Name                                      object
version                                         object
loan_type                                       object
bad_rate                                        object
Beta_Cash_Demo_Score_FPD0_t1_gini              float64
Beta_Cash_Demo_Score_FPD10_t1_gini             float64
Beta_Cash_Demo_Score_FPD30_t1_gini             float64
Beta_Cash_Demo_Score_FSPD30_t1_gini            float64
Beta_Cash_Demo_Score_FSTPD30_t1_gini           float64
Trench_category                                 object
Model_display_name                              object
Product_type                                    object
dtype: object

In [618]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,Beta_Cash_Demo_Score_FPD0_t1_gini,Beta_Cash_Demo_Score_FPD10_t1_gini,Beta_Cash_Demo_Score_FPD30_t1_gini,Beta_Cash_Demo_Score_FSPD30_t1_gini,Beta_Cash_Demo_Score_FSTPD30_t1_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.162108,NaN,NaN,NaN,NaN,Trench 1,beta_demo_model_cash,CASH
1,2024-10-01,2024-10-31,Month,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.239410,NaN,NaN,NaN,NaN,Trench 1,beta_demo_model_cash,CASH
2,2024-10-07,2024-10-13,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.260402,NaN,NaN,NaN,NaN,Trench 1,beta_demo_model_cash,CASH
3,2024-10-14,2024-10-20,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.270544,NaN,NaN,NaN,NaN,Trench 1,beta_demo_model_cash,CASH
4,2024-10-21,2024-10-27,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.261032,NaN,NaN,NaN,NaN,Trench 1,beta_demo_model_cash,CASH


In [619]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_model_cash_t1_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=878cad8d-c838-4ba3-9665-22d8eacd785a>

## Trench 2

## FPD0

## Test

In [620]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3068460,71ccb540-f30f-4293-8730-88b0aa8d6aa0,60830684600019,0.28531404802777655,Trench 2,2025-10-11 18:20:34,2025-10-11,2025-10,Test,0,1,Quick
1,1763519,1c4f014c-2f41-44ac-865b-20f3ae0b751e,60817635190019,0.356809917069828,Trench 2,2025-10-19 14:02:42,2025-10-19,2025-10,Test,0,1,Quick
2,3549543,a636b789-6464-4112-923c-58fa6d9969b5,60835495430015,0.4597191268102652,Trench 2,2025-10-10 12:44:06,2025-10-10,2025-10,Test,0,1,Quick
3,3217417,afacdd19-2ef0-4343-9e6b-a7ecc8a4a839,60832174170019,0.4779940950903083,Trench 2,2025-10-16 12:44:08,2025-10-18,2025-10,Test,0,1,Quick
4,3495892,6979686a-53f8-4e9c-9aab-a3c73cf81c11,60834958920016,0.5682964176432647,Trench 2,2025-10-14 22:51:42,2025-10-15,2025-10,Test,0,1,Quick


In [621]:
df1 = dfd.copy()

## Train

In [622]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
 and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1977661,0289d6e6-038e-45a8-9db2-d2b693ff7d87,60819776610017,0.487783,Trench 2,2024-10-05 23:25:42,2024-10-06,2024-10,Train,1,1,Quick
1,2407416,b8abdd1e-7354-45d9-8826-de38b1979d1d,60824074160018,0.518876,Trench 2,2024-10-25 13:22:17,2024-10-26,2024-10,Train,0,1,Quick
2,2451520,fca4df30-d21b-4dcb-ad3f-87546fec6a45,60824515200018,0.439202,Trench 2,2024-10-26 01:08:23,2024-10-29,2024-10,Train,0,1,Quick
3,2675827,eb08be48-6bfc-4edd-9598-025470a4a1bf,60826758270012,0.499387,Trench 2,2024-10-21 17:10:14,2024-10-22,2024-10,Train,0,1,Quick
4,2427305,877f8464-8aed-4bf9-9b46-ba336598e051,60824273050012,0.634899,Trench 2,2024-10-01 17:59:26,2024-10-01,2024-10,Train,1,1,Quick


In [623]:
df2 = dfd.copy()

In [624]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12220 entries, 0 to 12219
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12220 non-null  object        
 1   digitalLoanAccountId   12220 non-null  object        
 2   loanAccountNumber      12220 non-null  object        
 3   Beta_Cash_Demo_Score   12220 non-null  object        
 4   trenchCategory         12220 non-null  object        
 5   appln_submit_datetime  12220 non-null  datetime64[us]
 6   disbursementdate       12220 non-null  dbdate        
 7   Application_month      12220 non-null  object        
 8   Data_selection         12220 non-null  object        
 9   deffpd0                12220 non-null  Int64         
 10  flg_mature_fpd0        12220 non-null  Int64         
 11  new_loan_type          12220 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [625]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [626]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd0.csv")

In [627]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [628]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.256061,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.217154,Month,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,0.272374,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.252783,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.128160,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall


## FPD10

## Test

In [629]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3504490,bab68f07-47f8-459e-a20c-85ac99a70ac8,60835044900051,0.4980507394027957,Trench 2,2025-10-10 08:24:07,2025-10-10,2025-10,Test,0,1,Quick
1,3150715,bbf9d9b5-f29b-4ec3-989a-b49b0ec0aaf8,60831507150012,0.298353760731465,Trench 2,2025-10-11 13:25:08,2025-10-11,2025-10,Test,0,1,Quick
2,3176208,892b0847-c6f2-43ea-ab24-3ba53f42b333,60831762080012,0.6023025866971782,Trench 2,2025-10-10 18:02:27,2025-10-11,2025-10,Test,0,1,Quick
3,3592340,010b98b6-a8ce-4a5b-b314-212430210d68,60835923400013,0.546119289034815,Trench 2,2025-10-09 11:38:04,2025-10-10,2025-10,Test,0,1,Quick
4,1067794,37bb1ba6-6664-4a01-bae9-88e2fbffb8e6,60810677940061,0.4894876335837796,Trench 2,2025-10-15 15:45:42,2025-10-15,2025-10,Test,0,1,Quick


In [630]:
df1 = dfd.copy()

## Train

In [631]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1381545,6e8cf86f-f1ea-4de0-8825-a932ed4f89cc,60813815450019,0.589330,Trench 2,2024-10-15 15:00:38,2024-10-15,2024-10,Train,1,1,Quick
1,2831764,9240804d-3797-40ff-bb89-eff167a17bf6,60828317640028,0.564138,Trench 2,2024-10-21 21:21:34,2024-10-22,2024-10,Train,1,1,Quick
2,2119496,b239d12c-5bee-46d1-85c2-cac263352352,60821194960019,0.511868,Trench 2,2024-10-14 07:22:14,2024-10-14,2024-10,Train,0,1,Quick
3,1907275,a80c16f4-7f4c-4c44-9572-9b6399d7783a,60819072750016,0.596222,Trench 2,2024-10-05 13:31:56,2024-10-05,2024-10,Train,1,1,Quick
4,2440921,36b71579-7697-495d-9d5d-bf71d534c4ed,60824409210012,0.568476,Trench 2,2024-10-02 13:02:25,2024-10-02,2024-10,Train,0,1,Quick


In [632]:
df2 = dfd.copy()

In [633]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12026 entries, 0 to 12025
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             12026 non-null  object        
 1   digitalLoanAccountId   12026 non-null  object        
 2   loanAccountNumber      12026 non-null  object        
 3   Beta_Cash_Demo_Score   12026 non-null  object        
 4   trenchCategory         12026 non-null  object        
 5   appln_submit_datetime  12026 non-null  datetime64[us]
 6   disbursementdate       12026 non-null  dbdate        
 7   Application_month      12026 non-null  object        
 8   Data_selection         12026 non-null  object        
 9   deffpd10               12026 non-null  Int64         
 10  flg_mature_fpd10       12026 non-null  Int64         
 11  new_loan_type          12026 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [634]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [635]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd10.csv")

In [636]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [637]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.453619,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.287650,Month,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.274212,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.334702,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.114770,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall


## FPD30

## Test

In [638]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30, 
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1346048,896716c5-e4e5-4b3c-9218-0b605d293c72,60813460480017,0.5258142934829556,Trench 2,2025-09-25 11:37:29,2025-09-25,2025-09,Test,0,1,Quick


In [639]:
df1 = dfd.copy()

## Train

In [640]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1198939,af586111-519c-408f-8bdd-82f082fcb9ac,60811989390221,0.618712,Trench 2,2024-10-13 22:09:15,2024-10-13,2024-10,Train,0,1,Quick
1,2619107,9c83140a-0b2d-445a-b8eb-15314de4f537,60826191070013,0.574264,Trench 2,2024-10-14 11:27:49,2024-10-14,2024-10,Train,0,1,Quick
2,2315288,7ad6a458-0130-473e-81f5-57f6aed5f7c1,60823152880019,0.490412,Trench 2,2024-10-16 13:42:07,2024-10-16,2024-10,Train,1,1,Quick
3,1887248,64808748-8990-4a5c-b472-e4b843b98418,60818872480017,0.592460,Trench 2,2024-10-01 18:46:15,2024-10-01,2024-10,Train,1,1,Quick
4,2561035,303f73ce-2770-4e72-a02b-45dc4a392dfd,60825610350011,0.470572,Trench 2,2024-10-07 00:40:22,2024-10-07,2024-10,Train,0,1,Quick


In [641]:
df2 = dfd.copy()

In [642]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11660 entries, 0 to 11659
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11660 non-null  object        
 1   digitalLoanAccountId   11660 non-null  object        
 2   loanAccountNumber      11660 non-null  object        
 3   Beta_Cash_Demo_Score   11660 non-null  object        
 4   trenchCategory         11660 non-null  object        
 5   appln_submit_datetime  11660 non-null  datetime64[us]
 6   disbursementdate       11660 non-null  dbdate        
 7   Application_month      11660 non-null  object        
 8   Data_selection         11660 non-null  object        
 9   deffpd30               11660 non-null  Int64         
 10  flg_mature_fpd30       11660 non-null  Int64         
 11  new_loan_type          11660 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [643]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [644]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd30.csv")

In [645]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [646]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.435796,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.282906,Month,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.286411,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.303030,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.135635,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [647]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [648]:
df1 = dfd.copy()

## Train

In [649]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2595572,2a8848c8-0188-47ad-bac6-ca75b5e19def,60825955720012,0.583059,Trench 2,2024-10-21 22:29:14,2024-10-22,2024-10,Train,0,1,Quick
1,2621766,2bc8e74b-10bc-4c70-8b6d-294deecd08b9,60826217660015,0.571352,Trench 2,2024-10-09 23:22:01,2024-10-10,2024-10,Train,1,1,Quick
2,1034329,e028cc69-e00a-4ec9-aac0-cadb714b8f6b,60810343290021,0.468490,Trench 2,2024-10-17 01:17:34,2024-10-17,2024-10,Train,1,1,Quick
3,2659113,be9adc4b-da74-4df8-8702-c85eea3a3508,60826591130011,0.573762,Trench 2,2024-10-23 15:31:55,2024-10-25,2024-10,Train,1,1,Quick
4,1912861,e2433f6b-5574-4a94-8b15-00745ba778ff,60819128610013,0.588107,Trench 2,2024-10-25 10:55:51,2024-10-26,2024-10,Train,1,1,Quick


In [650]:
df2 = dfd.copy()

In [651]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10972 non-null  Int64         
 1   digitalLoanAccountId   10972 non-null  object        
 2   loanAccountNumber      10972 non-null  object        
 3   Beta_Cash_Demo_Score   10972 non-null  float64       
 4   trenchCategory         10972 non-null  object        
 5   appln_submit_datetime  10972 non-null  datetime64[us]
 6   disbursementdate       10972 non-null  dbdate        
 7   Application_month      10972 non-null  object        
 8   Data_selection         10972 non-null  object        
 9   deffspd30              10972 non-null  Int64         
 10  flg_mature_fspd_30     10972 non-null  Int64         
 11  new_loan_type          10972 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [652]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [653]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfspd30.csv")

In [654]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [655]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.427312,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.309332,Month,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.358267,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.363044,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.163012,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [656]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [657]:
df1 = dfd.copy()

## Train

In [658]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2540017,50dd788b-af24-4bad-830a-ba92508d950a,60825400170015,0.609682,Trench 2,2024-10-10 22:55:45,2024-10-11,2024-10,Train,1,1,Quick
1,2632286,889ab944-5513-4638-8098-e4bb4646d72a,60826322860013,0.596784,Trench 2,2024-10-10 09:16:58,2024-10-10,2024-10,Train,1,1,Quick
2,2599550,4803ff79-454b-49dc-97a3-d294602c9851,60825995500012,0.558273,Trench 2,2024-10-31 06:45:36,2024-10-31,2024-10,Train,1,1,Quick
3,2656992,603c609e-2f5a-478f-9004-fd0be731c5c7,60826569920018,0.433149,Trench 2,2024-10-19 14:40:58,2024-10-20,2024-10,Train,1,1,Quick
4,2617602,e13c9778-9bff-44cf-9658-59114c3303a7,60826176020015,0.590833,Trench 2,2024-10-05 16:52:01,2024-10-05,2024-10,Train,1,1,Quick


In [659]:
df2 = dfd.copy()

In [660]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10225 entries, 0 to 10224
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10225 non-null  Int64         
 1   digitalLoanAccountId   10225 non-null  object        
 2   loanAccountNumber      10225 non-null  object        
 3   Beta_Cash_Demo_Score   10225 non-null  float64       
 4   trenchCategory         10225 non-null  object        
 5   appln_submit_datetime  10225 non-null  datetime64[us]
 6   disbursementdate       10225 non-null  dbdate        
 7   Application_month      10225 non-null  object        
 8   Data_selection         10225 non-null  object        
 9   deffstpd30             10225 non-null  Int64         
 10  flg_mature_fstpd_30    10225 non-null  Int64         
 11  new_loan_type          10225 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [661]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [662]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfstpd30.csv")

In [663]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [664]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.358500,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.315782,Month,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.369074,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.373500,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.203213,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [665]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'Beta_Cash_Demo_Score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'Beta_Cash_Demo_Score_FPD10_gini',
       'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini'], dtype=object)

In [666]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','Beta_Cash_Demo_Score_FPD0_gini','Beta_Cash_Demo_Score_FPD10_gini',
      'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_Cash_Demo_Score_FPD0_gini':'Beta_Cash_Demo_Score_FPD0_t2_gini'
                         , 'Beta_Cash_Demo_Score_FPD10_gini':'Beta_Cash_Demo_Score_FPD10_t2_gini'
                         , 'Beta_Cash_Demo_Score_FPD30_gini':'Beta_Cash_Demo_Score_FPD30_t2_gini'
                         , 'Beta_Cash_Demo_Score_FSPD30_gini':'Beta_Cash_Demo_Score_FSPD30_t2_gini'
                        , 'Beta_Cash_Demo_Score_FSTPD30_gini':'Beta_Cash_Demo_Score_FSTPD30_t2_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 2'
final_df['Model_display_name'] = 'beta_demo_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                              datetime64[ns]
end_date                                datetime64[ns]
period                                          object
Model_Name                                      object
version                                         object
loan_type                                       object
bad_rate                                        object
Beta_Cash_Demo_Score_FPD0_t2_gini              float64
Beta_Cash_Demo_Score_FPD10_t2_gini             float64
Beta_Cash_Demo_Score_FPD30_t2_gini             float64
Beta_Cash_Demo_Score_FSPD30_t2_gini            float64
Beta_Cash_Demo_Score_FSTPD30_t2_gini           float64
Trench_category                                 object
Model_display_name                              object
Product_type                                    object
dtype: object

In [667]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,Beta_Cash_Demo_Score_FPD0_t2_gini,Beta_Cash_Demo_Score_FPD10_t2_gini,Beta_Cash_Demo_Score_FPD30_t2_gini,Beta_Cash_Demo_Score_FSPD30_t2_gini,Beta_Cash_Demo_Score_FSTPD30_t2_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.256061,NaN,NaN,NaN,NaN,Trench 2,beta_demo_model_cash,CASH
1,2024-10-01,2024-10-31,Month,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.217154,NaN,NaN,NaN,NaN,Trench 2,beta_demo_model_cash,CASH
2,2024-10-07,2024-10-13,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.272374,NaN,NaN,NaN,NaN,Trench 2,beta_demo_model_cash,CASH
3,2024-10-14,2024-10-20,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.252783,NaN,NaN,NaN,NaN,Trench 2,beta_demo_model_cash,CASH
4,2024-10-21,2024-10-27,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.128160,NaN,NaN,NaN,NaN,Trench 2,beta_demo_model_cash,CASH


In [668]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_model_cash_t2_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9303fa8f-ef25-4d88-ad63-34a540ae6b00>

## Trench 3

## FPD0

## Test

In [669]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2999824,b7d6f903-240a-4f26-aa1d-54c14811022a,60829998240039,0.3349292467596064,Trench 3,2025-10-08 06:17:43,2025-10-08,2025-10,Test,0,1,Quick
1,1130357,811a64ac-3222-465f-81f2-b43c59317314,60811303570024,0.43263834194536016,Trench 3,2025-10-23 23:12:01,2025-10-24,2025-10,Test,0,1,Quick
2,2838151,7653478e-bb05-4005-b865-5b32283b82a7,60828381510023,0.3725498007013658,Trench 3,2025-10-11 11:41:59,2025-10-12,2025-10,Test,0,1,Quick
3,2193082,aab723ba-4ca9-401c-aa67-98d0eea88472,60821930820033,0.5664235745393217,Trench 3,2025-10-10 13:22:35,2025-10-10,2025-10,Test,0,1,Quick
4,3682825,23b04a90-c51a-42c5-a3b5-d9343bc42913,60836828250028,0.4631835105573858,Trench 3,2025-10-17 22:54:27,2025-10-18,2025-10,Test,0,1,Quick


In [670]:
df1 = dfd.copy()

## Train

In [671]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd0 = 1
 and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2286875,1a0b18ea-233e-46bf-996f-30c805b3436a,60822868750023,0.585201,Trench 3,2024-10-05 10:35:40,2024-10-05,2024-10,Train,0,1,Quick
1,2127736,cbe26dc3-a509-4ec0-b609-5b78ad7b4431,60821277360047,0.403982,Trench 3,2024-10-11 12:02:27,2024-10-11,2024-10,Train,0,1,Quick
2,2793226,e9683aeb-7200-43dc-82bc-27dba9623601,60827932260037,0.372391,Trench 3,2024-10-17 14:22:36,2024-10-17,2024-10,Train,0,1,Quick
3,1145884,6409a9ef-6dc9-436f-86a1-bbc14c7a6e4d,60811458840027,0.172776,Trench 3,2024-10-01 09:04:19,2024-10-02,2024-10,Train,0,1,Quick
4,2400321,68a44518-29bc-4a4b-9078-7e3b50eed332,60824003210031,0.319145,Trench 3,2024-10-07 09:12:33,2024-10-07,2024-10,Train,0,1,Quick


In [672]:
df2 = dfd.copy()

In [673]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11455 entries, 0 to 11454
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11455 non-null  object        
 1   digitalLoanAccountId   11455 non-null  object        
 2   loanAccountNumber      11455 non-null  object        
 3   Beta_Cash_Demo_Score   11455 non-null  object        
 4   trenchCategory         11455 non-null  object        
 5   appln_submit_datetime  11455 non-null  datetime64[us]
 6   disbursementdate       11455 non-null  dbdate        
 7   Application_month      11455 non-null  object        
 8   Data_selection         11455 non-null  object        
 9   deffpd0                11455 non-null  Int64         
 10  flg_mature_fpd0        11455 non-null  Int64         
 11  new_loan_type          11455 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [674]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [675]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd0.csv")

In [676]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [677]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.061224,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
1,2024-10-01,2024-10-31,0.123607,Month,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
2,2024-10-07,2024-10-13,-0.001855,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
3,2024-10-14,2024-10-20,0.195829,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall
4,2024-10-21,2024-10-27,0.309463,Week,Beta_Cash_Demo_Score,1.1.0,FPD0,Overall


## FPD10

## Test

In [678]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3406942,a8f2b592-180f-4a38-867d-c8b5a0548a28,60834069420034,0.47069890450358465,Trench 3,2025-10-15 17:59:09,2025-10-15,2025-10,Test,0,1,Quick
1,3315151,a9518ddb-207f-4162-9dea-330591280a94,60833151510028,0.49456890962191435,Trench 3,2025-10-09 06:52:12,2025-10-09,2025-10,Test,0,1,Quick
2,2802174,b6613875-164f-4ca7-95db-46c1f3618293,60828021740021,0.2866559427536754,Trench 3,2025-10-10 15:26:23,2025-10-11,2025-10,Test,0,1,Quick
3,2120741,f5a444be-5468-4177-9a7f-a9b76ff0873c,60821207410046,0.2213769065102644,Trench 3,2025-10-15 17:27:17,2025-10-15,2025-10,Test,0,1,Quick
4,3326133,38ef0c26-202f-4185-88a5-14c2b22d4b7b,60833261330025,0.2936528812562057,Trench 3,2025-10-10 15:34:07,2025-10-10,2025-10,Test,0,1,Quick


In [679]:
df1 = dfd.copy()

## Train

In [680]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2260677,3e5dce8a-3330-45ae-863d-91735d35aedb,60822606770025,0.428095,Trench 3,2024-10-17 23:00:34,2024-10-18,2024-10,Train,0,1,Quick
1,2249368,7ca14255-8ba5-4e00-b13f-0a56d39e8c4c,60822493680021,0.300754,Trench 3,2024-10-24 15:32:53,2024-10-24,2024-10,Train,0,1,Quick
2,1887635,4615d9c0-3ad4-4518-aca6-6618b151ffc0,60818876350056,0.519203,Trench 3,2024-10-01 08:06:04,2024-10-01,2024-10,Train,0,1,Quick
3,2255216,4b3b8250-de37-4d0d-acc8-e5721b30246b,60822552160033,0.434551,Trench 3,2024-10-14 22:38:27,2024-10-16,2024-10,Train,0,1,Quick
4,2485725,77ac806b-edb2-4b9f-b902-68759fbb47e8,60824857250036,0.511726,Trench 3,2024-10-11 13:21:25,2024-10-11,2024-10,Train,0,1,Quick


In [681]:
df2 = dfd.copy()

In [682]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11298 entries, 0 to 11297
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11298 non-null  object        
 1   digitalLoanAccountId   11298 non-null  object        
 2   loanAccountNumber      11298 non-null  object        
 3   Beta_Cash_Demo_Score   11298 non-null  object        
 4   trenchCategory         11298 non-null  object        
 5   appln_submit_datetime  11298 non-null  datetime64[us]
 6   disbursementdate       11298 non-null  dbdate        
 7   Application_month      11298 non-null  object        
 8   Data_selection         11298 non-null  object        
 9   deffpd10               11298 non-null  Int64         
 10  flg_mature_fpd10       11298 non-null  Int64         
 11  new_loan_type          11298 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

In [683]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [684]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd10.csv")

In [685]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [686]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.211931,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
1,2024-10-01,2024-10-31,0.235389,Month,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
2,2024-10-07,2024-10-13,0.145455,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
3,2024-10-14,2024-10-20,0.281073,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall
4,2024-10-21,2024-10-27,0.323349,Week,Beta_Cash_Demo_Score,1.1.0,FPD10,Overall


## FPD30

## Test

In [687]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [688]:
df1 = dfd.copy()

## Train

In [689]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2044493,bae9f7df-e6c3-4b3f-b88b-8a6d89c20c3c,60820444930049,0.287313,Trench 3,2024-10-17 19:50:22,2024-10-17,2024-10,Train,0,1,Quick
1,1901719,30e30c8c-a9f2-4ff5-b0ba-008eb15e6209,60819017190043,0.540490,Trench 3,2024-10-08 15:46:14,2024-10-08,2024-10,Train,0,1,Quick
2,1112740,9a3b4818-5e68-455f-926a-8f4da679424c,60811127400025,0.525980,Trench 3,2024-10-12 06:36:19,2024-10-13,2024-10,Train,0,1,Quick
3,2322917,8bbb101e-0045-4dbc-b1b1-722282ac908a,60823229170034,0.436300,Trench 3,2024-10-08 12:53:27,2024-10-08,2024-10,Train,0,1,Quick
4,2453143,8fa88dda-44aa-41bd-97f6-d098cd6c2959,60824531430029,0.388271,Trench 3,2024-10-19 20:29:37,2024-10-19,2024-10,Train,1,1,Quick


In [690]:
df2 = dfd.copy()

In [691]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10792 entries, 0 to 10791
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10792 non-null  Int64         
 1   digitalLoanAccountId   10792 non-null  object        
 2   loanAccountNumber      10792 non-null  object        
 3   Beta_Cash_Demo_Score   10792 non-null  float64       
 4   trenchCategory         10792 non-null  object        
 5   appln_submit_datetime  10792 non-null  datetime64[us]
 6   disbursementdate       10792 non-null  dbdate        
 7   Application_month      10792 non-null  object        
 8   Data_selection         10792 non-null  object        
 9   deffpd30               10792 non-null  Int64         
 10  flg_mature_fpd30       10792 non-null  Int64         
 11  new_loan_type          10792 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [692]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [693]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfpd30.csv")

In [694]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [695]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,Beta_Cash_Demo_Score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.211931,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
1,2024-10-01,2024-10-31,0.243771,Month,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
2,2024-10-07,2024-10-13,0.169643,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
3,2024-10-14,2024-10-20,0.236058,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall
4,2024-10-21,2024-10-27,0.415254,Week,Beta_Cash_Demo_Score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [696]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [697]:
df1 = dfd.copy()

## Train

In [698]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2286875,1a0b18ea-233e-46bf-996f-30c805b3436a,60822868750023,0.585201,Trench 3,2024-10-05 10:35:40,2024-10-05,2024-10,Train,0,1,Quick
1,2127736,cbe26dc3-a509-4ec0-b609-5b78ad7b4431,60821277360047,0.403982,Trench 3,2024-10-11 12:02:27,2024-10-11,2024-10,Train,0,1,Quick
2,2793226,e9683aeb-7200-43dc-82bc-27dba9623601,60827932260037,0.372391,Trench 3,2024-10-17 14:22:36,2024-10-17,2024-10,Train,0,1,Quick
3,1145884,6409a9ef-6dc9-436f-86a1-bbc14c7a6e4d,60811458840027,0.172776,Trench 3,2024-10-01 09:04:19,2024-10-02,2024-10,Train,0,1,Quick
4,2400321,68a44518-29bc-4a4b-9078-7e3b50eed332,60824003210031,0.319145,Trench 3,2024-10-07 09:12:33,2024-10-07,2024-10,Train,0,1,Quick


In [699]:
df2 = dfd.copy()

In [700]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9594 entries, 0 to 9593
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9594 non-null   Int64         
 1   digitalLoanAccountId   9594 non-null   object        
 2   loanAccountNumber      9594 non-null   object        
 3   Beta_Cash_Demo_Score   9594 non-null   float64       
 4   trenchCategory         9594 non-null   object        
 5   appln_submit_datetime  9594 non-null   datetime64[us]
 6   disbursementdate       9594 non-null   dbdate        
 7   Application_month      9594 non-null   object        
 8   Data_selection         9594 non-null   object        
 9   deffspd30              9594 non-null   Int64         
 10  flg_mature_fspd_30     9594 non-null   Int64         
 11  new_loan_type          9594 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [701]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [702]:
# df_concat.to_csv(r"Beta_Cash_Demo_Scoretrenchnunfspd30.csv")

In [703]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [704]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.259831,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
1,2024-10-01,2024-10-31,0.283025,Month,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
2,2024-10-07,2024-10-13,0.002157,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
3,2024-10-14,2024-10-20,0.363873,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall
4,2024-10-21,2024-10-27,0.404337,Week,Beta_Cash_Demo_Score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [705]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),
model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory
  from latest_request p 
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
   ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [706]:
df1 = dfd.copy()

## Train

In [707]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_Cash_Demo_Score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_Cash_Demo_Score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_Cash_Demo_Score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,Beta_Cash_Demo_Score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2540017,50dd788b-af24-4bad-830a-ba92508d950a,60825400170015,0.609682,Trench 2,2024-10-10 22:55:45,2024-10-11,2024-10,Train,1,1,Quick
1,2632286,889ab944-5513-4638-8098-e4bb4646d72a,60826322860013,0.596784,Trench 2,2024-10-10 09:16:58,2024-10-10,2024-10,Train,1,1,Quick
2,2599550,4803ff79-454b-49dc-97a3-d294602c9851,60825995500012,0.558273,Trench 2,2024-10-31 06:45:36,2024-10-31,2024-10,Train,1,1,Quick
3,2656992,603c609e-2f5a-478f-9004-fd0be731c5c7,60826569920018,0.433149,Trench 2,2024-10-19 14:40:58,2024-10-20,2024-10,Train,1,1,Quick
4,2617602,e13c9778-9bff-44cf-9658-59114c3303a7,60826176020015,0.590833,Trench 2,2024-10-05 16:52:01,2024-10-05,2024-10,Train,1,1,Quick


In [708]:
df2 = dfd.copy()

In [709]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10225 entries, 0 to 10224
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10225 non-null  Int64         
 1   digitalLoanAccountId   10225 non-null  object        
 2   loanAccountNumber      10225 non-null  object        
 3   Beta_Cash_Demo_Score   10225 non-null  float64       
 4   trenchCategory         10225 non-null  object        
 5   appln_submit_datetime  10225 non-null  datetime64[us]
 6   disbursementdate       10225 non-null  dbdate        
 7   Application_month      10225 non-null  object        
 8   Data_selection         10225 non-null  object        
 9   deffstpd30             10225 non-null  Int64         
 10  flg_mature_fstpd_30    10225 non-null  Int64         
 11  new_loan_type          10225 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [710]:
df_concat['Beta_Cash_Demo_Score'] = pd.to_numeric(df_concat['Beta_Cash_Demo_Score'], errors='coerce')

In [711]:
# gini_results = calculate_periodic_gini(df_concat, 'Beta_Cash_Demo_Score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_Cash_Demo_Score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [712]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,Beta_Cash_Demo_Score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-09-30,2024-10-06,0.358500,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
1,2024-10-01,2024-10-31,0.315782,Month,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
2,2024-10-07,2024-10-13,0.369074,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
3,2024-10-14,2024-10-20,0.373500,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall
4,2024-10-21,2024-10-27,0.203213,Week,Beta_Cash_Demo_Score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [713]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'Beta_Cash_Demo_Score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'Beta_Cash_Demo_Score_FPD10_gini',
       'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini'], dtype=object)

In [714]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','Beta_Cash_Demo_Score_FPD0_gini','Beta_Cash_Demo_Score_FPD10_gini',
      'Beta_Cash_Demo_Score_FPD30_gini',
       'Beta_Cash_Demo_Score_FSPD30_gini',
       'Beta_Cash_Demo_Score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_Cash_Demo_Score_FPD0_gini':'Beta_Cash_Demo_Score_FPD0_t3_gini'
                         , 'Beta_Cash_Demo_Score_FPD10_gini':'Beta_Cash_Demo_Score_FPD10_t3_gini'
                         , 'Beta_Cash_Demo_Score_FPD30_gini':'Beta_Cash_Demo_Score_FPD30_t3_gini'
                         , 'Beta_Cash_Demo_Score_FSPD30_gini':'Beta_Cash_Demo_Score_FSPD30_t3_gini'
                        , 'Beta_Cash_Demo_Score_FSTPD30_gini':'Beta_Cash_Demo_Score_FSTPD30_t3_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 3'
final_df['Model_display_name'] = 'beta_demo_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                              datetime64[ns]
end_date                                datetime64[ns]
period                                          object
Model_Name                                      object
version                                         object
loan_type                                       object
bad_rate                                        object
Beta_Cash_Demo_Score_FPD0_t3_gini              float64
Beta_Cash_Demo_Score_FPD10_t3_gini             float64
Beta_Cash_Demo_Score_FPD30_t3_gini             float64
Beta_Cash_Demo_Score_FSPD30_t3_gini            float64
Beta_Cash_Demo_Score_FSTPD30_t3_gini           float64
Trench_category                                 object
Model_display_name                              object
Product_type                                    object
dtype: object

In [715]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,Beta_Cash_Demo_Score_FPD0_t3_gini,Beta_Cash_Demo_Score_FPD10_t3_gini,Beta_Cash_Demo_Score_FPD30_t3_gini,Beta_Cash_Demo_Score_FSPD30_t3_gini,Beta_Cash_Demo_Score_FSTPD30_t3_gini,Trench_category,Model_display_name,Product_type
0,2024-09-30,2024-10-06,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.061224,NaN,NaN,NaN,NaN,Trench 3,beta_demo_model_cash,CASH
1,2024-10-01,2024-10-31,Month,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.123607,NaN,NaN,NaN,NaN,Trench 3,beta_demo_model_cash,CASH
2,2024-10-07,2024-10-13,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,-0.001855,NaN,NaN,NaN,NaN,Trench 3,beta_demo_model_cash,CASH
3,2024-10-14,2024-10-20,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.195829,NaN,NaN,NaN,NaN,Trench 3,beta_demo_model_cash,CASH
4,2024-10-21,2024-10-27,Week,Beta_Cash_Demo_Score,1.1.0,Overall,FPD0,0.309463,NaN,NaN,NaN,NaN,Trench 3,beta_demo_model_cash,CASH


In [716]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_model_cash_t3_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7ea68cb3-d61c-4f8a-bb8d-52df7b0e6665>

# Beta-Cash-AppScore-Model

## Trench 1

## FPD0

## Test

In [717]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3730522,9e86edd0-0393-4b66-a6af-89cfb9c270f1,60837305220011,0.470956,Trench 1,2025-10-08 01:15:33,2025-10-08,2025-10,Test,1,1,Quick
1,3747036,6fda22b7-7b1c-4c7f-931e-25fb9c529ea9,60837470360015,0.505330,Trench 1,2025-10-15 23:14:02,2025-10-16,2025-10,Test,0,1,Quick
2,3752646,c2d84ae3-5bb1-45da-bf2c-3704b412f4f8,60837526460011,0.494008,Trench 1,2025-10-18 17:00:53,2025-10-18,2025-10,Test,0,1,Quick
3,3753329,5fcd09b2-f1bb-48e6-833a-4abaf533fa94,60837533290014,0.450587,Trench 1,2025-10-19 01:55:24,2025-10-19,2025-10,Test,0,1,Quick
4,3738417,961a173e-c4ac-41aa-b3db-3560a3e3f481,60837384170013,0.443776,Trench 1,2025-10-12 08:09:28,2025-10-12,2025-10,Test,1,1,Quick


In [718]:
df1 = dfd.copy()

## Train

In [719]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2381103,1ec6226d-608f-4bcf-91ae-ec3ff5823f99,60823811030014,0.437078,Trench 1,2024-01-27 09:33:40,2024-02-07,2024-01,Train,0,1,Quick
1,2369998,547634d8-2d5c-4df3-a90f-3dba97bea330,60823699980013,0.502605,Trench 1,2024-01-16 01:03:31,2024-01-18,2024-01,Train,0,1,Quick
2,2375605,bf6d8639-f48f-4341-9d8f-796058dc0445,60823756050012,0.458095,Trench 1,2024-01-21 17:38:24,2024-01-21,2024-01,Train,0,1,Quick
3,2374436,7038a448-f81e-4e09-bb96-b603c9c22283,60823744360019,0.459017,Trench 1,2024-01-23 05:34:30,2024-01-23,2024-01,Train,0,1,Quick
4,2380329,70fc2344-3c9d-4077-91ca-4cfd44da7873,60823803290016,0.552506,Trench 1,2024-01-26 12:54:56,2024-01-26,2024-01,Train,0,1,Quick


In [720]:
df2 = dfd.copy()

In [721]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10404 entries, 0 to 10403
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10404 non-null  object        
 1   digitalLoanAccountId   10404 non-null  object        
 2   loanAccountNumber      10404 non-null  object        
 3   beta_cash_app_score    10404 non-null  float64       
 4   trenchCategory         10404 non-null  object        
 5   appln_submit_datetime  10404 non-null  datetime64[us]
 6   disbursementdate       10404 non-null  dbdate        
 7   Application_month      10404 non-null  object        
 8   Data_selection         10404 non-null  object        
 9   deffpd0                10404 non-null  Int64         
 10  flg_mature_fpd0        10404 non-null  Int64         
 11  new_loan_type          10404 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), objec

In [722]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [723]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd0.csv")

In [724]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [725]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,beta_cash_app_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,-0.076923,Week,beta_cash_app_score,1.1.0,FPD0,Overall
1,2024-01-01,2024-01-31,0.255906,Month,beta_cash_app_score,1.1.0,FPD0,Overall
2,2024-01-08,2024-01-14,0.875000,Week,beta_cash_app_score,1.1.0,FPD0,Overall
3,2024-01-15,2024-01-21,0.430233,Week,beta_cash_app_score,1.1.0,FPD0,Overall
4,2024-01-22,2024-01-28,-0.051429,Week,beta_cash_app_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [726]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,3745167,ee4b0b65-c945-41da-a774-dcad8f6df143,60837451670017,0.418212,Trench 1,2025-10-16 10:28:32,2025-10-16,2025-10,Test,0,1,Quick
1,3718903,8d2235ed-56f8-4c78-9f2a-026a0d3db2d5,60837189030012,0.444388,Trench 1,2025-10-15 11:58:27,2025-10-15,2025-10,Test,1,1,Quick
2,3731138,7d541e25-fc36-4850-a544-e3df1a34483d,60837311380011,0.407659,Trench 1,2025-10-08 12:41:18,2025-10-08,2025-10,Test,0,1,Quick
3,3745142,c479b00b-4ea2-43d4-a131-f53b8bc0ed34,60837451420018,0.509814,Trench 1,2025-10-15 06:12:53,2025-10-15,2025-10,Test,0,1,Quick
4,3731439,e66fb818-3a6d-438b-b683-b0999c2f8c5d,60837314390011,0.400688,Trench 1,2025-10-08 14:59:37,2025-10-08,2025-10,Test,0,1,Quick


In [727]:
df1 = dfd.copy()

## Train

In [728]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2383905,397557b6-ce08-4bf4-958a-5c4453261132,60823839050018,0.451541,Trench 1,2024-01-29 21:35:10,2024-01-29,2024-01,Train,0,1,Quick
1,2371218,7c76397d-a1e7-4a55-ad02-90b91d5098e8,60823712180019,0.522512,Trench 1,2024-01-17 11:05:27,2024-01-19,2024-01,Train,0,1,Quick
2,2379970,e69e5c6c-cde1-4a19-878c-34e834938256,60823799700016,0.610168,Trench 1,2024-01-30 13:17:37,2024-02-11,2024-01,Train,0,1,Quick
3,2355213,924cbca0-b08f-4e11-bf2b-a9e08bbca47f,60823552130011,0.643342,Trench 1,2024-01-01 06:36:41,2024-01-01,2024-01,Train,1,1,Quick
4,2384419,5627daeb-edd1-41fe-b70e-80c387c8a2a3,60823844190013,0.510946,Trench 1,2024-01-30 16:17:55,2024-01-30,2024-01,Train,1,1,Quick


In [729]:
df2 = dfd.copy()

In [730]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10109 entries, 0 to 10108
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10109 non-null  object        
 1   digitalLoanAccountId   10109 non-null  object        
 2   loanAccountNumber      10109 non-null  object        
 3   beta_cash_app_score    10109 non-null  float64       
 4   trenchCategory         10109 non-null  object        
 5   appln_submit_datetime  10109 non-null  datetime64[us]
 6   disbursementdate       10109 non-null  dbdate        
 7   Application_month      10109 non-null  object        
 8   Data_selection         10109 non-null  object        
 9   deffpd10               10109 non-null  Int64         
 10  flg_mature_fpd10       10109 non-null  Int64         
 11  new_loan_type          10109 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), objec

In [731]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [732]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd0.csv")

In [733]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [734]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,beta_cash_app_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,0.071429,Week,beta_cash_app_score,1.1.0,FPD10,Overall
1,2024-01-01,2024-01-31,0.289216,Month,beta_cash_app_score,1.1.0,FPD10,Overall
2,2024-01-08,2024-01-14,0.696970,Week,beta_cash_app_score,1.1.0,FPD10,Overall
3,2024-01-15,2024-01-21,0.691489,Week,beta_cash_app_score,1.1.0,FPD10,Overall
4,2024-01-22,2024-01-28,-0.081481,Week,beta_cash_app_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [735]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [736]:
df1 = dfd.copy()

## Train

In [737]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,2378847,9b90ab7d-fce8-4781-a331-a0114a422f7a,60823788470011,0.693507,Trench 1,2024-01-24 21:21:37,2024-01-24,2024-01,Train,0,1,Quick
1,2368194,254748a9-0e8a-4b64-8de5-c79d0b00ee2a,60823681940011,0.435142,Trench 1,2024-01-14 13:18:26,2024-01-18,2024-01,Train,0,1,Flex
2,2355281,bc130ce0-3bff-4fa9-b03b-e6451f093a4c,60823552810012,0.511867,Trench 1,2024-01-01 11:55:20,2024-01-01,2024-01,Train,0,1,Quick
3,2370325,c337b6f8-47ac-4701-972e-568338b3e151,60823703250017,0.696469,Trench 1,2024-01-16 12:34:32,2024-01-16,2024-01,Train,1,1,Quick
4,2381067,1f67acc6-683c-4c71-bc40-66f5de1ab698,60823810670017,0.411479,Trench 1,2024-01-27 08:54:00,2024-01-27,2024-01,Train,0,1,Quick


In [738]:
df2 = dfd.copy()

In [739]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9630 entries, 0 to 9629
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9630 non-null   Int64         
 1   digitalLoanAccountId   9630 non-null   object        
 2   loanAccountNumber      9630 non-null   object        
 3   beta_cash_app_score    9630 non-null   float64       
 4   trenchCategory         9630 non-null   object        
 5   appln_submit_datetime  9630 non-null   datetime64[us]
 6   disbursementdate       9630 non-null   dbdate        
 7   Application_month      9630 non-null   object        
 8   Data_selection         9630 non-null   object        
 9   deffpd30               9630 non-null   Int64         
 10  flg_mature_fpd30       9630 non-null   Int64         
 11  new_loan_type          9630 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [740]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [741]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd30.csv")

In [742]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [743]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,beta_cash_app_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,0.071429,Week,beta_cash_app_score,1.1.0,FPD30,Overall
1,2024-01-01,2024-01-31,0.210702,Month,beta_cash_app_score,1.1.0,FPD30,Overall
2,2024-01-08,2024-01-14,0.696970,Week,beta_cash_app_score,1.1.0,FPD30,Overall
3,2024-01-15,2024-01-21,0.652778,Week,beta_cash_app_score,1.1.0,FPD30,Overall
4,2024-01-22,2024-01-28,-0.303571,Week,beta_cash_app_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [744]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [745]:
df1 = dfd.copy()

## Train

In [746]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2364283,467b78b9-20f9-4f2a-9feb-8156a7bfc742,60823642830015,0.488828,Trench 1,2024-01-10 12:22:31,2024-01-17,2024-01,Train,0,1,Flex
1,2371526,32ae999c-4aab-4258-a86b-965172864934,60823715260018,0.487185,Trench 1,2024-01-17 15:28:10,2024-01-18,2024-01,Train,0,1,Quick
2,2368621,30724107-65e0-4b97-b109-479f22282400,60823686210019,0.423962,Trench 1,2024-01-14 17:40:22,2024-01-14,2024-01,Train,0,1,Quick
3,2355259,bcf4d47c-ecb8-40ba-b2a6-1ccabe265977,60823552590014,0.442796,Trench 1,2024-01-01 11:27:32,2024-01-01,2024-01,Train,0,1,Quick
4,2360987,dcbeecb0-738f-4f8f-b3a5-77b062733fb4,60823609870014,0.441135,Trench 1,2024-01-07 06:31:10,2024-01-12,2024-01,Train,0,1,Quick


In [747]:
df2 = dfd.copy()

In [748]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9163 entries, 0 to 9162
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9163 non-null   Int64         
 1   digitalLoanAccountId   9163 non-null   object        
 2   loanAccountNumber      9163 non-null   object        
 3   beta_cash_app_score    9163 non-null   float64       
 4   trenchCategory         9163 non-null   object        
 5   appln_submit_datetime  9163 non-null   datetime64[us]
 6   disbursementdate       9163 non-null   dbdate        
 7   Application_month      9163 non-null   object        
 8   Data_selection         9163 non-null   object        
 9   deffspd30              9163 non-null   Int64         
 10  flg_mature_fspd_30     9163 non-null   Int64         
 11  new_loan_type          9163 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [749]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [750]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fspd30.csv")

In [751]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [752]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,beta_cash_app_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,-0.090909,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
1,2024-01-01,2024-01-31,0.116168,Month,beta_cash_app_score,1.1.0,FSPD30,Overall
2,2024-01-08,2024-01-14,0.875000,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
3,2024-01-15,2024-01-21,0.185185,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
4,2024-01-22,2024-01-28,-0.081481,Week,beta_cash_app_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [753]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [754]:
df1 = dfd.copy()

## Train

In [755]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,2374217,590870ae-1caf-46c1-b1c8-68e2cc58e515,60823742170011,0.603269,Trench 1,2024-01-20 13:46:14,2024-01-20,2024-01,Train,0,1,Quick
1,2383292,02c3a4de-b087-4fb8-9147-cbaca7c8c968,60823832920012,0.719519,Trench 1,2024-01-29 12:11:49,2024-01-30,2024-01,Train,1,1,Quick
2,2381820,edc4ee96-a750-48fe-be01-7cc8e31a7278,60823818200016,0.523328,Trench 1,2024-01-27 19:50:32,2024-01-30,2024-01,Train,0,1,Quick
3,2363496,1f0c1f0d-0462-4810-8186-13981d0e4f11,60823634960012,0.480935,Trench 1,2024-01-09 14:01:45,2024-01-09,2024-01,Train,0,1,Quick
4,2364203,404facc2-d462-401c-b7e3-ec3de4a80ed4,60823642030019,0.349152,Trench 1,2024-01-10 11:58:02,2024-01-10,2024-01,Train,0,1,Flex


In [756]:
df2 = dfd.copy()

In [757]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8770 entries, 0 to 8769
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8770 non-null   Int64         
 1   digitalLoanAccountId   8770 non-null   object        
 2   loanAccountNumber      8770 non-null   object        
 3   beta_cash_app_score    8770 non-null   float64       
 4   trenchCategory         8770 non-null   object        
 5   appln_submit_datetime  8770 non-null   datetime64[us]
 6   disbursementdate       8770 non-null   dbdate        
 7   Application_month      8770 non-null   object        
 8   Data_selection         8770 non-null   object        
 9   deffstpd30             8770 non-null   Int64         
 10  flg_mature_fstpd_30    8770 non-null   Int64         
 11  new_loan_type          8770 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [758]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [759]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fstpd30.csv")

In [760]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [761]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,beta_cash_app_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,-0.135802,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
1,2024-01-01,2024-01-31,0.132730,Month,beta_cash_app_score,1.1.0,FSTPD30,Overall
2,2024-01-08,2024-01-14,0.569892,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
3,2024-01-15,2024-01-21,0.086364,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
4,2024-01-22,2024-01-28,0.005714,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [762]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'beta_cash_app_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini'], dtype=object)

In [763]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate',
                     'beta_cash_app_score_FPD0_gini'
                     ,'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini']].copy()
final_df.rename(columns={'beta_cash_app_score_FPD0_gini':'beta_cash_app_score_FPD0_t1_gini'
                         , 'beta_cash_app_score_FPD10_gini':'beta_cash_app_score_FPD10_t1_gini'
                         , 'beta_cash_app_score_FPD30_gini':'beta_cash_app_score_FPD30_t1_gini'
                         , 'beta_cash_app_score_FSPD30_gini':'beta_cash_app_score_FSPD30_t1_gini'
                        , 'beta_cash_app_score_FSTPD30_gini':'beta_cash_app_score_FSTPD30_t1_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 1'
final_df['Model_display_name'] = 'apps_score_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                             datetime64[ns]
end_date                               datetime64[ns]
period                                         object
Model_Name                                     object
version                                        object
loan_type                                      object
bad_rate                                       object
beta_cash_app_score_FPD0_t1_gini              float64
beta_cash_app_score_FPD10_t1_gini             float64
beta_cash_app_score_FPD30_t1_gini             float64
beta_cash_app_score_FSPD30_t1_gini            float64
beta_cash_app_score_FSTPD30_t1_gini           float64
Trench_category                                object
Model_display_name                             object
Product_type                                   object
dtype: object

In [764]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,beta_cash_app_score_FPD0_t1_gini,beta_cash_app_score_FPD10_t1_gini,beta_cash_app_score_FPD30_t1_gini,beta_cash_app_score_FSPD30_t1_gini,beta_cash_app_score_FSTPD30_t1_gini,Trench_category,Model_display_name,Product_type
0,2024-01-01,2024-01-07,Week,beta_cash_app_score,1.1.0,Overall,FPD0,-0.076923,NaN,NaN,NaN,NaN,Trench 1,apps_score_cash,CASH
1,2024-01-01,2024-01-31,Month,beta_cash_app_score,1.1.0,Overall,FPD0,0.255906,NaN,NaN,NaN,NaN,Trench 1,apps_score_cash,CASH
2,2024-01-08,2024-01-14,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.875000,NaN,NaN,NaN,NaN,Trench 1,apps_score_cash,CASH
3,2024-01-15,2024-01-21,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.430233,NaN,NaN,NaN,NaN,Trench 1,apps_score_cash,CASH
4,2024-01-22,2024-01-28,Week,beta_cash_app_score,1.1.0,Overall,FPD0,-0.051429,NaN,NaN,NaN,NaN,Trench 1,apps_score_cash,CASH


In [765]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.app_score_cash_t1_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=65b794d3-dcdb-4d2c-a83c-7edbcb867c88>

## Trench 2

## FPD0

## Test

In [766]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3249006,c9abffd9-b4b2-4b8e-807a-17538f7ef68b,60832490060017,0.468957,Trench 2,2025-10-22 19:45:37,2025-10-22,2025-10,Test,1,1,Quick
1,1242524,059f2133-161e-43bd-b5fa-0192e86991ab,60812425240046,0.458683,Trench 2,2025-10-01 21:40:29,2025-10-01,2025-10,Test,0,1,Quick
2,3190544,e5324fe5-34e5-4718-803c-84b1b2402b7d,60831905440011,0.418145,Trench 2,2025-10-05 12:24:42,2025-10-06,2025-10,Test,1,1,Quick
3,3304174,a8a455c5-a8fa-437f-bc73-b0b737ca9573,60833041740018,0.424525,Trench 2,2025-10-12 18:19:50,2025-10-13,2025-10,Test,1,1,Quick
4,3439183,fb07b68d-aa0d-4f74-bf11-6281e9b4644d,60834391830011,0.434722,Trench 2,2025-10-19 15:21:08,2025-10-20,2025-10,Test,0,1,Quick


In [767]:
df1 = dfd.copy()

## Train

In [768]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,2093691,7a4d835a-7666-42e2-9bed-1568fc75f915,60820936910011,0.370256,Trench 2,2024-01-30 10:25:21,2024-02-01,2024-01,Train,0,1,Flex
1,1334303,633480ab-01ea-4568-80e0-9af0d12aa1ad,60813343030011,0.413218,Trench 2,2024-01-28 13:38:00,2024-01-28,2024-01,Train,0,1,Quick
2,2248357,ed19ccdb-ee80-4c6d-a558-564b7633a366,60822483570016,0.483874,Trench 2,2024-01-06 13:03:38,2024-01-22,2024-01,Train,0,1,Quick
3,2042637,2ce0348a-d51d-4b24-bf57-223aa3b9380e,60820426370015,0.471704,Trench 2,2024-01-11 22:07:41,2024-01-12,2024-01,Train,1,1,Quick
4,2153045,3df227f9-ef87-4d7b-8f59-0cde62159295,60821530450012,0.531817,Trench 2,2024-01-28 12:41:44,2024-01-28,2024-01,Train,0,1,Quick


In [769]:
df2 = dfd.copy()

In [770]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8501 entries, 0 to 8500
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8501 non-null   object        
 1   digitalLoanAccountId   8501 non-null   object        
 2   loanAccountNumber      8501 non-null   object        
 3   beta_cash_app_score    8501 non-null   float64       
 4   trenchCategory         8501 non-null   object        
 5   appln_submit_datetime  8501 non-null   datetime64[us]
 6   disbursementdate       8501 non-null   dbdate        
 7   Application_month      8501 non-null   object        
 8   Data_selection         8501 non-null   object        
 9   deffpd0                8501 non-null   Int64         
 10  flg_mature_fpd0        8501 non-null   Int64         
 11  new_loan_type          8501 non-null   object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(

In [771]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [772]:
# df_concat.to_csv(r"beta_cash_app_scoretrench2fpd0.csv")

In [773]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [774]:
f0 = gini_results.copy()
f0.head()

,start_date,end_date,beta_cash_app_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,NaN,Week,beta_cash_app_score,1.1.0,FPD0,Overall
1,2024-01-01,2024-01-31,0.448521,Month,beta_cash_app_score,1.1.0,FPD0,Overall
2,2024-01-08,2024-01-14,0.593750,Week,beta_cash_app_score,1.1.0,FPD0,Overall
3,2024-01-15,2024-01-21,0.404762,Week,beta_cash_app_score,1.1.0,FPD0,Overall
4,2024-01-22,2024-01-28,0.769231,Week,beta_cash_app_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [775]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1254309,61288dc5-dd33-4cd1-80a5-d692279717dd,60812543090011,0.473778,Trench 2,2025-10-17 09:17:30,2025-10-17,2025-10,Test,0,1,Quick
1,1665724,1bfa7d7a-871f-45f5-87aa-35655c792488,60816657240022,0.330291,Trench 2,2025-10-10 12:07:37,2025-10-13,2025-10,Test,0,1,Quick
2,3472684,c169c74f-2cc6-415c-8785-7c8a957e2d52,60834726840018,0.496410,Trench 2,2025-10-09 16:21:21,2025-10-10,2025-10,Test,0,1,Quick
3,3569649,53a8cef3-7e62-4726-850f-d616880df6e2,60835696490014,0.475635,Trench 2,2025-09-29 21:25:24,2025-09-30,2025-09,Test,1,1,Quick
4,2998153,82ec07e4-9444-4d63-b845-d3accf9c09d3,60829981530013,0.494870,Trench 2,2025-10-10 09:30:00,2025-10-10,2025-10,Test,0,1,Quick


In [776]:
df1 = dfd.copy()

## Train

In [777]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1924680,dd16b13d-09a7-47d5-8324-3df87312edaa,60819246800021,0.457500,Trench 2,2024-01-02 13:16:04,2024-01-02,2024-01,Train,0,1,Quick
1,2244767,12c50ebc-9a23-4d23-a6e4-58bd8c03387e,60822447670014,0.510171,Trench 2,2024-01-21 04:46:54,2024-01-21,2024-01,Train,1,1,Quick
2,2165749,a042d84f-417f-4d68-8b1e-edc7406a6b0c,60821657490011,0.526756,Trench 2,2024-01-03 20:20:27,2024-01-09,2024-01,Train,0,1,Quick
3,2245925,e0a86e08-a36a-4058-8dd6-b0820fb6558e,60822459250014,0.456634,Trench 2,2024-01-11 18:26:16,2024-01-12,2024-01,Train,0,1,Quick
4,1059196,4a2a8965-991c-45de-8b6d-b80f705f1ba5,60810591960041,0.360220,Trench 2,2024-01-10 21:22:03,2024-01-10,2024-01,Train,0,1,Quick


In [778]:
df2 = dfd.copy()

In [779]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8389 entries, 0 to 8388
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8389 non-null   object        
 1   digitalLoanAccountId   8389 non-null   object        
 2   loanAccountNumber      8389 non-null   object        
 3   beta_cash_app_score    8389 non-null   float64       
 4   trenchCategory         8389 non-null   object        
 5   appln_submit_datetime  8389 non-null   datetime64[us]
 6   disbursementdate       8389 non-null   dbdate        
 7   Application_month      8389 non-null   object        
 8   Data_selection         8389 non-null   object        
 9   deffpd10               8389 non-null   Int64         
 10  flg_mature_fpd10       8389 non-null   Int64         
 11  new_loan_type          8389 non-null   object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(

In [780]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [781]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd0.csv")

In [782]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [783]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,beta_cash_app_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,NaN,Week,beta_cash_app_score,1.1.0,FPD10,Overall
1,2024-01-01,2024-01-31,0.429412,Month,beta_cash_app_score,1.1.0,FPD10,Overall
2,2024-01-08,2024-01-14,0.593750,Week,beta_cash_app_score,1.1.0,FPD10,Overall
3,2024-01-15,2024-01-21,0.400000,Week,beta_cash_app_score,1.1.0,FPD10,Overall
4,2024-01-22,2024-01-28,NaN,Week,beta_cash_app_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [784]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1346048,896716c5-e4e5-4b3c-9218-0b605d293c72,60813460480017,0.59739,Trench 2,2025-09-25 11:37:29,2025-09-25,2025-09,Test,0,1,Quick


In [785]:
df1 = dfd.copy()

## Train

In [787]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1064796,efb581ca-6dd1-4dd6-93d7-7784ae632585,60810647960028,0.410703,Trench 2,2024-01-04 20:32:02,2024-01-07,2024-01,Train,0,1,Flex
1,2174868,72b05611-fb40-4c68-a38a-c7dc3355fd94,60821748680017,0.446971,Trench 2,2024-01-05 18:05:33,2024-01-05,2024-01,Train,0,1,Quick
2,2279162,fd46f286-62e3-49eb-b468-d38d4f6fc875,60822791620024,0.328123,Trench 2,2024-01-03 17:39:17,2024-01-10,2024-01,Train,0,1,Flex
3,2257795,12a7f1a9-f67a-4f16-a2ba-7057cbce58ba,60822577950015,0.493148,Trench 2,2024-01-25 14:01:52,2024-01-25,2024-01,Train,0,1,Quick
4,1302809,a473b0b4-c355-4135-b4a5-2fa04de93417,60813028090018,0.401579,Trench 2,2024-01-21 19:13:13,2024-01-21,2024-01,Train,0,1,Quick


In [788]:
df2 = dfd.copy()

In [789]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8175 entries, 0 to 8174
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8175 non-null   object        
 1   digitalLoanAccountId   8175 non-null   object        
 2   loanAccountNumber      8175 non-null   object        
 3   beta_cash_app_score    8175 non-null   float64       
 4   trenchCategory         8175 non-null   object        
 5   appln_submit_datetime  8175 non-null   datetime64[us]
 6   disbursementdate       8175 non-null   dbdate        
 7   Application_month      8175 non-null   object        
 8   Data_selection         8175 non-null   object        
 9   deffpd30               8175 non-null   Int64         
 10  flg_mature_fpd30       8175 non-null   Int64         
 11  new_loan_type          8175 non-null   object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), object(

In [790]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [791]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd30.csv")

In [792]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [793]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,beta_cash_app_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,NaN,Week,beta_cash_app_score,1.1.0,FPD30,Overall
1,2024-01-01,2024-01-31,0.429412,Month,beta_cash_app_score,1.1.0,FPD30,Overall
2,2024-01-08,2024-01-14,0.593750,Week,beta_cash_app_score,1.1.0,FPD30,Overall
3,2024-01-15,2024-01-21,0.400000,Week,beta_cash_app_score,1.1.0,FPD30,Overall
4,2024-01-22,2024-01-28,NaN,Week,beta_cash_app_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [794]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [795]:
df1 = dfd.copy()

## Train

In [796]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,2260033,89f9e0f9-e063-4ce5-840a-aacd691b4716,60822600330017,0.329634,Trench 2,2024-01-27 17:21:51,2024-01-29,2024-01,Train,0,1,Quick
1,2187359,6950bf81-a2e2-4b2e-b1de-91f10011bc15,60821873590018,0.444481,Trench 2,2024-01-30 17:04:29,2024-01-30,2024-01,Train,0,1,Quick
2,1235435,df77ce69-9ee9-46cb-a6b9-ba39d7b744bd,60812354350019,0.510893,Trench 2,2024-01-25 11:16:34,2024-01-26,2024-01,Train,0,1,Quick
3,1763640,4e1dd5be-cfc4-4851-96a7-4f5c88cf3752,60817636400019,0.519990,Trench 2,2024-01-31 05:56:42,2024-02-01,2024-01,Train,0,1,Quick
4,2310005,972b6e7c-0003-4b24-8bc9-89d0f6274a41,60823100050014,0.393784,Trench 2,2024-01-27 19:38:29,2024-02-22,2024-01,Train,0,1,Quick


In [797]:
df2 = dfd.copy()

In [798]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7858 entries, 0 to 7857
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             7858 non-null   Int64         
 1   digitalLoanAccountId   7858 non-null   object        
 2   loanAccountNumber      7858 non-null   object        
 3   beta_cash_app_score    7858 non-null   float64       
 4   trenchCategory         7858 non-null   object        
 5   appln_submit_datetime  7858 non-null   datetime64[us]
 6   disbursementdate       7858 non-null   dbdate        
 7   Application_month      7858 non-null   object        
 8   Data_selection         7858 non-null   object        
 9   deffspd30              7858 non-null   Int64         
 10  flg_mature_fspd_30     7858 non-null   Int64         
 11  new_loan_type          7858 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [799]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [800]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fspd30.csv")

In [801]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [802]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,beta_cash_app_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,NaN,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
1,2024-01-01,2024-01-31,0.371429,Month,beta_cash_app_score,1.1.0,FSPD30,Overall
2,2024-01-08,2024-01-14,0.261905,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
3,2024-01-15,2024-01-21,0.523810,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
4,2024-01-22,2024-01-28,0.230769,Week,beta_cash_app_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [803]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [804]:
df1 = dfd.copy()

## Train

In [805]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,1763640,4e1dd5be-cfc4-4851-96a7-4f5c88cf3752,60817636400019,0.519990,Trench 2,2024-01-31 05:56:42,2024-02-01,2024-01,Train,0,1,Quick
1,2310005,972b6e7c-0003-4b24-8bc9-89d0f6274a41,60823100050014,0.393784,Trench 2,2024-01-27 19:38:29,2024-02-22,2024-01,Train,0,1,Quick
2,2264194,5f56693b-eaac-4cd0-83a9-28b8f1e46b7c,60822641940043,0.438526,Trench 2,2024-01-17 13:18:47,2024-01-17,2024-01,Train,0,1,Quick
3,2106535,04eaba91-8fd1-448d-b6e8-42b0470e22d6,60821065350012,0.524267,Trench 2,2024-01-20 14:21:31,2024-01-20,2024-01,Train,0,1,Quick
4,2286814,88ad469d-23b7-416f-8fa2-ed8c8dbcae88,60822868140012,0.494798,Trench 2,2024-01-26 14:19:19,2024-01-26,2024-01,Train,1,1,Quick


In [806]:
df2 = dfd.copy()

In [807]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7525 entries, 0 to 7524
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             7525 non-null   Int64         
 1   digitalLoanAccountId   7525 non-null   object        
 2   loanAccountNumber      7525 non-null   object        
 3   beta_cash_app_score    7525 non-null   float64       
 4   trenchCategory         7525 non-null   object        
 5   appln_submit_datetime  7525 non-null   datetime64[us]
 6   disbursementdate       7525 non-null   dbdate        
 7   Application_month      7525 non-null   object        
 8   Data_selection         7525 non-null   object        
 9   deffstpd30             7525 non-null   Int64         
 10  flg_mature_fstpd_30    7525 non-null   Int64         
 11  new_loan_type          7525 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [808]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [809]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fstpd30.csv")

In [810]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [811]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,beta_cash_app_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2024-01-01,2024-01-07,NaN,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
1,2024-01-01,2024-01-31,0.283333,Month,beta_cash_app_score,1.1.0,FSTPD30,Overall
2,2024-01-08,2024-01-14,0.274725,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
3,2024-01-15,2024-01-21,0.340909,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
4,2024-01-22,2024-01-28,0.040000,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [812]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'beta_cash_app_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini'], dtype=object)

In [813]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version','loan_type', 'bad_rate','beta_cash_app_score_FPD0_gini'
                     ,'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini']].copy()
final_df.rename(columns={'beta_cash_app_score_FPD0_gini':'beta_cash_app_score_FPD0_t2_gini'
                         , 'beta_cash_app_score_FPD10_gini':'beta_cash_app_score_FPD10_t2_gini'
                         , 'beta_cash_app_score_FPD30_gini':'beta_cash_app_score_FPD30_t2_gini'
                         , 'beta_cash_app_score_FSPD30_gini':'beta_cash_app_score_FSPD30_t2_gini'
                        , 'beta_cash_app_score_FSTPD30_gini':'beta_cash_app_score_FSTPD30_t2_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 2'
final_df['Model_display_name'] = 'apps_score_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                             datetime64[ns]
end_date                               datetime64[ns]
period                                         object
Model_Name                                     object
version                                        object
loan_type                                      object
bad_rate                                       object
beta_cash_app_score_FPD0_t2_gini              float64
beta_cash_app_score_FPD10_t2_gini             float64
beta_cash_app_score_FPD30_t2_gini             float64
beta_cash_app_score_FSPD30_t2_gini            float64
beta_cash_app_score_FSTPD30_t2_gini           float64
Trench_category                                object
Model_display_name                             object
Product_type                                   object
dtype: object

In [814]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,beta_cash_app_score_FPD0_t2_gini,beta_cash_app_score_FPD10_t2_gini,beta_cash_app_score_FPD30_t2_gini,beta_cash_app_score_FSPD30_t2_gini,beta_cash_app_score_FSTPD30_t2_gini,Trench_category,Model_display_name,Product_type
0,2024-01-01,2024-01-07,Week,beta_cash_app_score,1.1.0,Overall,FPD0,NaN,NaN,NaN,NaN,NaN,Trench 2,apps_score_cash,CASH
1,2024-01-01,2024-01-31,Month,beta_cash_app_score,1.1.0,Overall,FPD0,0.448521,NaN,NaN,NaN,NaN,Trench 2,apps_score_cash,CASH
2,2024-01-08,2024-01-14,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.593750,NaN,NaN,NaN,NaN,Trench 2,apps_score_cash,CASH
3,2024-01-15,2024-01-21,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.404762,NaN,NaN,NaN,NaN,Trench 2,apps_score_cash,CASH
4,2024-01-22,2024-01-28,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.769231,NaN,NaN,NaN,NaN,Trench 2,apps_score_cash,CASH


In [815]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.app_score_cash_t2_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2c6a1194-7b16-406c-8035-1d9ac148e322>

## Trench 3

## FPD0

## Test

In [816]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,3278009,0ceb9964-50ac-4730-800b-f9b823521128,60832780090053,0.459874,Trench 3,2025-10-10 14:22:24,2025-10-10,2025-10,Test,0,1,Quick
1,2796579,5cde1c43-59b6-421d-96c8-e3622fc5c9f4,60827965790023,0.445827,Trench 3,2025-10-15 20:52:18,2025-10-15,2025-10,Test,0,1,Quick
2,3528452,bd357d92-fe0d-4eb2-9e40-a7fb7b612b75,60835284520027,0.467884,Trench 3,2025-10-10 17:49:11,2025-10-10,2025-10,Test,0,1,Quick
3,3390896,935ad709-45f6-479e-a907-198f579c0b9c,60833908960031,0.466845,Trench 3,2025-10-17 21:31:06,2025-10-18,2025-10,Test,0,1,Quick
4,2826643,3f344a2a-fa6f-4916-8d53-b8aa6789925e,60828266430033,0.481588,Trench 3,2025-10-14 11:43:37,2025-10-14,2025-10,Test,1,1,Quick


In [817]:
dfd['deffpd0'].value_counts()

deffpd0
0    173
1     36
Name: count, dtype: Int64

In [818]:
df1 = dfd.copy()

## Train

In [819]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type
0,1277843,509740c8-7921-4254-9ed7-d2a52273fd98,60812778430027,0.437500,Trench 3,2023-01-18 11:52:21,2023-01-18,2023-01,Train,0,1,Flex-up
1,1172819,6daddd23-ac13-464b-aa7c-f598af683df7,60811728190021,0.459337,Trench 3,2023-02-16 00:51:52,2023-02-16,2023-02,Train,0,1,Flex-up
2,1747664,968c67a9-8e05-497f-b5f3-4e785c210401,60817476640021,0.463987,Trench 3,2023-01-13 20:37:21,2023-01-18,2023-01,Train,0,1,Flex
3,1851396,751d36ae-5136-418b-9ddb-736536f89300,60818513960027,0.479288,Trench 3,2023-02-10 21:30:11,2023-02-11,2023-02,Train,0,1,Flex
4,1630442,14968be8-af85-46d6-884c-3446fa9062a4,60816304420021,0.433320,Trench 3,2023-01-30 09:34:05,2023-01-30,2023-01,Train,0,1,Quick


In [820]:
dfd['new_loan_type'].value_counts()

new_loan_type
Flex-up    24280
Quick       5547
Flex         205
Name: count, dtype: int64

In [821]:
df2 = dfd.copy()

In [822]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30241 entries, 0 to 30240
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             30241 non-null  object        
 1   digitalLoanAccountId   30241 non-null  object        
 2   loanAccountNumber      30241 non-null  object        
 3   beta_cash_app_score    30241 non-null  float64       
 4   trenchCategory         30241 non-null  object        
 5   appln_submit_datetime  30241 non-null  datetime64[us]
 6   disbursementdate       30241 non-null  dbdate        
 7   Application_month      30241 non-null  object        
 8   Data_selection         30241 non-null  object        
 9   deffpd0                30241 non-null  Int64         
 10  flg_mature_fpd0        30241 non-null  Int64         
 11  new_loan_type          30241 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), objec

In [823]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [824]:
# df_concat.to_csv(r"beta_cash_app_scoretrench2fpd0.csv")

In [825]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

In [826]:
f0 = gini_results.copy()
f0.head().sort_values('start_date', ascending=False)

,start_date,end_date,beta_cash_app_score_FPD0_gini,period,Model_Name,version,bad_rate,loan_type
4,2023-01-23,2023-01-29,0.222222,Week,beta_cash_app_score,1.1.0,FPD0,Overall
3,2023-01-16,2023-01-22,0.121212,Week,beta_cash_app_score,1.1.0,FPD0,Overall
2,2023-01-09,2023-01-15,0.000000,Week,beta_cash_app_score,1.1.0,FPD0,Overall
1,2023-01-02,2023-01-08,0.050000,Week,beta_cash_app_score,1.1.0,FPD0,Overall
0,2023-01-01,2023-01-31,0.085346,Month,beta_cash_app_score,1.1.0,FPD0,Overall


## FPD10

## Test

In [827]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,2161307,b09215e1-3284-479d-8f99-349b292fcd61,60821613070021,0.445807,Trench 3,2025-10-06 09:25:01,2025-10-06,2025-10,Test,0,1,Quick
1,1419506,5de90a47-4100-48aa-8984-310792a44bd7,60814195060031,0.428412,Trench 3,2025-10-17 16:17:22,2025-10-17,2025-10,Test,0,1,Quick
2,1602810,7d42ddc3-801a-4890-a1ea-c5d0b10adb87,60816028100051,0.495787,Trench 3,2025-10-12 23:21:14,2025-10-13,2025-10,Test,0,1,Quick
3,2248295,b90d3125-4f3e-4538-9f58-313adb001808,60822482950043,0.505659,Trench 3,2025-10-09 09:50:34,2025-10-09,2025-10,Test,0,1,Quick
4,2750896,8ad9a58b-9413-4578-b1cc-0d3ce10e1c48,60827508960029,0.521959,Trench 3,2025-10-13 10:39:31,2025-10-13,2025-10,Test,0,1,Quick


In [828]:
df1 = dfd.copy()

## Train

In [829]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type
0,1683684,2ce8d9d1-0afd-4f91-836c-86e755714199,60816836840023,0.453130,Trench 3,2023-01-31 10:22:53,2023-01-31,2023-01,Train,0,1,Quick
1,1578392,ec50215c-a0dc-4779-bb30-280bf0227423,60815783920023,0.531674,Trench 3,2023-02-14 19:44:07,2023-02-15,2023-02,Train,1,1,Flex
2,1714115,6135a2ce-5f0b-4c7b-ab35-7c7fb747b147,60817141150021,0.456522,Trench 3,2023-02-28 12:57:32,2023-02-28,2023-02,Train,1,1,Quick
3,1651640,ae64c66d-d030-4b51-9916-7986ac5c8a1f,60816516400028,0.493639,Trench 3,2023-01-23 11:17:24,2023-01-23,2023-01,Train,0,1,Quick
4,1949158,95258325-e632-47f0-9a0e-29fd4c56498b,60819491580028,0.539017,Trench 3,2023-03-24 14:37:45,2023-03-30,2023-03,Train,0,1,Flex


In [830]:
df2 = dfd.copy()

In [831]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30013 entries, 0 to 30012
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             30013 non-null  object        
 1   digitalLoanAccountId   30013 non-null  object        
 2   loanAccountNumber      30013 non-null  object        
 3   beta_cash_app_score    30013 non-null  float64       
 4   trenchCategory         30013 non-null  object        
 5   appln_submit_datetime  30013 non-null  datetime64[us]
 6   disbursementdate       30013 non-null  dbdate        
 7   Application_month      30013 non-null  object        
 8   Data_selection         30013 non-null  object        
 9   deffpd10               30013 non-null  Int64         
 10  flg_mature_fpd10       30013 non-null  Int64         
 11  new_loan_type          30013 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), float64(1), objec

In [832]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [833]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd0.csv")

In [834]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

In [835]:
f1 = gini_results.copy()
f1.head()

,start_date,end_date,beta_cash_app_score_FPD10_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,-0.212418,Month,beta_cash_app_score,1.1.0,FPD10,Overall
1,2023-01-02,2023-01-08,0.083333,Week,beta_cash_app_score,1.1.0,FPD10,Overall
2,2023-01-09,2023-01-15,-0.444444,Week,beta_cash_app_score,1.1.0,FPD10,Overall
3,2023-01-16,2023-01-22,0.266667,Week,beta_cash_app_score,1.1.0,FPD10,Overall
4,2023-01-23,2023-01-29,-0.575758,Week,beta_cash_app_score,1.1.0,FPD10,Overall


## FPD30

## Test

In [836]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type


In [837]:
df1 = dfd.copy()

## Train

In [838]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type
0,1683684,2ce8d9d1-0afd-4f91-836c-86e755714199,60816836840023,0.453130,Trench 3,2023-01-31 10:22:53,2023-01-31,2023-01,Train,0,1,Quick
1,1578392,ec50215c-a0dc-4779-bb30-280bf0227423,60815783920023,0.531674,Trench 3,2023-02-14 19:44:07,2023-02-15,2023-02,Train,1,1,Flex
2,1714115,6135a2ce-5f0b-4c7b-ab35-7c7fb747b147,60817141150021,0.456522,Trench 3,2023-02-28 12:57:32,2023-02-28,2023-02,Train,0,1,Quick
3,1602755,10911797-0dab-4423-a685-36559f63e1c1,60816027550027,0.453596,Trench 3,2023-02-22 16:12:21,2023-02-23,2023-02,Train,0,1,Flex
4,1466518,228adc34-cc5e-4ddd-a1d0-2f48b1ce346f,60814665180021,0.521692,Trench 3,2023-01-30 10:59:26,2023-01-31,2023-01,Train,0,1,Flex-up


In [839]:
df2 = dfd.copy()

In [840]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28878 entries, 0 to 28877
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             28878 non-null  Int64         
 1   digitalLoanAccountId   28878 non-null  object        
 2   loanAccountNumber      28878 non-null  object        
 3   beta_cash_app_score    28878 non-null  float64       
 4   trenchCategory         28878 non-null  object        
 5   appln_submit_datetime  28878 non-null  datetime64[us]
 6   disbursementdate       28878 non-null  dbdate        
 7   Application_month      28878 non-null  object        
 8   Data_selection         28878 non-null  object        
 9   deffpd30               28878 non-null  Int64         
 10  flg_mature_fpd30       28878 non-null  Int64         
 11  new_loan_type          28878 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [841]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [842]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fpd30.csv")

In [843]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

In [844]:
f2 = gini_results.copy()
f2.head()

,start_date,end_date,beta_cash_app_score_FPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,-0.250000,Month,beta_cash_app_score,1.1.0,FPD30,Overall
1,2023-01-02,2023-01-08,0.083333,Week,beta_cash_app_score,1.1.0,FPD30,Overall
2,2023-01-09,2023-01-15,-0.444444,Week,beta_cash_app_score,1.1.0,FPD30,Overall
3,2023-01-16,2023-01-22,0.125000,Week,beta_cash_app_score,1.1.0,FPD30,Overall
4,2023-01-23,2023-01-29,-0.575758,Week,beta_cash_app_score,1.1.0,FPD30,Overall


## FSPD30

## Test

In [845]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type


In [846]:
df1 = dfd.copy()

## Train

In [847]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type
0,1172819,6daddd23-ac13-464b-aa7c-f598af683df7,60811728190021,0.459337,Trench 3,2023-02-16 00:51:52,2023-02-16,2023-02,Train,0,1,Flex-up
1,1747664,968c67a9-8e05-497f-b5f3-4e785c210401,60817476640021,0.463987,Trench 3,2023-01-13 20:37:21,2023-01-18,2023-01,Train,0,1,Flex
2,1419343,38b05c2f-7ccd-46dd-a0d8-f2cb55658c2b,60814193430022,0.526687,Trench 3,2023-03-31 20:59:21,2023-03-31,2023-03,Train,0,1,Flex-up
3,1576514,3a842548-6c9e-4e1d-92ff-ba2e0a97494f,60815765140027,0.499035,Trench 3,2023-01-30 17:19:24,2023-01-30,2023-01,Train,0,1,Flex
4,1514511,10ab5d6f-88f3-4f5e-bb9a-0666a5044413,60815145110026,0.464698,Trench 3,2023-01-17 08:13:06,2023-01-17,2023-01,Train,0,1,Flex-up


In [848]:
df2 = dfd.copy()

In [849]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25989 entries, 0 to 25988
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             25989 non-null  Int64         
 1   digitalLoanAccountId   25989 non-null  object        
 2   loanAccountNumber      25989 non-null  object        
 3   beta_cash_app_score    25989 non-null  float64       
 4   trenchCategory         25989 non-null  object        
 5   appln_submit_datetime  25989 non-null  datetime64[us]
 6   disbursementdate       25989 non-null  dbdate        
 7   Application_month      25989 non-null  object        
 8   Data_selection         25989 non-null  object        
 9   deffspd30              25989 non-null  Int64         
 10  flg_mature_fspd_30     25989 non-null  Int64         
 11  new_loan_type          25989 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [850]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [851]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fspd30.csv")

In [852]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

In [853]:
f3 = gini_results.copy()
f3.head()

,start_date,end_date,beta_cash_app_score_FSPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,-0.045082,Month,beta_cash_app_score,1.1.0,FSPD30,Overall
1,2023-01-02,2023-01-08,0.272727,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
2,2023-01-09,2023-01-15,0.083333,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
3,2023-01-16,2023-01-22,0.153846,Week,beta_cash_app_score,1.1.0,FSPD30,Overall
4,2023-01-23,2023-01-29,-0.180000,Week,beta_cash_app_score,1.1.0,FSPD30,Overall


## FSTPD30

## Test

In [855]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  coalesce(SAFE_CAST(JSON_VALUE(p.prediction_clean, "$.combined_score") AS Float64)) AS beta_cash_app_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory"))  trenchCategory
  from latest_request p
  left join model_run m on p.digitalLoanAccountId = m.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type


In [856]:
df1 = dfd.copy()

## Train

In [857]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction beta_cash_app_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.beta_cash_app_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and (lower(loanmaster.new_loan_type) not like '%sil%' and lower(loanmaster.new_loan_type) not like '%big%')
  and r.beta_cash_app_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,loanAccountNumber,beta_cash_app_score,trenchCategory,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type
0,1651640,ae64c66d-d030-4b51-9916-7986ac5c8a1f,60816516400028,0.493639,Trench 3,2023-01-23 11:17:24,2023-01-23,2023-01,Train,0,1,Quick
1,1622068,2b36cc45-660c-4ac2-86c1-b96ae5027b31,60816220680021,0.442998,Trench 3,2023-03-28 09:18:46,2023-03-29,2023-03,Train,1,1,Quick
2,1576170,f457c1f0-fb43-4734-858c-cd46ced68019,60815761700023,0.437421,Trench 3,2023-02-15 05:39:22,2023-02-15,2023-02,Train,0,1,Quick
3,1419074,5ad907e9-5038-4a16-9e94-a33080ce35ea,60814190740021,0.504198,Trench 3,2023-01-31 00:36:06,2023-01-31,2023-01,Train,1,1,Quick
4,1128153,73a29cd8-d451-4f5d-8a98-30db74111e98,60811281530039,0.533506,Trench 3,2023-02-11 13:37:27,2023-02-11,2023-02,Train,0,1,Flex-up


In [858]:
df2 = dfd.copy()

In [859]:
df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22709 entries, 0 to 22708
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             22709 non-null  Int64         
 1   digitalLoanAccountId   22709 non-null  object        
 2   loanAccountNumber      22709 non-null  object        
 3   beta_cash_app_score    22709 non-null  float64       
 4   trenchCategory         22709 non-null  object        
 5   appln_submit_datetime  22709 non-null  datetime64[us]
 6   disbursementdate       22709 non-null  dbdate        
 7   Application_month      22709 non-null  object        
 8   Data_selection         22709 non-null  object        
 9   deffstpd30             22709 non-null  Int64         
 10  flg_mature_fstpd_30    22709 non-null  Int64         
 11  new_loan_type          22709 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\3061930065.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


In [860]:
df_concat['beta_cash_app_score'] = pd.to_numeric(df_concat['beta_cash_app_score'], errors='coerce')

In [861]:
# df_concat.to_csv(r"beta_cash_app_scoretrench1fstpd30.csv")

In [862]:
# gini_results = calculate_periodic_gini(df_concat, 'beta_cash_app_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'beta_cash_app_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

In [863]:
f4 = gini_results.copy()
f4.head()

,start_date,end_date,beta_cash_app_score_FSTPD30_gini,period,Model_Name,version,bad_rate,loan_type
0,2023-01-01,2023-01-31,0.074074,Month,beta_cash_app_score,1.1.0,FSTPD30,Overall
1,2023-01-02,2023-01-08,0.300000,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
2,2023-01-09,2023-01-15,0.357143,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
3,2023-01-16,2023-01-22,0.151515,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall
4,2023-01-23,2023-01-29,0.031746,Week,beta_cash_app_score,1.1.0,FSTPD30,Overall


## combining the dataframe

In [864]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'beta_cash_app_score_FPD0_gini',
       'period', 'Model_Name', 'version', 'bad_rate', 'loan_type',
       'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini'], dtype=object)

In [865]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type','bad_rate','beta_cash_app_score_FPD0_gini'
                     ,'beta_cash_app_score_FPD10_gini', 'beta_cash_app_score_FPD30_gini',
       'beta_cash_app_score_FSPD30_gini',
       'beta_cash_app_score_FSTPD30_gini']].copy()
final_df.rename(columns={'beta_cash_app_score_FPD0_gini':'beta_cash_app_score_FPD0_t3_gini'
                         , 'beta_cash_app_score_FPD10_gini':'beta_cash_app_score_FPD10_t3_gini'
                         , 'beta_cash_app_score_FPD30_gini':'beta_cash_app_score_FPD30_t3_gini'
                         , 'beta_cash_app_score_FSPD30_gini':'beta_cash_app_score_FSPD30_t3_gini'
                        , 'beta_cash_app_score_FSTPD30_gini':'beta_cash_app_score_FSTPD30_t3_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 3'
final_df['Model_display_name'] = 'apps_score_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

start_date                             datetime64[ns]
end_date                               datetime64[ns]
period                                         object
Model_Name                                     object
version                                        object
loan_type                                      object
bad_rate                                       object
beta_cash_app_score_FPD0_t3_gini              float64
beta_cash_app_score_FPD10_t3_gini             float64
beta_cash_app_score_FPD30_t3_gini             float64
beta_cash_app_score_FSPD30_t3_gini            float64
beta_cash_app_score_FSTPD30_t3_gini           float64
Trench_category                                object
Model_display_name                             object
Product_type                                   object
dtype: object

In [866]:
final_df.head()

,start_date,end_date,period,Model_Name,version,loan_type,bad_rate,beta_cash_app_score_FPD0_t3_gini,beta_cash_app_score_FPD10_t3_gini,beta_cash_app_score_FPD30_t3_gini,beta_cash_app_score_FSPD30_t3_gini,beta_cash_app_score_FSTPD30_t3_gini,Trench_category,Model_display_name,Product_type
0,2023-01-01,2023-01-31,Month,beta_cash_app_score,1.1.0,Overall,FPD0,0.085346,NaN,NaN,NaN,NaN,Trench 3,apps_score_cash,CASH
1,2023-01-02,2023-01-08,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.050000,NaN,NaN,NaN,NaN,Trench 3,apps_score_cash,CASH
2,2023-01-09,2023-01-15,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.000000,NaN,NaN,NaN,NaN,Trench 3,apps_score_cash,CASH
3,2023-01-16,2023-01-22,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.121212,NaN,NaN,NaN,NaN,Trench 3,apps_score_cash,CASH
4,2023-01-23,2023-01-29,Week,beta_cash_app_score,1.1.0,Overall,FPD0,0.222222,NaN,NaN,NaN,NaN,Trench 3,apps_score_cash,CASH


In [867]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.app_score_cash_t3_v1_gini5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=035a3541-be48-4227-8a1c-6ed79eece347>

# Beta-Cash-Stack-Model

## Trench 1

## FPD0

In [868]:
## Trench 1
## Test

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 1'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)

f0 = gini_results.copy()
print(f0.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16857 entries, 0 to 16856
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16857 non-null  object        
 1   digitalLoanAccountId   16857 non-null  object        
 2   loanAccountNumber      16857 non-null  object        
 3   Beta_cash_stack_score  16857 non-null  object        
 4   trenchCategory         16857 non-null  object        
 5   appln_submit_datetime  16857 non-null  datetime64[us]
 6   disbursementdate       16857 non-null  dbdate        
 7   Application_month      16857 non-null  object        
 8   Data_selection         16857 non-null  object        
 9   deffpd0                16857 non-null  Int64         
 10  flg_mature_fpd0        16857 non-null  Int64         
 11  new_loan_type          16857 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

## FPD10

In [869]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 1'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

f1 = gini_results.copy()
print(f1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16347 entries, 0 to 16346
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             16347 non-null  object        
 1   digitalLoanAccountId   16347 non-null  object        
 2   loanAccountNumber      16347 non-null  object        
 3   Beta_cash_stack_score  16347 non-null  object        
 4   trenchCategory         16347 non-null  object        
 5   appln_submit_datetime  16347 non-null  datetime64[us]
 6   disbursementdate       16347 non-null  dbdate        
 7   Application_month      16347 non-null  object        
 8   Data_selection         16347 non-null  object        
 9   deffpd10               16347 non-null  Int64         
 10  flg_mature_fpd10       16347 non-null  Int64         
 11  new_loan_type          16347 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

## FPD30

In [870]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffpd30', 
    'FPD30',
    product_column='new_loan_type'
)

f2 = gini_results.copy()
print(f2.head())

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\511487448.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15513 entries, 0 to 15512
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             15513 non-null  Int64         
 1   digitalLoanAccountId   15513 non-null  object        
 2   loanAccountNumber      15513 non-null  object        
 3   Beta_cash_stack_score  15513 non-null  float64       
 4   trenchCategory         15513 non-null  object        
 5   appln_submit_datetime  15513 non-null  datetime64[us]
 6   disbursementdate       15513 non-null  dbdate        
 7   Application_month      15513 non-null  object        
 8   Data_selection         15513 non-null  object        
 9   deffpd30               15513 non-null  Int64         
 10  flg_mature_fpd30       15513 non-null  Int64         
 11  new_loan_type          15513 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

## FSPD30

In [871]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffspd30', 
    'FSPD30',
    product_column='new_loan_type'
)

f3 = gini_results.copy()
print(f3.head())

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\907586233.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14297 entries, 0 to 14296
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             14297 non-null  Int64         
 1   digitalLoanAccountId   14297 non-null  object        
 2   loanAccountNumber      14297 non-null  object        
 3   Beta_cash_stack_score  14297 non-null  float64       
 4   trenchCategory         14297 non-null  object        
 5   appln_submit_datetime  14297 non-null  datetime64[us]
 6   disbursementdate       14297 non-null  dbdate        
 7   Application_month      14297 non-null  object        
 8   Data_selection         14297 non-null  object        
 9   deffspd30              14297 non-null  Int64         
 10  flg_mature_fspd_30     14297 non-null  Int64         
 11  new_loan_type          14297 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

## FSTPD30

In [872]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 1'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffstpd30', 
    'FSTPD30',
    product_column='new_loan_type'
)

f4 = gini_results.copy()
print(f4.head())

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\1832418571.py:133: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13104 entries, 0 to 13103
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             13104 non-null  Int64         
 1   digitalLoanAccountId   13104 non-null  object        
 2   loanAccountNumber      13104 non-null  object        
 3   Beta_cash_stack_score  13104 non-null  float64       
 4   trenchCategory         13104 non-null  object        
 5   appln_submit_datetime  13104 non-null  datetime64[us]
 6   disbursementdate       13104 non-null  dbdate        
 7   Application_month      13104 non-null  object        
 8   Data_selection         13104 non-null  object        
 9   deffstpd30             13104 non-null  Int64         
 10  flg_mature_fstpd_30    13104 non-null  Int64         
 11  new_loan_type          13104 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

## combining the dataframe

In [873]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version','loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','Beta_cash_stack_score_FPD0_gini',
                     'Beta_cash_stack_score_FPD10_gini',
       'Beta_cash_stack_score_FPD30_gini',
       'Beta_cash_stack_score_FSPD30_gini',
       'Beta_cash_stack_score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_cash_stack_score_FPD0_gini':'Beta_cash_stack_score_FPD0_t1_gini'
                         , 'Beta_cash_stack_score_FPD10_gini':'Beta_cash_stack_score_FPD10_t1_gini'
                         , 'Beta_cash_stack_score_FPD30_gini':'Beta_cash_stack_score_FPD30_t1_gini'
                         , 'Beta_cash_stack_score_FSPD30_gini':'Beta_cash_stack_score_FSPD30_t1_gini'
                        , 'Beta_cash_stack_score_FSTPD30_gini':'Beta_cash_stack_score_FSTPD30_t1_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 1'
final_df['Model_display_name'] = 'beta_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

final_df.head()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_stack_model_cash_t1_v1_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e2c4edbc-d502-4e48-b4cc-c308f414a12d>

## Trench 2

In [878]:
## Trench 2
## Test

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 2'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffpd0', 
    'FPD0',
    product_column='new_loan_type'
)   

f0 = gini_results.copy()
print(f0.head())

## FPD10

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,  
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 2'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat, 
    'Beta_cash_stack_score', 
    'deffpd10', 
    'FPD10',
    product_column='new_loan_type'
)

f1 = gini_results.copy()
print(f1.head())

## FPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffpd30',
    'FPD30',
    product_column='new_loan_type'
)

f2 = gini_results.copy()
print(f2.head())

## FSPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffspd30',
    'FSPD30',
    product_column='new_loan_type'
)

f3 = gini_results.copy()
print(f3.head())

## FSTPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 2'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffstpd30',
    'FSTPD30',
    product_column='new_loan_type'
)

f4 = gini_results.copy()
print(f4.head())

import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','Beta_cash_stack_score_FPD0_gini',
                     'Beta_cash_stack_score_FPD10_gini',
       'Beta_cash_stack_score_FPD30_gini',
       'Beta_cash_stack_score_FSPD30_gini',
       'Beta_cash_stack_score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_cash_stack_score_FPD0_gini':'Beta_cash_stack_score_FPD0_t2_gini'
                         , 'Beta_cash_stack_score_FPD10_gini':'Beta_cash_stack_score_FPD10_t2_gini'
                         , 'Beta_cash_stack_score_FPD30_gini':'Beta_cash_stack_score_FPD30_t2_gini'
                         , 'Beta_cash_stack_score_FSPD30_gini':'Beta_cash_stack_score_FSPD30_t2_gini'
                        , 'Beta_cash_stack_score_FSTPD30_gini':'Beta_cash_stack_score_FSTPD30_t2_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 2'
final_df['Model_display_name'] = 'beta_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

final_df.head()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_stack_model_cash_t2_v1_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11621 entries, 0 to 11620
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11621 non-null  object        
 1   digitalLoanAccountId   11621 non-null  object        
 2   loanAccountNumber      11621 non-null  object        
 3   Beta_cash_stack_score  11621 non-null  object        
 4   trenchCategory         11621 non-null  object        
 5   appln_submit_datetime  11621 non-null  datetime64[us]
 6   disbursementdate       11621 non-null  dbdate        
 7   Application_month      11621 non-null  object        
 8   Data_selection         11621 non-null  object        
 9   deffpd0                11621 non-null  Int64         
 10  flg_mature_fpd0        11621 non-null  Int64         
 11  new_loan_type          11621 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\685199837.py:593: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10405 entries, 0 to 10404
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10405 non-null  Int64         
 1   digitalLoanAccountId   10405 non-null  object        
 2   loanAccountNumber      10405 non-null  object        
 3   Beta_cash_stack_score  10405 non-null  float64       
 4   trenchCategory         10405 non-null  object        
 5   appln_submit_datetime  10405 non-null  datetime64[us]
 6   disbursementdate       10405 non-null  dbdate        
 7   Application_month      10405 non-null  object        
 8   Data_selection         10405 non-null  object        
 9   deffspd30              10405 non-null  Int64         
 10  flg_mature_fspd_30     10405 non-null  Int64         
 11  new_loan_type          10405 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\685199837.py:745: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)
C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9678 entries, 0 to 9677
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9678 non-null   Int64         
 1   digitalLoanAccountId   9678 non-null   object        
 2   loanAccountNumber      9678 non-null   object        
 3   Beta_cash_stack_score  9678 non-null   float64       
 4   trenchCategory         9678 non-null   object        
 5   appln_submit_datetime  9678 non-null   datetime64[us]
 6   disbursementdate       9678 non-null   dbdate        
 7   Application_month      9678 non-null   object        
 8   Data_selection         9678 non-null   object        
 9   deffstpd30             9678 non-null   Int64         
 10  flg_mature_fstpd_30    9678 non-null   Int64         
 11  new_loan_type          9678 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=85003784-a223-461c-ab79-b9b0e7672ce4>

## Trench 3

In [877]:
## Trench 3
## Test

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd0,
  del.flg_mature_fpd0,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd0 = 1
  and r.trenchCategory = 'Trench 3'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd0', 'FPD0')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffpd0',
    'FPD0',
    product_column='new_loan_type'
)

f0 = gini_results.copy()
print(f0.head())

## FPD10

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd10 = 1
  and r.trenchCategory = 'Trench 3'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd10', 'FPD10')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffpd10',
    'FPD10',
    product_column='new_loan_type'
)

f1 = gini_results.copy()
print(f1.head())

## FPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fpd30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffpd30', 'FPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffpd30',
    'FPD30',
    product_column='new_loan_type'
)

f2 = gini_results.copy()
print(f2.head())

## FSPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
    loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fspd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffspd30', 'FSPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffspd30',
    'FSPD30',
    product_column='new_loan_type'
)

f3 = gini_results.copy()
print(f3.head())

## FSTPD30

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
latest_request as (
select * from parsed
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelDisplayName ORDER BY start_time DESC ) = 1),

model_run as (
select customerId,digitalLoanAccountId,modelName, publish_time,
REPLACE(REPLACE(requestPayload, "'", '"'), "None", "null") AS requestPayload_clean
from `prj-prod-dataplatform.audit_balance.ml_request_details` 
WHERE modelName = 'Beta-Cash-Model-response'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId, digitalLoanAccountId,modelName ORDER BY publish_time DESC ) = 1)
, 
  modelname as (
  select p.customerId,
  p.digitalLoanAccountId,
  start_time,
  prediction AS Beta_cash_stack_score,
  coalesce(p.trenchCategory, JSON_VALUE(m.requestPayload_clean, "$.predictions.trenchCategory")) AS trenchCategory,
  from latest_request p
  left join model_run m on m.digitalLoanAccountId = p.digitalLoanAccountId
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
"""

dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df1 = dfd.copy()

## Train

sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
and modelVersionId = 'v1'
),
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction Beta_cash_stack_score,
  trenchCategory
  from parsed
  )
  ,
deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data)
  select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.Beta_cash_stack_score,
  r.trenchCategory,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Beta_cash_stack_score is not null
  and del.flg_mature_fstpd_30 = 1
  and r.trenchCategory = 'Trench 3'
  ;
  """
  
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

df2 = dfd.copy()

df_concat = pd.concat([df2, df1], ignore_index=True)
df_concat.info()

df_concat['Beta_cash_stack_score'] = pd.to_numeric(df_concat['Beta_cash_stack_score'], errors='coerce')

# gini_results = calculate_periodic_gini(df_concat, 'Beta_cash_stack_score', 'deffstpd30', 'FSTPD30')
gini_results = calculate_periodic_gini_producttype(
    df_concat,
    'Beta_cash_stack_score',
    'deffstpd30',
    'FSTPD30',
    product_column='new_loan_type'
)

f4 = gini_results.copy()
print(f4.head())

import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'loan_type', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'loan_type', 'bad_rate','Beta_cash_stack_score_FPD0_gini',
                     'Beta_cash_stack_score_FPD10_gini',
       'Beta_cash_stack_score_FPD30_gini',
       'Beta_cash_stack_score_FSPD30_gini',
       'Beta_cash_stack_score_FSTPD30_gini']].copy()
final_df.rename(columns={'Beta_cash_stack_score_FPD0_gini':'Beta_cash_stack_score_FPD0_t3_gini'
                         , 'Beta_cash_stack_score_FPD10_gini':'Beta_cash_stack_score_FPD10_t3_gini'
                         , 'Beta_cash_stack_score_FPD30_gini':'Beta_cash_stack_score_FPD30_t3_gini'
                         , 'Beta_cash_stack_score_FSPD30_gini':'Beta_cash_stack_score_FSPD30_t3_gini'
                        , 'Beta_cash_stack_score_FSTPD30_gini':'Beta_cash_stack_score_FSTPD30_t3_gini', 
                        }, inplace = True)
final_df['Trench_category'] = 'Trench 3'
final_df['Model_display_name'] = 'beta_stack_model_cash'
final_df['Product_type'] = 'CASH'
final_df.dtypes

final_df.head()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_stack_model_cash_t3_v1_gini_v5"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11439 entries, 0 to 11438
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             11439 non-null  object        
 1   digitalLoanAccountId   11439 non-null  object        
 2   loanAccountNumber      11439 non-null  object        
 3   Beta_cash_stack_score  11439 non-null  object        
 4   trenchCategory         11439 non-null  object        
 5   appln_submit_datetime  11439 non-null  datetime64[us]
 6   disbursementdate       11439 non-null  dbdate        
 7   Application_month      11439 non-null  object        
 8   Data_selection         11439 non-null  object        
 9   deffpd0                11439 non-null  Int64         
 10  flg_mature_fpd0        11439 non-null  Int64         
 11  new_loan_type          11439 non-null  object        
dtypes: Int64(2), datetime64[us](1), dbdate(1), object(8)
memory 

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\24486679.py:441: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10780 entries, 0 to 10779
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             10780 non-null  Int64         
 1   digitalLoanAccountId   10780 non-null  object        
 2   loanAccountNumber      10780 non-null  object        
 3   Beta_cash_stack_score  10780 non-null  float64       
 4   trenchCategory         10780 non-null  object        
 5   appln_submit_datetime  10780 non-null  datetime64[us]
 6   disbursementdate       10780 non-null  dbdate        
 7   Application_month      10780 non-null  object        
 8   Data_selection         10780 non-null  object        
 9   deffpd30               10780 non-null  Int64         
 10  flg_mature_fpd30       10780 non-null  Int64         
 11  new_loan_type          10780 non-null  object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), objec

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\24486679.py:593: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             9589 non-null   Int64         
 1   digitalLoanAccountId   9589 non-null   object        
 2   loanAccountNumber      9589 non-null   object        
 3   Beta_cash_stack_score  9589 non-null   float64       
 4   trenchCategory         9589 non-null   object        
 5   appln_submit_datetime  9589 non-null   datetime64[us]
 6   disbursementdate       9589 non-null   dbdate        
 7   Application_month      9589 non-null   object        
 8   Data_selection         9589 non-null   object        
 9   deffspd30              9589 non-null   Int64         
 10  flg_mature_fspd_30     9589 non-null   Int64         
 11  new_loan_type          9589 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_17708\24486679.py:745: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df2, df1], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8355 entries, 0 to 8354
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customerId             8355 non-null   Int64         
 1   digitalLoanAccountId   8355 non-null   object        
 2   loanAccountNumber      8355 non-null   object        
 3   Beta_cash_stack_score  8355 non-null   float64       
 4   trenchCategory         8355 non-null   object        
 5   appln_submit_datetime  8355 non-null   datetime64[us]
 6   disbursementdate       8355 non-null   dbdate        
 7   Application_month      8355 non-null   object        
 8   Data_selection         8355 non-null   object        
 9   deffstpd30             8355 non-null   Int64         
 10  flg_mature_fstpd_30    8355 non-null   Int64         
 11  new_loan_type          8355 non-null   object        
dtypes: Int64(3), datetime64[us](1), dbdate(1), float64(1), object(

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5d0fbecf-e628-40e0-bcf7-8f0a56d527fc>

# End